<a href="https://colab.research.google.com/github/avd3v/MiBand2/blob/master/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [5]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [6]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [9]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import Dataset
# dataset = load_dataset("mlabonne/FineTome-100k", split = "train")
dataset_conversations = [[{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аббсяов Руслан Анатольевич'}, {'from': 'gpt', 'value': 'Аббсяов Руслан Анатольевич работает в организации Волшебный Доктор на должности\n fЗаместитель руководителя транспортной службы дирекции, в филиале Управление (Автопарк дирекции, Аппарат дирекции). Дата начала работы в организации: 2008-09-16. Дата рождения: 1976-09-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Абоясбва Валентина Алексеевна'}, {'from': 'gpt', 'value': 'Абоясбва Валентина Алексеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2015-12-09. Дата рождения: 1970-05-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Абдуилн Ринат Юратович'}, {'from': 'gpt', 'value': 'Абдуилн Ринат Юратович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2017-12-06. Дата рождения: 1980-05-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Абауллдев Мухаммад Мухаммадович'}, {'from': 'gpt', 'value': 'Абауллдев Мухаммад Мухаммадович работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2025-01-23. Дата рождения: 1995-04-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Абдлулаева Рояла Шакировна'}, {'from': 'gpt', 'value': 'Абдлулаева Рояла Шакировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2023-01-09. Дата рождения: 2002-06-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Абдвлоуа Ляйсан Рустамовна'}, {'from': 'gpt', 'value': 'Абдвлоуа Ляйсан Рустамовна работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2022-09-19. Дата рождения: 1995-04-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Абравома Наталия Михайловна'}, {'from': 'gpt', 'value': 'Абравома Наталия Михайловна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2019-05-13. Дата рождения: 1979-12-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Абармова Стелла Гургеновна'}, {'from': 'gpt', 'value': 'Абармова Стелла Гургеновна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-03-21. Дата рождения: 1985-01-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ашыбева Светлана Алексеевна'}, {'from': 'gpt', 'value': 'Ашыбева Светлана Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2021-05-17. Дата рождения: 1987-06-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Авгаян Сатеник Ашотовна'}, {'from': 'gpt', 'value': 'Авгаян Сатеник Ашотовна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-11-05. Дата рождения: 1991-04-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Авьдесеян Анжела Семеновна'}, {'from': 'gpt', 'value': 'Авьдесеян Анжела Семеновна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Экспресс-лаборатория, Экспресс-лаборатория). Дата начала работы в организации: 2015-12-16. Дата рождения: 1988-03-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аверчикина Любовь Александровна'}, {'from': 'gpt', 'value': 'Аверчикина Любовь Александровна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф01 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-06-06. Дата рождения: 1983-08-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аверечнков Денис Сергеевич'}, {'from': 'gpt', 'value': 'Аверечнков Денис Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-онколог, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-11-05. Дата рождения: 1982-07-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Агасивеа Анжелика Робертовна'}, {'from': 'gpt', 'value': 'Агасивеа Анжелика Робертовна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2014-05-06. Дата рождения: 1969-06-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Агафовона Снежанна Владимировна'}, {'from': 'gpt', 'value': 'Агафовона Снежанна Владимировна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2018-10-15. Дата рождения: 1983-03-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Агевнина Виктория Павловна'}, {'from': 'gpt', 'value': 'Агевнина Виктория Павловна работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф05 (Администрация, Администрация). Дата начала работы в организации: 2017-06-14. Дата рождения: 1975-08-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аеегва Екатерина Алексеевна'}, {'from': 'gpt', 'value': 'Аеегва Екатерина Алексеевна работает в организации Волшебный Доктор на должности\n fРуководитель интернет-проектов, в филиале Управление (Отдел интернет-проектов, Департамент маркетинга и рекламы). Дата начала работы в организации: 2018-02-20. Дата рождения: 1989-07-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Агвееа Юлия Викторовна'}, {'from': 'gpt', 'value': 'Агвееа Юлия Викторовна работает в организации Волшебный Доктор на должности\n fМенеджер по экстренной госпитализации, в филиале ГЦ-Ф15 (Отдел мониторинга и экстренной госпитализации, Коммерческий департамент ГЦ). Дата начала работы в организации: 2024-03-18. Дата рождения: 1991-03-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Агейвеча Ольга Владимировна'}, {'from': 'gpt', 'value': 'Агейвеча Ольга Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стерилизационной, в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2023-05-18. Дата рождения: 1976-07-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Агивора Зарема Гидовна'}, {'from': 'gpt', 'value': 'Агивора Зарема Гидовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф12 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2015-09-28. Дата рождения: 1983-11-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аграновсакя Ольга Николаевна'}, {'from': 'gpt', 'value': 'Аграновсакя Ольга Николаевна работает в организации Волшебный Доктор на должности\n fНачальник отдела подбора и адаптации персонала, в филиале Управление (Отдел подбора и адаптации, Департамент по работе с персоналом). Дата начала работы в организации: 2024-07-23. Дата рождения: 1974-08-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Адаомва Имара Габибуллаховна'}, {'from': 'gpt', 'value': 'Адаомва Имара Габибуллаховна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале ГЦ-Ф15 (Центр гастроэнтерологии, Центр гастроэнтерологии). Дата начала работы в организации: 2024-07-22. Дата рождения: 1997-12-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Адмаян Нуне Александровна'}, {'from': 'gpt', 'value': 'Адмаян Нуне Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2011-07-27. Дата рождения: 1969-07-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аоилдва Аминат Ахмедхановна'}, {'from': 'gpt', 'value': 'Аоилдва Аминат Ахмедхановна работает в организации Волшебный Доктор на должности\n fВедущий специалист отдела подбора и адаптации, в филиале Управление (Отдел подбора и адаптации, Департамент по работе с персоналом). Дата начала работы в организации: 2024-08-15. Дата рождения: 2000-04-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ааарсзнов Олег Сергеевич'}, {'from': 'gpt', 'value': 'Ааарсзнов Олег Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-эндоскопист, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2021-06-01. Дата рождения: 1990-11-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Азвмоыа Рамиля Рашидовна'}, {'from': 'gpt', 'value': 'Азвмоыа Рамиля Рашидовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-04-22. Дата рождения: 1970-02-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Айботяв Дмитрий Тахирович'}, {'from': 'gpt', 'value': 'Айботяв Дмитрий Тахирович работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф10 (Администрация, Администрация). Дата начала работы в организации: 2021-03-09. Дата рождения: 1980-01-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аебашкв Кемал Ахматович'}, {'from': 'gpt', 'value': 'Аебашкв Кемал Ахматович работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-12-09. Дата рождения: 1996-02-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Амикова Валентина Владимировна'}, {'from': 'gpt', 'value': 'Амикова Валентина Владимировна работает в организации Волшебный Доктор на должности\n fБуфетчица, в филиале ГЦ-Ф15 (Служба сервиса, Стационар). Дата начала работы в организации: 2023-11-16. Дата рождения: 1965-08-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Абшка Наала Робертовна'}, {'from': 'gpt', 'value': 'Абшка Наала Робертовна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2024-10-14. Дата рождения: 1999-01-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Албанвота Ксения Александровна'}, {'from': 'gpt', 'value': 'Албанвота Ксения Александровна работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2022-08-29. Дата рождения: 1983-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Алвееа Надежда Михайловна'}, {'from': 'gpt', 'value': 'Алвееа Надежда Михайловна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф06 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2018-12-06. Дата рождения: 1976-08-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Алеосандркв Алексей Дмитриевич'}, {'from': 'gpt', 'value': 'Алеосандркв Алексей Дмитриевич работает в организации Волшебный Доктор на должности\n fВрач-бактериолог, в филиале Лаборатория (Отделение микробиологии, Отделение микробиологии). Дата начала работы в организации: 2013-12-11. Дата рождения: 1954-08-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Алскееев Вячеслав Вячеславович'}, {'from': 'gpt', 'value': 'Алскееев Вячеслав Вячеславович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2021-02-24. Дата рождения: 1986-04-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Алекесев Михаил Сергеевич'}, {'from': 'gpt', 'value': 'Алекесев Михаил Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2019-01-09. Дата рождения: 1960-11-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Алексевеа Ольга Александровна'}, {'from': 'gpt', 'value': 'Алексевеа Ольга Александровна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2024-07-01. Дата рождения: 1969-12-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Алексеева Светлана Юрьевна'}, {'from': 'gpt', 'value': 'Алексеева Светлана Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2019-11-11. Дата рождения: 1993-06-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Алексевцена Татьяна Сергеевна'}, {'from': 'gpt', 'value': 'Алексевцена Татьяна Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2011-03-24. Дата рождения: 1984-02-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аленнивока Елена Андреевна'}, {'from': 'gpt', 'value': 'Аленнивока Елена Андреевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2025-02-03. Дата рождения: 1991-08-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Алкщенео Светлана Михайловна'}, {'from': 'gpt', 'value': 'Алкщенео Светлана Михайловна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2015-05-25. Дата рождения: 1967-05-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аилев Мухаммат Ибрагимович'}, {'from': 'gpt', 'value': 'Аилев Мухаммат Ибрагимович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-08-12. Дата рождения: 1995-10-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аеилва Айшат Каплановна'}, {'from': 'gpt', 'value': 'Аеилва Айшат Каплановна работает в организации Волшебный Доктор на должности\n fВрач-детский кардиолог, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2024-08-12. Дата рождения: 1996-10-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аитунлна Анастасия Сергеевна'}, {'from': 'gpt', 'value': 'Аитунлна Анастасия Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2024-06-10. Дата рождения: 1996-07-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Авьдикеела Гульнара Тюлегеновна'}, {'from': 'gpt', 'value': 'Авьдикеела Гульнара Тюлегеновна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2013-09-23. Дата рождения: 1970-01-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Альрасми Надежда Николаевна'}, {'from': 'gpt', 'value': 'Альрасми Надежда Николаевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф17-ДЦ (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-02-05. Дата рождения: 1983-02-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аамев Низами Викторович'}, {'from': 'gpt', 'value': 'Аамев Низами Викторович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-12-16. Дата рождения: 1996-11-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аабмрцумян Мери Греликовна'}, {'from': 'gpt', 'value': 'Аабмрцумян Мери Греликовна работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2024-05-06. Дата рождения: 1993-09-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аазмева Анжела Усмановна'}, {'from': 'gpt', 'value': 'Аазмева Анжела Усмановна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф11 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2025-02-17. Дата рождения: 1995-05-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Авинома Алия Уражановна'}, {'from': 'gpt', 'value': 'Авинома Алия Уражановна работает в организации Волшебный Доктор на должности\n fВрач клинической лабораторной диагностики, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2022-12-22. Дата рождения: 1984-01-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Амнитаева Земфира Хабибуллаевна'}, {'from': 'gpt', 'value': 'Амнитаева Земфира Хабибуллаевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2008-09-26. Дата рождения: 1964-05-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Амивора Айшат Сатмаровна'}, {'from': 'gpt', 'value': 'Амивора Айшат Сатмаровна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2024-01-22. Дата рождения: 1984-08-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Амсоова Жанна Александровна'}, {'from': 'gpt', 'value': 'Амсоова Жанна Александровна работает в организации Волшебный Доктор на должности\n fЛогопед, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2015-02-11. Дата рождения: 1974-12-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ананеьва Анна Юрьевна'}, {'from': 'gpt', 'value': 'Ананеьва Анна Юрьевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2020-09-24. Дата рождения: 1971-12-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Андреев Алексей Олегович'}, {'from': 'gpt', 'value': 'Андреев Алексей Олегович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2020-03-02. Дата рождения: 1987-11-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Анрдеев Максим Андреевич'}, {'from': 'gpt', 'value': 'Анрдеев Максим Андреевич работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2024-10-28. Дата рождения: 1995-11-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Андрвееа Анастасия Геннадьевна'}, {'from': 'gpt', 'value': 'Андрвееа Анастасия Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2025-01-13. Дата рождения: 1995-11-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Андревеа Елена Геннадьевна'}, {'from': 'gpt', 'value': 'Андревеа Елена Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф05 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2013-02-20. Дата рождения: 1969-01-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Анрдеева Надежда Алексеевна'}, {'from': 'gpt', 'value': 'Анрдеева Надежда Алексеевна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2021-09-02. Дата рождения: 1969-04-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Андрцйеев Игорь Леонидович'}, {'from': 'gpt', 'value': 'Андрцйеев Игорь Леонидович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-11-26. Дата рождения: 1965-01-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Андронова Елена Сергеевна'}, {'from': 'gpt', 'value': 'Андронова Елена Сергеевна работает в организации Волшебный Доктор на должности\n fВрач клинической лабораторной диагностики, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2020-08-13. Дата рождения: 1985-03-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аднронова Надежда Изосимовна'}, {'from': 'gpt', 'value': 'Аднронова Надежда Изосимовна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2018-09-04. Дата рождения: 1959-03-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аниоимсв Роман Михайлович'}, {'from': 'gpt', 'value': 'Аниоимсв Роман Михайлович работает в организации Волшебный Доктор на должности\n fВедущий инженер по эксплуатации медицинского оборудования, в филиале Управление (Отдел эксплуатации медицинского оборудования, Медицинский департамент). Дата начала работы в организации: 2021-11-15. Дата рождения: 1974-08-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Амисинова Мария Геннадьевна'}, {'from': 'gpt', 'value': 'Амисинова Мария Геннадьевна работает в организации Волшебный Доктор на должности\n fГлавный специалист центра клиентского сервиса, в филиале Управление (Центр клиентского сервиса, Коммерческий департамент). Дата начала работы в организации: 2007-08-01. Дата рождения: 1987-02-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Анискевич Сергей Дмитриевич'}, {'from': 'gpt', 'value': 'Анискевич Сергей Дмитриевич работает в организации Волшебный Доктор на должности\n fСтарший врач скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2021-04-12. Дата рождения: 1961-03-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Анвкаб Ангелина Вячеславовна'}, {'from': 'gpt', 'value': 'Анвкаб Ангелина Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2022-12-05. Дата рождения: 1983-12-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Антиомнов Дмитрий Сергеевич'}, {'from': 'gpt', 'value': 'Антиомнов Дмитрий Сергеевич работает в организации Волшебный Доктор на должности\n fРуководитель Службы Эксплуатации, в филиале Управление (Административный департамент, Административный департамент). Дата начала работы в организации: 2023-09-18. Дата рождения: 1988-06-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Анвоненкота Татьяна Сергеевна'}, {'from': 'gpt', 'value': 'Анвоненкота Татьяна Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф16 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2016-01-25. Дата рождения: 1981-04-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Антноова Екатерина Сергеевна'}, {'from': 'gpt', 'value': 'Антноова Екатерина Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу участковая, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2025-02-06. Дата рождения: 1998-05-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Антоонва Инна Анатольевна'}, {'from': 'gpt', 'value': 'Антоонва Инна Анатольевна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2020-10-08. Дата рождения: 1967-03-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Антноян Янек Гарикович'}, {'from': 'gpt', 'value': 'Антноян Янек Гарикович работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2015-03-25. Дата рождения: 1987-10-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Анеощтнко Олеся Вадимовна'}, {'from': 'gpt', 'value': 'Анеощтнко Олеся Вадимовна работает в организации Волшебный Доктор на должности\n fВрач-детский хирург, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2024-11-27. Дата рождения: 1994-02-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аичнна Мария Геннадьевна'}, {'from': 'gpt', 'value': 'Аичнна Мария Геннадьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2020-11-05. Дата рождения: 1999-10-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Атрепова Наталия Юрьевна'}, {'from': 'gpt', 'value': 'Атрепова Наталия Юрьевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-01-22. Дата рождения: 1972-11-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Арапивеа Марина Михайловна'}, {'from': 'gpt', 'value': 'Арапивеа Марина Михайловна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф03 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2009-12-11. Дата рождения: 1979-05-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Арбозув Игорь Евгеньевич'}, {'from': 'gpt', 'value': 'Арбозув Игорь Евгеньевич работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2021-09-01. Дата рождения: 1986-02-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аржавона Елена Александровна'}, {'from': 'gpt', 'value': 'Аржавона Елена Александровна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2019-04-10. Дата рождения: 1981-06-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аресньева Яна Олеговна'}, {'from': 'gpt', 'value': 'Аресньева Яна Олеговна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф01 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2024-08-19. Дата рождения: 1968-09-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Артмеова Эльвира Леонидовна'}, {'from': 'gpt', 'value': 'Артмеова Эльвира Леонидовна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2025-03-03. Дата рождения: 1980-05-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Артеьмев Валерий Александрович'}, {'from': 'gpt', 'value': 'Артеьмев Валерий Александрович работает в организации Волшебный Доктор на должности\n fВрач-травматолог-ортопед, в филиале ГЦ-Ф15 (Отделение травматологии и ортопедии, Стационар). Дата начала работы в организации: 2024-12-09. Дата рождения: 1975-04-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аркюшто Оксана Александровна'}, {'from': 'gpt', 'value': 'Аркюшто Оксана Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2014-04-14. Дата рождения: 1970-05-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Арутюянн Елена Рафаеловна'}, {'from': 'gpt', 'value': 'Арутюянн Елена Рафаеловна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-06-27. Дата рождения: 1981-09-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ароипхв Андрей Михайлович'}, {'from': 'gpt', 'value': 'Ароипхв Андрей Михайлович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф11 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-06-01. Дата рождения: 1980-02-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Асданили Марина Ильинична'}, {'from': 'gpt', 'value': 'Асданили Марина Ильинична работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф16 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2013-10-07. Дата рождения: 1962-12-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Асмаонв Алан Исмаилович'}, {'from': 'gpt', 'value': 'Асмаонв Алан Исмаилович работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале ГЦ-Ф15 (Отделение оторинолангологии, Стационар). Дата начала работы в организации: 2017-07-10. Дата рождения: 1985-08-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ахвелдиани Ирина Васильевна'}, {'from': 'gpt', 'value': 'Ахвелдиани Ирина Васильевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф14 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2017-09-25. Дата рождения: 1955-09-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Аалхкова Патимат Магомедовна'}, {'from': 'gpt', 'value': 'Аалхкова Патимат Магомедовна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2015-12-16. Дата рождения: 1956-08-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ахмадуиллна Регина Фуатовна'}, {'from': 'gpt', 'value': 'Ахмадуиллна Регина Фуатовна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2025-01-13. Дата рождения: 1989-07-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ахьявода Барета Павловна'}, {'from': 'gpt', 'value': 'Ахьявода Барета Павловна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2010-08-31. Дата рождения: 1970-10-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ашахаонв Руслан Ашаханович'}, {'from': 'gpt', 'value': 'Ашахаонв Руслан Ашаханович работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф03 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2023-02-13. Дата рождения: 1995-01-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бабаев Игорь Ибрагимович'}, {'from': 'gpt', 'value': 'Бабаев Игорь Ибрагимович работает в организации Волшебный Доктор на должности\n fТехник, в филиале Ф09 (АХО, Вспомогательная служба). Дата начала работы в организации: 2017-04-04. Дата рождения: 1973-08-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Баабева София Александровна'}, {'from': 'gpt', 'value': 'Баабева София Александровна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2023-09-18. Дата рождения: 1995-07-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бабняац Армен Анатольевич'}, {'from': 'gpt', 'value': 'Бабняац Армен Анатольевич работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф12 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2017-03-14. Дата рождения: 1978-07-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бакбин Дмитрий Игоревич'}, {'from': 'gpt', 'value': 'Бакбин Дмитрий Игоревич работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф11 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2024-06-03. Дата рождения: 1998-10-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бабжуина Марина Юрьевна'}, {'from': 'gpt', 'value': 'Бабжуина Марина Юрьевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2019-07-02. Дата рождения: 1985-04-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Балунашвиби Мария Теймуразовна'}, {'from': 'gpt', 'value': 'Балунашвиби Мария Теймуразовна работает в организации Волшебный Доктор на должности\n fЗаведующий клинико-диагностическим отделением, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-11-29. Дата рождения: 1986-12-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бедаава Татьяна Ивановна'}, {'from': 'gpt', 'value': 'Бедаава Татьяна Ивановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-09-23. Дата рождения: 1974-01-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Блдааян Артур Карленович'}, {'from': 'gpt', 'value': 'Блдааян Артур Карленович работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф12 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2024-05-20. Дата рождения: 1994-11-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бадеамв Сергей Владимирович'}, {'from': 'gpt', 'value': 'Бадеамв Сергей Владимирович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2019-12-10. Дата рождения: 1979-05-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Беава Галина Григорьевна'}, {'from': 'gpt', 'value': 'Беава Галина Григорьевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф14 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2019-07-08. Дата рождения: 1961-04-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бажевона Мария Александровна'}, {'from': 'gpt', 'value': 'Бажевона Мария Александровна работает в организации Волшебный Доктор на должности\n fГлавный оператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2019-02-12. Дата рождения: 1971-05-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бвзаи Ирина Николаевна'}, {'from': 'gpt', 'value': 'Бвзаи Ирина Николаевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2018-04-11. Дата рождения: 1959-02-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Базилеивч Наталья Яковлевна'}, {'from': 'gpt', 'value': 'Базилеивч Наталья Яковлевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2021-09-27. Дата рождения: 1965-07-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бавдойа Ольга Николаевна'}, {'from': 'gpt', 'value': 'Бавдойа Ольга Николаевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2011-04-12. Дата рождения: 1979-08-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Байвока Вера Григорьевна'}, {'from': 'gpt', 'value': 'Байвока Вера Григорьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф02 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2022-10-24. Дата рождения: 1968-12-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бакитена Анна Валериевна'}, {'from': 'gpt', 'value': 'Бакитена Анна Валериевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф06 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2021-02-24. Дата рождения: 1993-01-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Банлакова Кетеван Григорьевна'}, {'from': 'gpt', 'value': 'Банлакова Кетеван Григорьевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2015-10-14. Дата рождения: 1975-07-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бавланока Маргарита Валериевна'}, {'from': 'gpt', 'value': 'Бавланока Маргарита Валериевна работает в организации Волшебный Доктор на должности\n fМенеджер по продажам юридическим лицам, в филиале Управление (Центр корпоративных продаж, Коммерческий департамент). Дата начала работы в организации: 2013-08-09. Дата рождения: 1979-07-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Блаа Виктория Юрьевна'}, {'from': 'gpt', 'value': 'Блаа Виктория Юрьевна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2024-12-24. Дата рождения: 1991-01-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Балвкиреаа Анастасия Юрьевна'}, {'from': 'gpt', 'value': 'Балвкиреаа Анастасия Юрьевна работает в организации Волшебный Доктор на должности\n fВедущий специалист отдела корпоративной документации, в филиале Управление (Отдел корпоративной документации, Финансовый департамент). Дата начала работы в организации: 2024-07-25. Дата рождения: 1991-08-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Блааметов Самир Гюльахмедович'}, {'from': 'gpt', 'value': 'Блааметов Самир Гюльахмедович работает в организации Волшебный Доктор на должности\n fВрач-травматолог-ортопед, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2018-10-08. Дата рождения: 1990-06-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Баралёв Антон Сергеевич'}, {'from': 'gpt', 'value': 'Баралёв Антон Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-04-19. Дата рождения: 1981-07-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Балавёра Дарья Алексеевна'}, {'from': 'gpt', 'value': 'Балавёра Дарья Алексеевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-10-04. Дата рождения: 1987-11-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Балицкая Александра Сергеевна'}, {'from': 'gpt', 'value': 'Балицкая Александра Сергеевна работает в организации Волшебный Доктор на должности\n fЗаведующий клинико-диагностическим отделением, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2019-07-15. Дата рождения: 1990-01-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бюлак Екатерина Александровна'}, {'from': 'gpt', 'value': 'Бюлак Екатерина Александровна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2016-11-21. Дата рождения: 1982-01-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Банищков Сергей Александрович'}, {'from': 'gpt', 'value': 'Банищков Сергей Александрович работает в организации Волшебный Доктор на должности\n fРуководитель отдела разработки интернет-проектов, в филиале Управление (Отдел разработки интернет-проектов, Центр разработки ПО). Дата начала работы в организации: 2016-09-06. Дата рождения: 1989-02-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бараквоа Анастасия Владимировна'}, {'from': 'gpt', 'value': 'Бараквоа Анастасия Владимировна работает в организации Волшебный Доктор на должности\n fСценарист, в филиале Управление (Отдел медиа-проектов, Департамент по работе с персоналом). Дата начала работы в организации: 2024-10-24. Дата рождения: 1994-02-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Баыамрков Данила Алексеевич'}, {'from': 'gpt', 'value': 'Баыамрков Данила Алексеевич работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2020-03-03. Дата рождения: 1997-02-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Банаров Данил Юрьевич'}, {'from': 'gpt', 'value': 'Банаров Данил Юрьевич работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-хирург, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2023-10-23. Дата рождения: 1998-05-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бнрааова Вера Евгеньевна'}, {'from': 'gpt', 'value': 'Бнрааова Вера Евгеньевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф02 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2021-04-14. Дата рождения: 1996-03-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бараонва Екатерина Александровна'}, {'from': 'gpt', 'value': 'Бараонва Екатерина Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2016-11-23. Дата рождения: 1981-11-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бвраноаа Екатерина Сергеевна'}, {'from': 'gpt', 'value': 'Бвраноаа Екатерина Сергеевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела маркетинга и рекламы, в филиале Управление (Отдел интернет-проектов, Департамент маркетинга и рекламы). Дата начала работы в организации: 2024-05-23. Дата рождения: 1984-05-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Браанович Наталья Александровна'}, {'from': 'gpt', 'value': 'Браанович Наталья Александровна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф01 (Отделение педиатрии, Отделение педиатрии). Дата начала работы в организации: 2014-04-21. Дата рождения: 1977-01-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бархатина Тамара Юрьевна'}, {'from': 'gpt', 'value': 'Бархатина Тамара Юрьевна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2018-10-24. Дата рождения: 1966-05-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Баниров Иван Петрович'}, {'from': 'gpt', 'value': 'Баниров Иван Петрович работает в организации Волшебный Доктор на должности\n fТехник сменный, в филиале ГЦ-Ф15 (Отдел административно-хозяйственного и технического обеспечения, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2017-01-17. Дата рождения: 1970-06-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Баркеува Ирайганат Камильевна'}, {'from': 'gpt', 'value': 'Баркеува Ирайганат Камильевна работает в организации Волшебный Доктор на должности\n fВрач-онколог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2025-01-13. Дата рождения: 1993-01-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Банулира Ольга Сергеевна'}, {'from': 'gpt', 'value': 'Банулира Ольга Сергеевна работает в организации Волшебный Доктор на должности\n fСтарший менеджер центра психического здоровья, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2021-08-23. Дата рождения: 1988-09-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Баркшниыов Владимир Леонидович'}, {'from': 'gpt', 'value': 'Баркшниыов Владимир Леонидович работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог, в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2016-01-20. Дата рождения: 1947-05-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Барыншикова Вера Витальевна'}, {'from': 'gpt', 'value': 'Барыншикова Вера Витальевна работает в организации Волшебный Доктор на должности\n fСпециалист по расчетам с юридическими лицами, в филиале Управление (Отдел учета реализации, Финансовый департамент). Дата начала работы в организации: 2024-08-06. Дата рождения: 1997-05-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Баряигна Инесса Александровна'}, {'from': 'gpt', 'value': 'Баряигна Инесса Александровна работает в организации Волшебный Доктор на должности\n fВедущий специалист отдела кадров, в филиале Управление (Отдел кадров, Департамент по работе с персоналом). Дата начала работы в организации: 2017-12-19. Дата рождения: 1984-04-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бюсак Злата Владимировна'}, {'from': 'gpt', 'value': 'Бюсак Злата Владимировна работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале ГЦ-Ф15 (Администрация, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2019-08-12. Дата рождения: 1986-07-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Баотгов Алексей Васильевич'}, {'from': 'gpt', 'value': 'Баотгов Алексей Васильевич работает в организации Волшебный Доктор на должности\n fНачальник склада, в филиале Управление (Отдел закупок, Медицинский департамент). Дата начала работы в организации: 2015-03-24. Дата рождения: 1975-06-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бакшатова Екатерина Алексеевна'}, {'from': 'gpt', 'value': 'Бакшатова Екатерина Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-08-05. Дата рождения: 1995-06-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Батшаров Джамбулат Магомедович'}, {'from': 'gpt', 'value': 'Батшаров Джамбулат Магомедович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-хирург, в филиале Ф11 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2024-04-25. Дата рождения: 1995-09-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бедян Софья Гургеновна'}, {'from': 'gpt', 'value': 'Бедян Софья Гургеновна работает в организации Волшебный Доктор на должности\n fМедицинская сестра кабинета подологии, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2022-11-22. Дата рождения: 1999-11-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бвееа Елена Владимировна'}, {'from': 'gpt', 'value': 'Бвееа Елена Владимировна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2020-07-06. Дата рождения: 1985-11-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бенажов Антон Витальевич'}, {'from': 'gpt', 'value': 'Бенажов Антон Витальевич работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2024-02-05. Дата рождения: 1987-02-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Брзеодных Татьяна Борисовна'}, {'from': 'gpt', 'value': 'Брзеодных Татьяна Борисовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Педиатрическое отделение, Стационар). Дата начала работы в организации: 2021-09-02. Дата рождения: 1975-06-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Безуквадола Анастасия Степановна'}, {'from': 'gpt', 'value': 'Безуквадола Анастасия Степановна работает в организации Волшебный Доктор на должности\n fВрач-фтизиатр, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2025-02-03. Дата рождения: 1989-12-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бекмамебтова Мерием Эскендеровна'}, {'from': 'gpt', 'value': 'Бекмамебтова Мерием Эскендеровна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф10 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2025-01-30. Дата рождения: 1983-10-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бектемиров Радик Асхатович'}, {'from': 'gpt', 'value': 'Бектемиров Радик Асхатович работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2015-05-06. Дата рождения: 1967-02-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Болев Александр Александрович'}, {'from': 'gpt', 'value': 'Болев Александр Александрович работает в организации Волшебный Доктор на должности\n fСтарший программист, в филиале Управление (Отдел разработки корпоративных систем, Центр разработки ПО). Дата начала работы в организации: 2017-01-19. Дата рождения: 1985-12-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Белвоа Анна Николаевна'}, {'from': 'gpt', 'value': 'Белвоа Анна Николаевна работает в организации Волшебный Доктор на должности\n fГлавный специалист по противопожарной безопасности, в филиале Управление (Отдел административно-хозяйственного и технического обеспечения, Административный департамент). Дата начала работы в организации: 2023-06-13. Дата рождения: 1978-02-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Белзоерцева Мария Сергеевна'}, {'from': 'gpt', 'value': 'Белзоерцева Мария Сергеевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале ГЦ-Ф15 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2025-02-06. Дата рождения: 2002-02-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Беуолсова Яна Александровна'}, {'from': 'gpt', 'value': 'Беуолсова Яна Александровна работает в организации Волшебный Доктор на должности\n fНачальник отдела финансового контроллинга, в филиале Управление (Отдел финансового контроллинга, Финансовый департамент). Дата начала работы в организации: 2008-09-08. Дата рождения: 1982-05-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Белокьва Анна Андреевна'}, {'from': 'gpt', 'value': 'Белокьва Анна Андреевна работает в организации Волшебный Доктор на должности\n fВрач-ортодонт, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2022-07-25. Дата рождения: 1996-07-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Белоченкьва Наталья Анатольевна'}, {'from': 'gpt', 'value': 'Белоченкьва Наталья Анатольевна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2019-09-05. Дата рождения: 1966-05-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бееялва Анна Николаевна'}, {'from': 'gpt', 'value': 'Бееялва Анна Николаевна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2023-12-29. Дата рождения: 1977-10-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Беялкова Ирина Олеговна'}, {'from': 'gpt', 'value': 'Беялкова Ирина Олеговна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф12 (Администрация, Администрация). Дата начала работы в организации: 2008-07-25. Дата рождения: 1989-10-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Берженова Мария Валерьевна'}, {'from': 'gpt', 'value': 'Берженова Мария Валерьевна работает в организации Волшебный Доктор на должности\n fНачальник юридического отдела, в филиале Управление (Юридический отдел, Коммерческий департамент). Дата начала работы в организации: 2019-06-03. Дата рождения: 1993-12-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Беразовскея Анна Сергеевна'}, {'from': 'gpt', 'value': 'Беразовскея Анна Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф10 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-10-02. Дата рождения: 1997-11-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Березоисквй Андрей Владимирович'}, {'from': 'gpt', 'value': 'Березоисквй Андрей Владимирович работает в организации Волшебный Доктор на должности\n fИнженер по эксплуатации медицинского оборудования сменный, в филиале Управление (Отдел эксплуатации медицинского оборудования, Медицинский департамент). Дата начала работы в организации: 2023-08-21. Дата рождения: 1974-06-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бкрниеов Евгений Валериевич'}, {'from': 'gpt', 'value': 'Бкрниеов Евгений Валериевич работает в организации Волшебный Доктор на должности\n fЗаведующий отделением урологии, в филиале ГЦ-Ф15 (Отделение урологии, Стационар). Дата начала работы в организации: 2021-03-01. Дата рождения: 1978-12-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Берсенева Юлия Алексеевна'}, {'from': 'gpt', 'value': 'Берсенева Юлия Алексеевна работает в организации Волшебный Доктор на должности\n fГлавный оператор, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2014-04-08. Дата рождения: 1984-04-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бенедиса Татьяна Алексеевна'}, {'from': 'gpt', 'value': 'Бенедиса Татьяна Алексеевна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Экспресс-лаборатория, Экспресс-лаборатория). Дата начала работы в организации: 2015-01-26. Дата рождения: 1971-09-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бнссмертеая Марина Вячеславовна'}, {'from': 'gpt', 'value': 'Бнссмертеая Марина Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2021-02-24. Дата рождения: 1967-08-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Билеивч Наталья Борисовна'}, {'from': 'gpt', 'value': 'Билеивч Наталья Борисовна работает в организации Волшебный Доктор на должности\n fГлавный оператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2017-10-24. Дата рождения: 1972-09-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Билиечнко Алла Ивановна'}, {'from': 'gpt', 'value': 'Билиечнко Алла Ивановна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2019-12-02. Дата рождения: 1976-02-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бильнечко Вера Валерьевна'}, {'from': 'gpt', 'value': 'Бильнечко Вера Валерьевна работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф02 (Администрация, Администрация). Дата начала работы в организации: 2023-07-17. Дата рождения: 1982-01-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бкрюиов Владимир Владимирович'}, {'from': 'gpt', 'value': 'Бкрюиов Владимир Владимирович работает в организации Волшебный Доктор на должности\n fБизнес-тренер, в филиале Управление (Учебный центр/Отдел корпоративного обучения, Департамент по работе с персоналом). Дата начала работы в организации: 2024-09-05. Дата рождения: 1991-07-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бисеснова Алёна Игоревна'}, {'from': 'gpt', 'value': 'Бисеснова Алёна Игоревна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-02-05. Дата рождения: 1989-04-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Блакжо Валентина Алексеевна'}, {'from': 'gpt', 'value': 'Блакжо Валентина Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2013-09-11. Дата рождения: 1955-11-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Блонива Олеся Александровна'}, {'from': 'gpt', 'value': 'Блонива Олеся Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф06 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2009-09-02. Дата рождения: 1985-07-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бнилова Татьяна Александровна'}, {'from': 'gpt', 'value': 'Бнилова Татьяна Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-10-21. Дата рождения: 1967-10-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Блхоин Никита Михайлович'}, {'from': 'gpt', 'value': 'Блхоин Никита Михайлович работает в организации Волшебный Доктор на должности\n fВедущий врач-уролог, в филиале ГЦ-Ф15 (Отделение урологии, Стационар). Дата начала работы в организации: 2019-11-25. Дата рождения: 1986-10-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бтгаоырева Мария Алексеевна'}, {'from': 'gpt', 'value': 'Бтгаоырева Мария Алексеевна работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача, в филиале Ф12 (Администрация, Администрация). Дата начала работы в организации: 2022-01-17. Дата рождения: 1982-05-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Богаечва Наталья Владимировна'}, {'from': 'gpt', 'value': 'Богаечва Наталья Владимировна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф02 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-04. Дата рождения: 1972-06-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Боадгнов Антон Павлович'}, {'from': 'gpt', 'value': 'Боадгнов Антон Павлович работает в организации Волшебный Доктор на должности\n fРуководитель направления эндоскопической диагностики, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2024-01-29. Дата рождения: 1981-09-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бдгооухов Роман Владимирович'}, {'from': 'gpt', 'value': 'Бдгооухов Роман Владимирович работает в организации Волшебный Доктор на должности\n fВедущий специалист отдела кадров, в филиале Управление (Отдел кадров, Департамент по работе с персоналом). Дата начала работы в организации: 2019-10-01. Дата рождения: 1997-09-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Богомолова Светлана Валентиновна'}, {'from': 'gpt', 'value': 'Богомолова Светлана Валентиновна работает в организации Волшебный Доктор на должности\n fЗаведующий педиатрическим отделением, в филиале Ф07 (Отделение педиатрии, Отделение педиатрии). Дата начала работы в организации: 2008-09-08. Дата рождения: 1979-01-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Богасловскоя Ярослава Сергеевна'}, {'from': 'gpt', 'value': 'Богасловскоя Ярослава Сергеевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф10 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2025-01-27. Дата рождения: 1994-04-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бовеа Галина Ивановна'}, {'from': 'gpt', 'value': 'Бовеа Галина Ивановна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф14 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2016-02-03. Дата рождения: 1971-10-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бйоко Дмитрий Николаевич'}, {'from': 'gpt', 'value': 'Бйоко Дмитрий Николаевич работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача, в филиале Ф05 (Администрация, Администрация). Дата начала работы в организации: 2015-09-09. Дата рождения: 1977-05-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бкйченоо Евгения Анатольевна'}, {'from': 'gpt', 'value': 'Бкйченоо Евгения Анатольевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2015-03-03. Дата рождения: 1975-10-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Болгоова Светлана Юрьевна'}, {'from': 'gpt', 'value': 'Болгоова Светлана Юрьевна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение микробиологии, Отделение микробиологии). Дата начала работы в организации: 2016-11-09. Дата рождения: 1972-12-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Больвоша Елена Викторовна'}, {'from': 'gpt', 'value': 'Больвоша Елена Викторовна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2018-07-03. Дата рождения: 1963-01-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Боанрцева Татьяна Юрьевна'}, {'from': 'gpt', 'value': 'Боанрцева Татьяна Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2024-09-09. Дата рождения: 1980-01-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бардошова Лариса Викторовна'}, {'from': 'gpt', 'value': 'Бардошова Лариса Викторовна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2011-08-15. Дата рождения: 1964-02-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бордзая Наталия Васильевна'}, {'from': 'gpt', 'value': 'Бордзая Наталия Васильевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-03-05. Дата рождения: 1965-03-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бзроова Светлана Александровна'}, {'from': 'gpt', 'value': 'Бзроова Светлана Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2023-06-19. Дата рождения: 1969-04-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бсриокова Олеся Сергеевна'}, {'from': 'gpt', 'value': 'Бсриокова Олеся Сергеевна работает в организации Волшебный Доктор на должности\n fЗаместитель начальника отдела финансового контроллинга, в филиале Управление (Отдел финансового контроллинга, Финансовый департамент). Дата начала работы в организации: 2018-02-08. Дата рождения: 1986-05-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Болодурин Константин Александрович'}, {'from': 'gpt', 'value': 'Болодурин Константин Александрович работает в организации Волшебный Доктор на должности\n fФельдшер скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2020-08-04. Дата рождения: 1994-08-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Борноина Евгения Петровна'}, {'from': 'gpt', 'value': 'Борноина Евгения Петровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-02-15. Дата рождения: 2003-04-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Босвеиа Ольга Александровна'}, {'from': 'gpt', 'value': 'Босвеиа Ольга Александровна работает в организации Волшебный Доктор на должности\n fЭксперт, в филиале Управление (Отдел экспертизы качества медицинской помощи, Медицинский департамент). Дата начала работы в организации: 2003-02-03. Дата рождения: 1945-09-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Брагниа Елена Геннадьевна'}, {'from': 'gpt', 'value': 'Брагниа Елена Геннадьевна работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф03 (Администрация, Администрация). Дата начала работы в организации: 2009-09-01. Дата рождения: 1975-06-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бревнежа Елена Владимировна'}, {'from': 'gpt', 'value': 'Бревнежа Елена Владимировна работает в организации Волшебный Доктор на должности\n fВрач-онколог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2019-11-05. Дата рождения: 1971-08-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бнежрева Наталия Сергеевна'}, {'from': 'gpt', 'value': 'Бнежрева Наталия Сергеевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф07 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2016-04-05. Дата рождения: 1976-06-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Биитврна Елена Владимировна'}, {'from': 'gpt', 'value': 'Биитврна Елена Владимировна работает в организации Волшебный Доктор на должности\n fБизнес-тренер, в филиале Управление (Учебный центр/Отдел корпоративного обучения, Департамент по работе с персоналом). Дата начала работы в организации: 2024-10-14. Дата рождения: 1984-07-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Быицрна Татьяна Николаевна'}, {'from': 'gpt', 'value': 'Быицрна Татьяна Николаевна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф06 (АХО, Вспомогательная служба). Дата начала работы в организации: 2025-01-16. Дата рождения: 1993-04-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Брмиошоева Майрам Азизшоевна'}, {'from': 'gpt', 'value': 'Брмиошоева Майрам Азизшоевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф12 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-06-20. Дата рождения: 1979-05-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ббунов Владимир Анатольевич'}, {'from': 'gpt', 'value': 'Ббунов Владимир Анатольевич работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф01 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2020-07-30. Дата рождения: 1974-03-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бугкйсаий Станислав Евгеньевич'}, {'from': 'gpt', 'value': 'Бугкйсаий Станислав Евгеньевич работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача, в филиале Ф02 (Администрация, Администрация). Дата начала работы в организации: 2025-02-10. Дата рождения: 1986-07-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бидарун Вадим Александрович'}, {'from': 'gpt', 'value': 'Бидарун Вадим Александрович работает в организации Волшебный Доктор на должности\n fГлавный специалист по поддержке пользователей, в филиале Управление (Отдел технической поддержки, Департамент ИТ). Дата начала работы в организации: 2021-07-19. Дата рождения: 2000-06-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Буника Юлия Александровна'}, {'from': 'gpt', 'value': 'Буника Юлия Александровна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2024-10-14. Дата рождения: 1981-10-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Болатува Олеся Сергеевна'}, {'from': 'gpt', 'value': 'Болатува Олеся Сергеевна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф05 (Администрация, Администрация). Дата начала работы в организации: 2017-04-12. Дата рождения: 1986-12-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бглуарь Елена Витальевна'}, {'from': 'gpt', 'value': 'Бглуарь Елена Витальевна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2020-03-03. Дата рождения: 1974-12-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Букцынина Валентина Наильевна'}, {'from': 'gpt', 'value': 'Букцынина Валентина Наильевна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2022-11-14. Дата рождения: 1989-05-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Буарлкин Артур Игоревич'}, {'from': 'gpt', 'value': 'Буарлкин Артур Игоревич работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2020-11-19. Дата рождения: 1996-10-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бирдун Даниил Андреевич'}, {'from': 'gpt', 'value': 'Бирдун Даниил Андреевич работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2021-08-23. Дата рождения: 2000-12-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бурдкыо Карина Олеговна'}, {'from': 'gpt', 'value': 'Бурдкыо Карина Олеговна работает в организации Волшебный Доктор на должности\n fСекретарь-делопроизводитель, в филиале Управление (Отдел корпоративной документации, Финансовый департамент). Дата начала работы в организации: 2024-09-09. Дата рождения: 2001-02-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бурмисртова Татьяна Руслановна'}, {'from': 'gpt', 'value': 'Бурмисртова Татьяна Руслановна работает в организации Волшебный Доктор на должности\n fМенеджер стоматологического центра сменный, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2021-09-27. Дата рождения: 2003-02-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Баслуева Ольга Ивановна'}, {'from': 'gpt', 'value': 'Баслуева Ольга Ивановна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф10 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2011-09-12. Дата рождения: 1955-06-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бутокава Оксана Алексеевна'}, {'from': 'gpt', 'value': 'Бутокава Оксана Алексеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу участковая, в филиале Ф01 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2020-11-17. Дата рождения: 1972-06-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бутарынй Александр Олегович'}, {'from': 'gpt', 'value': 'Бутарынй Александр Олегович работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2023-03-13. Дата рождения: 1991-11-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бвтоуа Елена Александровна'}, {'from': 'gpt', 'value': 'Бвтоуа Елена Александровна работает в организации Волшебный Доктор на должности\n fЭксперт, в филиале Управление (Отдел экспертизы качества медицинской помощи, Медицинский департамент). Дата начала работы в организации: 2024-06-17. Дата рождения: 1976-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бумшелева Елена Анатольевна'}, {'from': 'gpt', 'value': 'Бумшелева Елена Анатольевна работает в организации Волшебный Доктор на должности\n fОператор ОМС, в филиале ГЦ-Ф15 (Отдел ОМС, Коммерческий департамент ГЦ). Дата начала работы в организации: 2019-01-15. Дата рождения: 1975-10-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бмшуина Анастасия Александровна'}, {'from': 'gpt', 'value': 'Бмшуина Анастасия Александровна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2022-03-21. Дата рождения: 1983-07-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Буеушв Алексей Владимирович'}, {'from': 'gpt', 'value': 'Буеушв Алексей Владимирович работает в организации Волшебный Доктор на должности\n fТехник, в филиале ГЦ-Ф15 (Отдел административно-хозяйственного и технического обеспечения, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-02-04. Дата рождения: 1974-10-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бушуева Екатерина Александровна'}, {'from': 'gpt', 'value': 'Бушуева Екатерина Александровна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2023-07-03. Дата рождения: 1996-01-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бкыова Диана Олеговна'}, {'from': 'gpt', 'value': 'Бкыова Диана Олеговна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2024-01-15. Дата рождения: 1981-12-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бострыв Андрей Геннадьевич'}, {'from': 'gpt', 'value': 'Бострыв Андрей Геннадьевич работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-12-10. Дата рождения: 1973-07-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бвхоыцева Татьяна Сергеевна'}, {'from': 'gpt', 'value': 'Бвхоыцева Татьяна Сергеевна работает в организации Волшебный Доктор на должности\n fВрач функциональной диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2019-03-18. Дата рождения: 1990-05-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Бяникна Надежда Владимировна'}, {'from': 'gpt', 'value': 'Бяникна Надежда Владимировна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2018-01-25. Дата рождения: 1954-07-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Валеива Венера Вахитовна'}, {'from': 'gpt', 'value': 'Валеива Венера Вахитовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2024-04-01. Дата рождения: 1968-01-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Валиоква Марина Владимировна'}, {'from': 'gpt', 'value': 'Валиоква Марина Владимировна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф16 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2022-11-21. Дата рождения: 1982-09-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вллиулаина Лейсан Наилевна'}, {'from': 'gpt', 'value': 'Вллиулаина Лейсан Наилевна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2009-03-17. Дата рождения: 1975-02-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ванкюова Ирина Александровна'}, {'from': 'gpt', 'value': 'Ванкюова Ирина Александровна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф07 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2019-09-02. Дата рождения: 1979-12-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Влсиаати Людмила Васильевна'}, {'from': 'gpt', 'value': 'Влсиаати Людмила Васильевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2018-10-03. Дата рождения: 1970-01-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ваеильсв Данила Михайлович'}, {'from': 'gpt', 'value': 'Ваеильсв Данила Михайлович работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале ГЦ-Ф15 (Отделение СМП, Стационар). Дата начала работы в организации: 2019-02-18. Дата рождения: 1997-12-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Василвеьа Анна Александровна'}, {'from': 'gpt', 'value': 'Василвеьа Анна Александровна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2021-12-23. Дата рождения: 1999-09-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Висальева Галина Павловна'}, {'from': 'gpt', 'value': 'Висальева Галина Павловна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2009-03-31. Дата рождения: 1982-01-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Васивьела Людмила Михайловна'}, {'from': 'gpt', 'value': 'Васивьела Людмила Михайловна работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Ф03 (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2011-09-21. Дата рождения: 1979-07-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Васниа Виктория Евгеньевна'}, {'from': 'gpt', 'value': 'Васниа Виктория Евгеньевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2023-05-25. Дата рождения: 1993-09-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Васкьина Екатерина Николаевна'}, {'from': 'gpt', 'value': 'Васкьина Екатерина Николаевна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2015-08-18. Дата рождения: 1967-02-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вюсаков Сергей Андреевич'}, {'from': 'gpt', 'value': 'Вюсаков Сергей Андреевич работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2024-02-28. Дата рождения: 1987-04-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вавюкоса Анастасия Ивановна'}, {'from': 'gpt', 'value': 'Вавюкоса Анастасия Ивановна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2022-02-10. Дата рождения: 1993-07-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вохобава Саодат Алихоновна'}, {'from': 'gpt', 'value': 'Вохобава Саодат Алихоновна работает в организации Волшебный Доктор на должности\n fМенеджер центра косметологии и пластической хирургии, в филиале Управление (Центр косметологии и пластической хирургии, Коммерческий департамент). Дата начала работы в организации: 2024-02-05. Дата рождения: 1979-05-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Веебр Людмила Николаевна'}, {'from': 'gpt', 'value': 'Веебр Людмила Николаевна работает в организации Волшебный Доктор на должности\n fКоординатор пульта, в филиале ГЦ-Ф15 (Медицинский пульт, Коммерческий департамент ГЦ). Дата начала работы в организации: 2023-06-05. Дата рождения: 1983-01-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Великнаова Арина Алексеевна'}, {'from': 'gpt', 'value': 'Великнаова Арина Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф16 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2024-12-19. Дата рождения: 1994-09-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Велимович Ольга Анатольевна'}, {'from': 'gpt', 'value': 'Велимович Ольга Анатольевна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф07 (Администрация, Администрация). Дата начала работы в организации: 2017-09-27. Дата рождения: 1975-10-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Верениякдн Алексей Юрьевич'}, {'from': 'gpt', 'value': 'Верениякдн Алексей Юрьевич работает в организации Волшебный Доктор на должности\n fДиректор департамента специализированных проектов, в филиале ГЦ-Ф15 (Администрация, Администрация). Дата начала работы в организации: 2024-04-01. Дата рождения: 1973-09-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вегещарин Лев Владиславович'}, {'from': 'gpt', 'value': 'Вегещарин Лев Владиславович работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог (хирург), в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-03-11. Дата рождения: 1979-08-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Врешинин Андрей Евгеньевич'}, {'from': 'gpt', 'value': 'Врешинин Андрей Евгеньевич работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2014-03-07. Дата рождения: 1972-02-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Весеклов Николай Васильевич'}, {'from': 'gpt', 'value': 'Весеклов Николай Васильевич работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2012-10-03. Дата рождения: 1991-02-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Викоортв Никита Константинович'}, {'from': 'gpt', 'value': 'Викоортв Никита Константинович работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2022-06-14. Дата рождения: 1990-09-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Викотрова Инесса Владимировна'}, {'from': 'gpt', 'value': 'Викотрова Инесса Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2022-02-01. Дата рождения: 1990-03-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вкиулова Светлана Николаевна'}, {'from': 'gpt', 'value': 'Вкиулова Светлана Николаевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2009-01-30. Дата рождения: 1961-03-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вичьлинская Юлия Анатольевна'}, {'from': 'gpt', 'value': 'Вичьлинская Юлия Анатольевна работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф16 (Администрация, Администрация). Дата начала работы в организации: 2007-09-01. Дата рождения: 1979-06-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вииютрна Юлия Ивановна'}, {'from': 'gpt', 'value': 'Вииютрна Юлия Ивановна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2020-11-02. Дата рождения: 1986-04-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Встаиик Наталья Андреевна'}, {'from': 'gpt', 'value': 'Встаиик Наталья Андреевна работает в организации Волшебный Доктор на должности\n fСпециалист отдела финансового контроллинга, в филиале Управление (Отдел финансового контроллинга, Финансовый департамент). Дата начала работы в организации: 2020-01-23. Дата рождения: 1990-12-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вивреха Ольга Геннадьевна'}, {'from': 'gpt', 'value': 'Вивреха Ольга Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф06 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2024-01-29. Дата рождения: 1974-09-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Влдаимиров Александр Андреевич'}, {'from': 'gpt', 'value': 'Влдаимиров Александр Андреевич работает в организации Волшебный Доктор на должности\n fВедущий программист, в филиале Управление (Отдел разработки корпоративных систем, Центр разработки ПО). Дата начала работы в организации: 2022-01-26. Дата рождения: 1995-03-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Влавнеза Оксана Владимировна'}, {'from': 'gpt', 'value': 'Влавнеза Оксана Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-12-26. Дата рождения: 1989-12-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Всалкина Наталья Геннадьевна'}, {'from': 'gpt', 'value': 'Всалкина Наталья Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог, в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2025-01-27. Дата рождения: 1990-10-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ввасола Алина Романовна'}, {'from': 'gpt', 'value': 'Ввасола Алина Романовна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2023-04-03. Дата рождения: 1995-06-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Влвсоаа Ольга Борисовна'}, {'from': 'gpt', 'value': 'Влвсоаа Ольга Борисовна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф06 (Администрация, Администрация). Дата начала работы в организации: 2018-10-17. Дата рождения: 1969-09-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Влвсоаа Ольга Владимировна'}, {'from': 'gpt', 'value': 'Влвсоаа Ольга Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2025-02-05. Дата рождения: 1986-04-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вдоова Анастасия Васильевна'}, {'from': 'gpt', 'value': 'Вдоова Анастасия Васильевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф12 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-07-22. Дата рождения: 1999-04-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Водвоеина Лариса Ивановна'}, {'from': 'gpt', 'value': 'Водвоеина Лариса Ивановна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2020-08-03. Дата рождения: 1956-03-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вонзесенская Анастасия Александровна'}, {'from': 'gpt', 'value': 'Вонзесенская Анастасия Александровна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2021-09-07. Дата рождения: 1997-11-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Военесзнская Оксана Викторовна'}, {'from': 'gpt', 'value': 'Военесзнская Оксана Викторовна работает в организации Волшебный Доктор на должности\n fДиректор медицинского департамента, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2007-10-09. Дата рождения: 1973-05-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вонйов Вячеслав Владимирович'}, {'from': 'gpt', 'value': 'Вонйов Вячеслав Владимирович работает в организации Волшебный Доктор на должности\n fВодитель скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2020-06-15. Дата рождения: 1988-01-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вотйова Анастасия Александровна'}, {'from': 'gpt', 'value': 'Вотйова Анастасия Александровна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф16 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-04-08. Дата рождения: 2004-10-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Воглина Светлана Владимировна'}, {'from': 'gpt', 'value': 'Воглина Светлана Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра эндоскопии, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-10-15. Дата рождения: 1961-03-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Волоква Наталья Валериевна'}, {'from': 'gpt', 'value': 'Волоква Наталья Валериевна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2020-09-21. Дата рождения: 1977-03-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Волндиоа Марина Юрьевна'}, {'from': 'gpt', 'value': 'Волндиоа Марина Юрьевна работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф11 (Администрация, Администрация). Дата начала работы в организации: 2013-08-12. Дата рождения: 1966-10-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Водьлман Алексей Анатольевич'}, {'from': 'gpt', 'value': 'Водьлман Алексей Анатольевич работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф06 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-04-25. Дата рождения: 1971-03-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Воорбьева Алина Вячеславовна'}, {'from': 'gpt', 'value': 'Воорбьева Алина Вячеславовна работает в организации Волшебный Доктор на должности\n fГигиенист стоматологический, в филиале Ф02 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-02-22. Дата рождения: 2000-08-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вожорцова Любовь Николаевна'}, {'from': 'gpt', 'value': 'Вожорцова Любовь Николаевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2014-11-19. Дата рождения: 1950-10-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вроонина Евгения Михайловна'}, {'from': 'gpt', 'value': 'Вроонина Евгения Михайловна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2017-02-15. Дата рождения: 1989-04-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Воорнина Екатерина Ивановна'}, {'from': 'gpt', 'value': 'Воорнина Екатерина Ивановна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2022-03-04. Дата рождения: 1983-11-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ворноова Мария Михайловна'}, {'from': 'gpt', 'value': 'Ворноова Мария Михайловна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф09 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2019-09-30. Дата рождения: 1988-04-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ворноцова Ольга Владимировна'}, {'from': 'gpt', 'value': 'Ворноцова Ольга Владимировна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф09 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2020-09-14. Дата рождения: 1973-03-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вострвкоиа Джульета Михайловна'}, {'from': 'gpt', 'value': 'Вострвкоиа Джульета Михайловна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2018-04-03. Дата рождения: 1974-10-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вощинина Наталия Владимировна'}, {'from': 'gpt', 'value': 'Вощинина Наталия Владимировна работает в организации Волшебный Доктор на должности\n fРуководитель стоматологического центра, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-10-14. Дата рождения: 1985-06-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Врибай Татьяна Григорьевна'}, {'from': 'gpt', 'value': 'Врибай Татьяна Григорьевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2014-04-08. Дата рождения: 1973-04-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Вйдаыко Игорь Олегович'}, {'from': 'gpt', 'value': 'Вйдаыко Игорь Олегович работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф03 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-10-02. Дата рождения: 1997-01-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Восыцкая Татьяна Евгеньевна'}, {'from': 'gpt', 'value': 'Восыцкая Татьяна Евгеньевна работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале Ф02 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2018-12-05. Дата рождения: 1970-05-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гавееба Даяна Дамдиновна'}, {'from': 'gpt', 'value': 'Гавееба Даяна Дамдиновна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2023-10-02. Дата рождения: 1994-09-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гаврилвоа Виктория Михайловна'}, {'from': 'gpt', 'value': 'Гаврилвоа Виктория Михайловна работает в организации Волшебный Доктор на должности\n fМедицинский регистратор, в филиале Лаборатория (Экспресс-лаборатория, Экспресс-лаборатория). Дата начала работы в организации: 2025-01-23. Дата рождения: 2006-10-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гавриличкона Виктория Игоревна'}, {'from': 'gpt', 'value': 'Гавриличкона Виктория Игоревна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале Ф02 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2021-10-25. Дата рождения: 1993-11-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гдаадова Майрам Абдулазизовна'}, {'from': 'gpt', 'value': 'Гдаадова Майрам Абдулазизовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2018-02-12. Дата рождения: 1962-03-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Глджибаааева Диана Гаджибековна'}, {'from': 'gpt', 'value': 'Глджибаааева Диана Гаджибековна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2017-01-18. Дата рождения: 1991-03-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Галбатуйлаева Саида Нарудиновна'}, {'from': 'gpt', 'value': 'Галбатуйлаева Саида Нарудиновна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-07-04. Дата рождения: 2002-01-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гкйдуаова Наталия Александровна'}, {'from': 'gpt', 'value': 'Гкйдуаова Наталия Александровна работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2023-08-17. Дата рождения: 1976-06-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ганнайова Юлия Маратовна'}, {'from': 'gpt', 'value': 'Ганнайова Юлия Маратовна работает в организации Волшебный Доктор на должности\n fВрач-эндоскопист, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2019-07-08. Дата рождения: 1985-04-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Галимвоа Наталья Рашитовна'}, {'from': 'gpt', 'value': 'Галимвоа Наталья Рашитовна работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Ф17-ДЦ (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-07-22. Дата рождения: 1987-12-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Галицныа Ирина Николаевна'}, {'from': 'gpt', 'value': 'Галицныа Ирина Николаевна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2019-01-14. Дата рождения: 1969-09-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Глаич Татьяна Геннадьевна'}, {'from': 'gpt', 'value': 'Глаич Татьяна Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2023-11-27. Дата рождения: 1976-11-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гилкана Жанна Николаевна'}, {'from': 'gpt', 'value': 'Гилкана Жанна Николаевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2019-02-25. Дата рождения: 1979-12-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ганкила Юлия Сергеевна'}, {'from': 'gpt', 'value': 'Ганкила Юлия Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2025-02-13. Дата рождения: 1982-08-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гажинева Фатима Беслановна'}, {'from': 'gpt', 'value': 'Гажинева Фатима Беслановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф02 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2025-01-30. Дата рождения: 2006-08-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гкпенао Олег Константинович'}, {'from': 'gpt', 'value': 'Гкпенао Олег Константинович работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале ГЦ-Ф15 (Центр гастроэнтерологии, Центр гастроэнтерологии). Дата начала работы в организации: 2021-11-08. Дата рождения: 1970-06-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гаоонпва Татьяна Александровна'}, {'from': 'gpt', 'value': 'Гаоонпва Татьяна Александровна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф09 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1959-05-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гараева Елена Владимировна'}, {'from': 'gpt', 'value': 'Гараева Елена Владимировна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2022-10-18. Дата рождения: 1990-02-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ганащерко Наталья Геннадьевна'}, {'from': 'gpt', 'value': 'Ганащерко Наталья Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф12 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2016-04-11. Дата рождения: 1959-03-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гуркан Екатерина Анатольевна'}, {'from': 'gpt', 'value': 'Гуркан Екатерина Анатольевна работает в организации Волшебный Доктор на должности\n fМенеджер по мониторингу, в филиале ГЦ-Ф15 (Отдел мониторинга и экстренной госпитализации, Коммерческий департамент ГЦ). Дата начала работы в организации: 2021-06-23. Дата рождения: 1995-05-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гаркьовенко Галина Николаевна'}, {'from': 'gpt', 'value': 'Гаркьовенко Галина Николаевна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2013-10-09. Дата рождения: 1975-01-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гаисева Алина Ахсарбековна'}, {'from': 'gpt', 'value': 'Гаисева Алина Ахсарбековна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2023-04-17. Дата рождения: 1994-11-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гбухаерг Ирина Викторовна'}, {'from': 'gpt', 'value': 'Гбухаерг Ирина Викторовна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2013-09-18. Дата рождения: 1969-03-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гаишмова Экатерина Гивиевна'}, {'from': 'gpt', 'value': 'Гаишмова Экатерина Гивиевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2023-12-11. Дата рождения: 1971-07-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гвоздева Ольга Александровна'}, {'from': 'gpt', 'value': 'Гвоздева Ольга Александровна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-03-21. Дата рождения: 1980-09-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гооздквва Анастасия Геннадьевна'}, {'from': 'gpt', 'value': 'Гооздквва Анастасия Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач-детский эндокринолог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-09-24. Дата рождения: 1997-01-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гевокрян Софья Рубеновна'}, {'from': 'gpt', 'value': 'Гевокрян Софья Рубеновна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2019-10-14. Дата рождения: 1993-04-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гельемль Алла Владимировна'}, {'from': 'gpt', 'value': 'Гельемль Алла Владимировна работает в организации Волшебный Доктор на должности\n fВедущий специалист отдела кадров, в филиале Управление (Отдел кадров, Департамент по работе с персоналом). Дата начала работы в организации: 2025-01-13. Дата рождения: 1980-03-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Генцкиова Бальджир Даниловна'}, {'from': 'gpt', 'value': 'Генцкиова Бальджир Даниловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф12 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2015-02-25. Дата рождения: 1983-04-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Геваеца Елена Константиновна'}, {'from': 'gpt', 'value': 'Геваеца Елена Константиновна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2016-10-03. Дата рождения: 1991-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гильванома Луиза Муртазалиевна'}, {'from': 'gpt', 'value': 'Гильванома Луиза Муртазалиевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-09-16. Дата рождения: 1982-12-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гяноин Гор Арутюнович'}, {'from': 'gpt', 'value': 'Гяноин Гор Арутюнович работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф10 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2025-01-20. Дата рождения: 1998-06-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Глвбоеа Юлия Борисовна'}, {'from': 'gpt', 'value': 'Глвбоеа Юлия Борисовна работает в организации Волшебный Доктор на должности\n fВрач - пластический хирург, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2025-02-10. Дата рождения: 1974-07-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гнедина Екатерина Анатольевна'}, {'from': 'gpt', 'value': 'Гнедина Екатерина Анатольевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2016-04-26. Дата рождения: 1970-02-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гневдилоза Наталья Викторовна'}, {'from': 'gpt', 'value': 'Гневдилоза Наталья Викторовна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2011-06-02. Дата рождения: 1985-01-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гоигн Александр Владимирович'}, {'from': 'gpt', 'value': 'Гоигн Александр Владимирович работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2019-06-24. Дата рождения: 1979-08-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гдломан Лолита Юрьевна'}, {'from': 'gpt', 'value': 'Гдломан Лолита Юрьевна работает в организации Волшебный Доктор на должности\n fКлинический психолог, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2024-09-09. Дата рождения: 1999-08-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Голоквов Сергей Николаевич'}, {'from': 'gpt', 'value': 'Голоквов Сергей Николаевич работает в организации Волшебный Доктор на должности\n fРуководитель центра, в филиале Управление (Центр разработки ПО, Центр разработки ПО). Дата начала работы в организации: 2010-06-15. Дата рождения: 1985-01-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Голлвоёва Ирина Васильевна'}, {'from': 'gpt', 'value': 'Голлвоёва Ирина Васильевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф09 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2008-08-18. Дата рождения: 1963-01-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гоцолевич Кирилл Иванович'}, {'from': 'gpt', 'value': 'Гоцолевич Кирилл Иванович работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2024-04-08. Дата рождения: 1983-04-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гоулбев Денис Владимирович'}, {'from': 'gpt', 'value': 'Гоулбев Денис Владимирович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф17-ДЦ (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-03-06. Дата рождения: 1990-07-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Голвбеуа Людмила Викторовна'}, {'from': 'gpt', 'value': 'Голвбеуа Людмила Викторовна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф10 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2015-05-26. Дата рождения: 1969-05-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гялок Елена Владимировна'}, {'from': 'gpt', 'value': 'Гялок Елена Владимировна работает в организации Волшебный Доктор на должности\n fЗаместитель руководителя отдела закупок, в филиале ГЦ-Ф15 (Отдел хозобеспечения, Стационар). Дата начала работы в организации: 2018-05-07. Дата рождения: 1964-06-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гкнчареноо Маргарита Александровна'}, {'from': 'gpt', 'value': 'Гкнчареноо Маргарита Александровна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф14 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2023-11-09. Дата рождения: 1977-05-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Грнчаоова Лилия Сергеевна'}, {'from': 'gpt', 'value': 'Грнчаоова Лилия Сергеевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале ГЦ-Ф15 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2021-12-27. Дата рождения: 1993-07-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Горбечава Светлана Романовна'}, {'from': 'gpt', 'value': 'Горбечава Светлана Романовна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Отдел госпитальных продаж, Коммерческий департамент). Дата начала работы в организации: 2024-09-05. Дата рождения: 1998-02-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гореедва Анна Степановна'}, {'from': 'gpt', 'value': 'Гореедва Анна Степановна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2013-11-11. Дата рождения: 1986-04-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гордацкея Елена Сергеевна'}, {'from': 'gpt', 'value': 'Гордацкея Елена Сергеевна работает в организации Волшебный Доктор на должности\n fНачальник отдела нормирования труда и мотивации, в филиале Управление (Отдел нормирования труда и мотивации персонала, Департамент по работе с персоналом). Дата начала работы в организации: 2009-03-25. Дата рождения: 1978-12-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гдроецкая Наталья Валентиновна'}, {'from': 'gpt', 'value': 'Гдроецкая Наталья Валентиновна работает в организации Волшебный Доктор на должности\n fЗаместитель директора департамента по работе с персоналом, в филиале Управление (Департамент по работе с персоналом, Департамент по работе с персоналом). Дата начала работы в организации: 2005-08-17. Дата рождения: 1960-06-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Горнюшедко Анастасия Сергеевна'}, {'from': 'gpt', 'value': 'Горнюшедко Анастасия Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по физиотерапии, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2013-02-12. Дата рождения: 1984-11-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гоиельнркова Наталья Владимировна'}, {'from': 'gpt', 'value': 'Гоиельнркова Наталья Владимировна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф02 (Администрация, Администрация). Дата начала работы в организации: 2010-01-19. Дата рождения: 1978-03-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Горинов Сергей Иванович'}, {'from': 'gpt', 'value': 'Горинов Сергей Иванович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2020-02-10. Дата рождения: 1959-02-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Горняквоа Дарина Владимировна'}, {'from': 'gpt', 'value': 'Горняквоа Дарина Владимировна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2022-12-01. Дата рождения: 1996-06-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гоноверко Надежда Ивановна'}, {'from': 'gpt', 'value': 'Гоноверко Надежда Ивановна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2020-09-01. Дата рождения: 1963-11-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гцродеокая Людмила Борисовна'}, {'from': 'gpt', 'value': 'Гцродеокая Людмила Борисовна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф10 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2015-05-21. Дата рождения: 1981-07-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Городцекая Юлия Борисовна'}, {'from': 'gpt', 'value': 'Городцекая Юлия Борисовна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог (хирург), в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-08-12. Дата рождения: 1991-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гоордчанина Кристина Юрьевна'}, {'from': 'gpt', 'value': 'Гоордчанина Кристина Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф03 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2013-09-02. Дата рождения: 1983-05-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гкршоов Дмитрий Вячеславович'}, {'from': 'gpt', 'value': 'Гкршоов Дмитрий Вячеславович работает в организации Волшебный Доктор на должности\n fПерсональный водитель автомобиля руководителя, в филиале Управление (Автопарк дирекции, Аппарат дирекции). Дата начала работы в организации: 2008-12-03. Дата рождения: 1972-01-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Горшквоа Елена Михайловна'}, {'from': 'gpt', 'value': 'Горшквоа Елена Михайловна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-05-30. Дата рождения: 2000-09-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гравока Светлана Ивановна'}, {'from': 'gpt', 'value': 'Гравока Светлана Ивановна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2017-04-12. Дата рождения: 1994-03-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Грнакин Алексей Викторович'}, {'from': 'gpt', 'value': 'Грнакин Алексей Викторович работает в организации Волшебный Доктор на должности\n fДиректор по специализированным административно-хозяйственным проектам, в филиале Управление (Аппарат управления, Аппарат управления). Дата начала работы в организации: 2008-11-24. Дата рождения: 1973-05-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Грвфоаа Юлия Вячеславовна'}, {'from': 'gpt', 'value': 'Грвфоаа Юлия Вячеславовна работает в организации Волшебный Доктор на должности\n fГенеральный директор, в филиале Управление (Руководство, Руководство). Дата начала работы в организации: 1999-09-01. Дата рождения: 1973-08-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гвачера Наталья Евгеньевна'}, {'from': 'gpt', 'value': 'Гвачера Наталья Евгеньевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2024-05-02. Дата рождения: 1973-01-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гребкнниеов Андрей Павлович'}, {'from': 'gpt', 'value': 'Гребкнниеов Андрей Павлович работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф03 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2012-04-11. Дата рождения: 1977-06-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гриабнова Любовь Сергеевна'}, {'from': 'gpt', 'value': 'Гриабнова Любовь Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2023-06-19. Дата рождения: 1994-10-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Григврооа Елена Юрьевна'}, {'from': 'gpt', 'value': 'Григврооа Елена Юрьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2018-07-16. Дата рождения: 1971-11-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гоигррьев Георгий Александрович'}, {'from': 'gpt', 'value': 'Гоигррьев Георгий Александрович работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале ГЦ-Ф15 (Отделение гинекологии №1, Стационар). Дата начала работы в организации: 2024-08-19. Дата рождения: 1982-08-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Григовьера Анна Александровна'}, {'from': 'gpt', 'value': 'Григовьера Анна Александровна работает в организации Волшебный Доктор на должности\n fВрач-бактериолог, в филиале Лаборатория (Отделение микробиологии, Отделение микробиологии). Дата начала работы в организации: 2021-08-02. Дата рождения: 1991-09-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гриворьега Людмила Владимировна'}, {'from': 'gpt', 'value': 'Гриворьега Людмила Владимировна работает в организации Волшебный Доктор на должности\n fОператор, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2014-04-08. Дата рождения: 1967-05-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Грвднеиа Гузалия Шамиловна'}, {'from': 'gpt', 'value': 'Грвднеиа Гузалия Шамиловна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2020-09-01. Дата рождения: 1965-11-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Грибнаум Евгений Геннадьевич'}, {'from': 'gpt', 'value': 'Грибнаум Евгений Геннадьевич работает в организации Волшебный Доктор на должности\n fЗаместитель начальника транспортного отдела, в филиале Управление (Транспортный отдел, Административный департамент). Дата начала работы в организации: 2008-10-15. Дата рождения: 1983-05-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Грингуаз Олеся Викторовна'}, {'from': 'gpt', 'value': 'Грингуаз Олеся Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф16 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2023-11-16. Дата рождения: 1987-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гиршанин Алексей Викторович'}, {'from': 'gpt', 'value': 'Гиршанин Алексей Викторович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф03 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2014-04-01. Дата рождения: 1961-09-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гчишеркина Эмма Леонидовна'}, {'from': 'gpt', 'value': 'Гчишеркина Эмма Леонидовна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2018-08-22. Дата рождения: 1965-12-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гиршина Екатерина Олеговна'}, {'from': 'gpt', 'value': 'Гиршина Екатерина Олеговна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф16 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2023-07-03. Дата рождения: 1994-12-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гиишрна Нина Владимировна'}, {'from': 'gpt', 'value': 'Гиишрна Нина Владимировна работает в организации Волшебный Доктор на должности\n fНачальник архива, в филиале Управление (Архив , Финансовый департамент). Дата начала работы в организации: 2010-03-12. Дата рождения: 1968-11-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гршиина Татьяна Андреевна'}, {'from': 'gpt', 'value': 'Гршиина Татьяна Андреевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2018-07-16. Дата рождения: 1995-06-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гморков Андрей Владимирович'}, {'from': 'gpt', 'value': 'Гморков Андрей Владимирович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2016-08-24. Дата рождения: 1981-05-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Грудзов Михаил Михайлович'}, {'from': 'gpt', 'value': 'Грудзов Михаил Михайлович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф02 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2021-04-19. Дата рождения: 1991-08-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гуабренкова Вероника Сергеевна'}, {'from': 'gpt', 'value': 'Гуабренкова Вероника Сергеевна работает в организации Волшебный Доктор на должности\n fЗаведующий клинико-диагностическим отделением, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2016-10-17. Дата рождения: 1980-05-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гузвееа Екатерина Алексеевна'}, {'from': 'gpt', 'value': 'Гузвееа Екатерина Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале ГЦ-Ф15 (Отделение ВОП/педиатрии, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2022-05-30. Дата рождения: 1992-12-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гилуя Херсина Херсоновна'}, {'from': 'gpt', 'value': 'Гилуя Херсина Херсоновна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2024-09-23. Дата рождения: 1982-03-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Груеев Данил Николаевич'}, {'from': 'gpt', 'value': 'Груеев Данил Николаевич работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-02-19. Дата рождения: 1997-04-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гирун Борис Семенович'}, {'from': 'gpt', 'value': 'Гирун Борис Семенович работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-02-17. Дата рождения: 1971-05-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гсуев Андрей Иванович'}, {'from': 'gpt', 'value': 'Гсуев Андрей Иванович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2023-11-02. Дата рождения: 1993-11-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гсуев Григорий Сергеевич'}, {'from': 'gpt', 'value': 'Гсуев Григорий Сергеевич работает в организации Волшебный Доктор на должности\n fВедущий видеограф-монтажер, в филиале Управление (Отдел медиа-проектов, Департамент по работе с персоналом). Дата начала работы в организации: 2024-10-24. Дата рождения: 1996-05-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гуесва Алина Андреевна'}, {'from': 'gpt', 'value': 'Гуесва Алина Андреевна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-08-26. Дата рождения: 1998-12-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гувеса Анастасия Евгеньевна'}, {'from': 'gpt', 'value': 'Гувеса Анастасия Евгеньевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2013-06-05. Дата рождения: 1987-02-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гсуева Татьяна Сергеевна'}, {'from': 'gpt', 'value': 'Гсуева Татьяна Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2016-08-22. Дата рождения: 1991-06-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гуйеснов Аскер Магомедтагирович'}, {'from': 'gpt', 'value': 'Гуйеснов Аскер Магомедтагирович работает в организации Волшебный Доктор на должности\n fВрач по лечебной физкультуре, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-12-10. Дата рождения: 1997-12-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Гуесйнова Ольга Юрьевна'}, {'from': 'gpt', 'value': 'Гуесйнова Ольга Юрьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2020-11-24. Дата рождения: 1976-07-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Давкденио Елена Борисовна'}, {'from': 'gpt', 'value': 'Давкденио Елена Борисовна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2020-10-05. Дата рождения: 1986-07-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Давединко Наталья Васильевна'}, {'from': 'gpt', 'value': 'Давединко Наталья Васильевна работает в организации Волшебный Доктор на должности\n fВрач-ревматолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2017-07-19. Дата рождения: 1951-12-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Давыодва Анна Николаевна'}, {'from': 'gpt', 'value': 'Давыодва Анна Николаевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале ГЦ-Ф15 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-01-29. Дата рождения: 1978-02-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Давыдова Наталья Константиновна'}, {'from': 'gpt', 'value': 'Давыдова Наталья Константиновна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2025-02-24. Дата рождения: 1983-09-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Даоилнв Борис Дмитриевич'}, {'from': 'gpt', 'value': 'Даоилнв Борис Дмитриевич работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-хирург, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-03-11. Дата рождения: 1996-09-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Даинлова Мария Константиновна'}, {'from': 'gpt', 'value': 'Даинлова Мария Константиновна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2007-09-11. Дата рождения: 1975-10-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Длниаюк Виталий Степанович'}, {'from': 'gpt', 'value': 'Длниаюк Виталий Степанович работает в организации Волшебный Доктор на должности\n fТехник, в филиале ГЦ-Ф15 (Отдел административно-хозяйственного и технического обеспечения, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-11-17. Дата рождения: 1974-04-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Данцнрааова Лилия Олеговна'}, {'from': 'gpt', 'value': 'Данцнрааова Лилия Олеговна работает в организации Волшебный Доктор на должности\n fСпециалист по тестированию ПО, в филиале Управление (Отдел тестирования, Центр разработки ПО). Дата начала работы в организации: 2022-11-07. Дата рождения: 1996-01-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дегтявёра Алина Романовна'}, {'from': 'gpt', 'value': 'Дегтявёра Алина Романовна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2023-10-12. Дата рождения: 1997-01-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Декарнова Татьяна Ивановна'}, {'from': 'gpt', 'value': 'Декарнова Татьяна Ивановна работает в организации Волшебный Доктор на должности\n fМаркетолог-аналитик, в филиале Управление (Отдел рекламы, Департамент маркетинга и рекламы). Дата начала работы в организации: 2013-05-14. Дата рождения: 1987-06-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Деменчук Николай Викторович'}, {'from': 'gpt', 'value': 'Деменчук Николай Викторович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2020-02-25. Дата рождения: 1973-08-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Деоидмва Татьяна Александровна'}, {'from': 'gpt', 'value': 'Деоидмва Татьяна Александровна работает в организации Волшебный Доктор на должности\n fКоординатор пульта, в филиале ГЦ-Ф15 (Медицинский пульт, Коммерческий департамент ГЦ). Дата начала работы в организации: 2024-01-22. Дата рождения: 1973-09-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дечменко Михаил Игоревич'}, {'from': 'gpt', 'value': 'Дечменко Михаил Игоревич работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2024-02-26. Дата рождения: 2002-10-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Денесинко Ирина Валериевна'}, {'from': 'gpt', 'value': 'Денесинко Ирина Валериевна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф09 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2025-01-27. Дата рождения: 1982-04-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Деревяк Юлия Сергеевна'}, {'from': 'gpt', 'value': 'Деревяк Юлия Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-косметолог, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2015-02-11. Дата рождения: 1990-08-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Деригоазлв Иван Александрович'}, {'from': 'gpt', 'value': 'Деригоазлв Иван Александрович работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2021-04-05. Дата рождения: 1977-10-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дерниа Кристина Артемовна'}, {'from': 'gpt', 'value': 'Дерниа Кристина Артемовна работает в организации Волшебный Доктор на должности\n fБухгалтер по учету основных средств, в филиале Управление (Отдел учета расходов, Финансовый департамент). Дата начала работы в организации: 2015-10-26. Дата рождения: 1986-03-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Джаамлханов Нияз Пирмагомедович'}, {'from': 'gpt', 'value': 'Джаамлханов Нияз Пирмагомедович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф02 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2017-06-01. Дата рождения: 1963-01-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Джахалманова Гаджибике Низамитдиновна'}, {'from': 'gpt', 'value': 'Джахалманова Гаджибике Низамитдиновна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2006-04-24. Дата рождения: 1966-11-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Джхмалаанова Заира Мехрабовна'}, {'from': 'gpt', 'value': 'Джхмалаанова Заира Мехрабовна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф10 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2012-10-17. Дата рождения: 1988-12-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Джихалова Сабина Камрановна'}, {'from': 'gpt', 'value': 'Джихалова Сабина Камрановна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2025-01-30. Дата рождения: 1993-02-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Джоншвеоа Оджатбегим Шошарафовна'}, {'from': 'gpt', 'value': 'Джоншвеоа Оджатбегим Шошарафовна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2023-05-15. Дата рождения: 1992-10-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Длалио Жан Алиович'}, {'from': 'gpt', 'value': 'Длалио Жан Алиович работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф12 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2014-01-29. Дата рождения: 1985-02-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Доип Екатерина Сергеевна'}, {'from': 'gpt', 'value': 'Доип Екатерина Сергеевна работает в организации Волшебный Доктор на должности\n fОператор, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2024-09-16. Дата рождения: 1978-11-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дтсиер Анна Юрьевна'}, {'from': 'gpt', 'value': 'Дтсиер Анна Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2022-06-14. Дата рождения: 1986-11-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дмиеритва Алёна Александровна'}, {'from': 'gpt', 'value': 'Дмиеритва Алёна Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф14 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-10-10. Дата рождения: 2002-04-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дтимриева Валентина Александровна'}, {'from': 'gpt', 'value': 'Дтимриева Валентина Александровна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале ГЦ-Ф15 (Отделение ВОП/педиатрии, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2012-10-22. Дата рождения: 1955-02-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Деитримва Елизавета Максимовна'}, {'from': 'gpt', 'value': 'Деитримва Елизавета Максимовна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2015-08-11. Дата рождения: 1957-08-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дмивриета Мария Владимировна'}, {'from': 'gpt', 'value': 'Дмивриета Мария Владимировна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-03-04. Дата рождения: 1983-09-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Двогалёв Андрей Олегович'}, {'from': 'gpt', 'value': 'Двогалёв Андрей Олегович работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2019-09-09. Дата рождения: 1992-06-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Доганида Елена Руслановна'}, {'from': 'gpt', 'value': 'Доганида Елена Руслановна работает в организации Волшебный Доктор на должности\n fМенеджер стоматологического центра сменный, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-06-17. Дата рождения: 1992-09-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Доглова Ирина Степановна'}, {'from': 'gpt', 'value': 'Доглова Ирина Степановна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале ГЦ-Ф15 (Отделение гинекологии №1, Стационар). Дата начала работы в организации: 2020-06-22. Дата рождения: 1973-03-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дглоова Надежда Валериановна'}, {'from': 'gpt', 'value': 'Дглоова Надежда Валериановна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Отдел госпитальных продаж, Коммерческий департамент). Дата начала работы в организации: 2019-11-07. Дата рождения: 1987-12-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дмлоатова Любовь Николаевна'}, {'from': 'gpt', 'value': 'Дмлоатова Любовь Николаевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2009-10-29. Дата рождения: 1984-03-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Доиотлн Ринат Джиганшаевич'}, {'from': 'gpt', 'value': 'Доиотлн Ринат Джиганшаевич работает в организации Волшебный Доктор на должности\n fВрач-эндоскопист, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-12-03. Дата рождения: 1989-12-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Домоакця Лидия Ивановна'}, {'from': 'gpt', 'value': 'Домоакця Лидия Ивановна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф12 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1967-03-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Донакся Екатерина Сергеевна'}, {'from': 'gpt', 'value': 'Донакся Екатерина Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2014-12-08. Дата рождения: 1988-01-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дочненко Нина Павловна'}, {'from': 'gpt', 'value': 'Дочненко Нина Павловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2015-11-11. Дата рождения: 1961-10-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Джроокина Лилия Мубараковна'}, {'from': 'gpt', 'value': 'Джроокина Лилия Мубараковна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2021-08-09. Дата рождения: 1973-08-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дороинна Елизавета Владимировна'}, {'from': 'gpt', 'value': 'Дороинна Елизавета Владимировна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2017-10-02. Дата рождения: 1992-05-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дорвхооа Тамара Ивановна'}, {'from': 'gpt', 'value': 'Дорвхооа Тамара Ивановна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале ГЦ-Ф15 (Отдел административно-хозяйственного и технического обеспечения, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-10-01. Дата рождения: 1963-05-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дроощенко Елена Владимировна'}, {'from': 'gpt', 'value': 'Дроощенко Елена Владимировна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2018-10-17. Дата рождения: 1975-06-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дровона Елена Вячеславовна'}, {'from': 'gpt', 'value': 'Дровона Елена Вячеславовна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2018-06-14. Дата рождения: 1974-09-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дрижунина Анастасия Федоровна'}, {'from': 'gpt', 'value': 'Дрижунина Анастасия Федоровна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2017-05-24. Дата рождения: 1996-08-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дубкнео Татьяна Анатольевна'}, {'from': 'gpt', 'value': 'Дубкнео Татьяна Анатольевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2011-09-27. Дата рождения: 1963-08-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дунибина Ирина Александровна'}, {'from': 'gpt', 'value': 'Дунибина Ирина Александровна работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф06 (Администрация, Администрация). Дата начала работы в организации: 2015-12-02. Дата рождения: 1981-03-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дугниов Дмитрий Владиславович'}, {'from': 'gpt', 'value': 'Дугниов Дмитрий Владиславович работает в организации Волшебный Доктор на должности\n fЗаместитель руководителя отдела закупок, в филиале Управление (Отдел закупок, Медицинский департамент). Дата начала работы в организации: 2015-09-01. Дата рождения: 1992-09-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Думянан Карен Размикович'}, {'from': 'gpt', 'value': 'Думянан Карен Размикович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф16 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2016-02-17. Дата рождения: 1984-12-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Думкиян Гаянэ Мисаковна'}, {'from': 'gpt', 'value': 'Думкиян Гаянэ Мисаковна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2012-01-12. Дата рождения: 1964-07-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дыобвская Екатерина Михайловна'}, {'from': 'gpt', 'value': 'Дыобвская Екатерина Михайловна работает в организации Волшебный Доктор на должности\n fНачальник отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2010-09-03. Дата рождения: 1986-01-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дышевока Милана Зарифовна'}, {'from': 'gpt', 'value': 'Дышевока Милана Зарифовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-08-08. Дата рождения: 1997-12-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дышеоква Милена Зарифовна'}, {'from': 'gpt', 'value': 'Дышеоква Милена Зарифовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2021-05-11. Дата рождения: 1999-07-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дьявкоча Ирина Вячеславовна'}, {'from': 'gpt', 'value': 'Дьявкоча Ирина Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач функциональной диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-04-19. Дата рождения: 1971-08-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Диндюк Светлана Петровна'}, {'from': 'gpt', 'value': 'Диндюк Светлана Петровна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф02 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2017-06-14. Дата рождения: 1956-07-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Дядниа Елена Борисовна'}, {'from': 'gpt', 'value': 'Дядниа Елена Борисовна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2018-08-20. Дата рождения: 1983-03-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еагрвфов Дмитрий Александрович'}, {'from': 'gpt', 'value': 'Еагрвфов Дмитрий Александрович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2025-01-20. Дата рождения: 1995-05-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еввенома Наталья Валерьевна'}, {'from': 'gpt', 'value': 'Еввенома Наталья Валерьевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2020-08-17. Дата рождения: 1973-09-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Евсюнниа Татьяна Алексеевна'}, {'from': 'gpt', 'value': 'Евсюнниа Татьяна Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-косметолог, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2017-05-17. Дата рождения: 1987-11-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Евтешунко Елена Александровна'}, {'from': 'gpt', 'value': 'Евтешунко Елена Александровна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2023-10-23. Дата рождения: 1969-06-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Евтушенго-Сикаева Мария Вячеславовна'}, {'from': 'gpt', 'value': 'Евтушенго-Сикаева Мария Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2012-10-03. Дата рождения: 1976-02-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ериазагян Джульетта Кареновна'}, {'from': 'gpt', 'value': 'Ериазагян Джульетта Кареновна работает в организации Волшебный Доктор на должности\n fВрач-ординатор, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2024-09-02. Дата рождения: 1999-06-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еогров Владимир Анатольевич'}, {'from': 'gpt', 'value': 'Еогров Владимир Анатольевич работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2021-04-05. Дата рождения: 1964-07-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еогров Владислав Михайлович'}, {'from': 'gpt', 'value': 'Еогров Владислав Михайлович работает в организации Волшебный Доктор на должности\n fГлавный врач-кардиолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2012-02-07. Дата рождения: 1969-01-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Егврооа Анастасия Ярославовна'}, {'from': 'gpt', 'value': 'Егврооа Анастасия Ярославовна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2020-08-27. Дата рождения: 1971-01-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ежвоа Кристина Мушеговна'}, {'from': 'gpt', 'value': 'Ежвоа Кристина Мушеговна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2024-01-09. Дата рождения: 1997-02-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Езвеиа Мадина Мухамедовна'}, {'from': 'gpt', 'value': 'Езвеиа Мадина Мухамедовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2025-02-06. Дата рождения: 1983-02-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еикмова Татьяна Владимировна'}, {'from': 'gpt', 'value': 'Еикмова Татьяна Владимировна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2022-04-11. Дата рождения: 1978-07-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Елника Мария Виталиевна'}, {'from': 'gpt', 'value': 'Елника Мария Виталиевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф01 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-02-08. Дата рождения: 1993-06-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Елмнаова Анна Владимировна'}, {'from': 'gpt', 'value': 'Елмнаова Анна Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Педиатрическое отделение, Стационар). Дата начала работы в организации: 2017-05-24. Дата рождения: 1989-08-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Емлеьяненко Михаил Владимирович'}, {'from': 'gpt', 'value': 'Емлеьяненко Михаил Владимирович работает в организации Волшебный Доктор на должности\n fВрач по рентгенэндоваскулярным диагностике и лечению, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-10-28. Дата рождения: 1980-12-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Емеяьлнова Алина Михайловна'}, {'from': 'gpt', 'value': 'Емеяьлнова Алина Михайловна работает в организации Волшебный Доктор на должности\n fНачальник отдела медицинских проектов и внедрений, в филиале Управление (Отдел медицинских проектов и внедрений, Департамент Специализированных проектов). Дата начала работы в организации: 2009-12-07. Дата рождения: 1984-02-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еьелмянова Галина Юрьевна'}, {'from': 'gpt', 'value': 'Еьелмянова Галина Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2010-10-11. Дата рождения: 1961-05-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еоельянмва Людмила Викторовна'}, {'from': 'gpt', 'value': 'Еоельянмва Людмила Викторовна работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2022-03-14. Дата рождения: 1989-03-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еиемрн Дмитрий Сергеевич'}, {'from': 'gpt', 'value': 'Еиемрн Дмитрий Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2017-02-08. Дата рождения: 1989-06-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Емерин Кирилл Владимирович'}, {'from': 'gpt', 'value': 'Емерин Кирилл Владимирович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2015-10-12. Дата рождения: 1993-09-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еримена Валентина Антоновна'}, {'from': 'gpt', 'value': 'Еримена Валентина Антоновна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-05-23. Дата рождения: 1965-04-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ёнира Наталья Викторовна'}, {'from': 'gpt', 'value': 'Ёнира Наталья Викторовна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф03 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2018-06-27. Дата рождения: 1970-05-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ерлимова Татьяна Арнольдовна'}, {'from': 'gpt', 'value': 'Ерлимова Татьяна Арнольдовна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф14 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2022-09-29. Дата рождения: 1968-09-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ерломаев Андрей Владимирович'}, {'from': 'gpt', 'value': 'Ерломаев Андрей Владимирович работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-03-19. Дата рождения: 1997-04-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Екмолинсрая Эмма Андреевна'}, {'from': 'gpt', 'value': 'Екмолинсрая Эмма Андреевна работает в организации Волшебный Доктор на должности\n fРуководитель направления амбулаторной травматологии и ортопедии, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2024-09-03. Дата рождения: 1988-06-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еромшкина Альбина Наильевна'}, {'from': 'gpt', 'value': 'Еромшкина Альбина Наильевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу участковая, в филиале Ф17-ДЦ (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-02-08. Дата рождения: 1982-06-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ервоша Влада Сергеевна'}, {'from': 'gpt', 'value': 'Ервоша Влада Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-03-09. Дата рождения: 1992-10-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Еошрва Дарья Сергеевна'}, {'from': 'gpt', 'value': 'Еошрва Дарья Сергеевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2020-11-23. Дата рождения: 1993-10-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ескуленао Светлана Николаевна'}, {'from': 'gpt', 'value': 'Ескуленао Светлана Николаевна работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2019-02-13. Дата рождения: 1969-03-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ескиова Валерия Михайловна'}, {'from': 'gpt', 'value': 'Ескиова Валерия Михайловна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2015-06-17. Дата рождения: 1979-03-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Есвпоиа Елена Петровна'}, {'from': 'gpt', 'value': 'Есвпоиа Елена Петровна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2007-10-03. Дата рождения: 1977-09-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ефомива Жанна Ивановна'}, {'from': 'gpt', 'value': 'Ефомива Жанна Ивановна работает в организации Волшебный Доктор на должности\n fЭстетист, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2018-11-21. Дата рождения: 1974-03-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ефомива Татьяна Владимировна'}, {'from': 'gpt', 'value': 'Ефомива Татьяна Владимировна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-03-23. Дата рождения: 1971-12-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ефмеров Сергей Александрович'}, {'from': 'gpt', 'value': 'Ефмеров Сергей Александрович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-хирург, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2016-11-14. Дата рождения: 1987-03-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жнвороакова Светлана Алексеевна'}, {'from': 'gpt', 'value': 'Жнвороакова Светлана Алексеевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2021-09-13. Дата рождения: 1977-04-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жкраих Татьяна Геннадьевна'}, {'from': 'gpt', 'value': 'Жкраих Татьяна Геннадьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф09 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2012-09-19. Дата рождения: 1973-01-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жакрова Ольга Геннадьевна'}, {'from': 'gpt', 'value': 'Жакрова Ольга Геннадьевна работает в организации Волшебный Доктор на должности\n fВедущий системный аналитик, в филиале Управление (Центр разработки ПО, Центр разработки ПО). Дата начала работы в организации: 2022-03-21. Дата рождения: 1996-05-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ждаонв Михаил Александрович'}, {'from': 'gpt', 'value': 'Ждаонв Михаил Александрович работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2024-10-21. Дата рождения: 1991-07-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жолезнева Наталья Ивановна'}, {'from': 'gpt', 'value': 'Жолезнева Наталья Ивановна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф02 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2009-07-09. Дата рождения: 1962-06-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Желтывеша Евгения Васильевна'}, {'from': 'gpt', 'value': 'Желтывеша Евгения Васильевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф06 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2020-11-12. Дата рождения: 1982-06-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жемалетдивона Карина Римовна'}, {'from': 'gpt', 'value': 'Жемалетдивона Карина Римовна работает в организации Волшебный Доктор на должности\n fВрач-косметолог, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2020-09-07. Дата рождения: 1970-08-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жвицов Николай Владимирович'}, {'from': 'gpt', 'value': 'Жвицов Николай Владимирович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф17-ДЦ (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2016-07-26. Дата рождения: 1979-10-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жиоалгва Анастасия Ивановна'}, {'from': 'gpt', 'value': 'Жиоалгва Анастасия Ивановна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф03 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2007-08-01. Дата рождения: 1981-05-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жиеаргва Ирина Станиславовна'}, {'from': 'gpt', 'value': 'Жиеаргва Ирина Станиславовна работает в организации Волшебный Доктор на должности\n fВрач-онколог, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2021-07-27. Дата рождения: 1972-02-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жидквоа Светлана Юрьевна'}, {'from': 'gpt', 'value': 'Жидквоа Светлана Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2020-07-13. Дата рождения: 1985-03-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Журввлеаа Любовь Эдуардовна'}, {'from': 'gpt', 'value': 'Журввлеаа Любовь Эдуардовна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2021-10-25. Дата рождения: 1996-03-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жураввела Регина Исраиловна'}, {'from': 'gpt', 'value': 'Жураввела Регина Исраиловна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф01 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2025-01-16. Дата рождения: 1984-06-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Жураковакся Наталья Александровна'}, {'from': 'gpt', 'value': 'Жураковакся Наталья Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2025-03-04. Дата рождения: 1972-06-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Заеблина Ирина Константиновна'}, {'from': 'gpt', 'value': 'Заеблина Ирина Константиновна работает в организации Волшебный Доктор на должности\n fГлавный оператор, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2014-09-09. Дата рождения: 1981-06-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Забонита Елена Валерьевна'}, {'from': 'gpt', 'value': 'Забонита Елена Валерьевна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2024-01-15. Дата рождения: 1979-12-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Завааскдя Ольга Михайловна'}, {'from': 'gpt', 'value': 'Завааскдя Ольга Михайловна работает в организации Волшебный Доктор на должности\n fСпециалист Отдела кадров, в филиале Управление (Отдел кадров, Департамент по работе с персоналом). Дата начала работы в организации: 2024-12-12. Дата рождения: 1978-01-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зиака Любовь Сергеевна'}, {'from': 'gpt', 'value': 'Зиака Любовь Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2017-01-23. Дата рождения: 1990-12-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зайвеца Полина Владимировна'}, {'from': 'gpt', 'value': 'Зайвеца Полина Владимировна работает в организации Волшебный Доктор на должности\n fВрач-косметолог, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2016-01-27. Дата рождения: 1989-08-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зкаиева Наталья Ивановна'}, {'from': 'gpt', 'value': 'Зкаиева Наталья Ивановна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале ГЦ-Ф15 (Отделение ВОП/педиатрии, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2017-02-20. Дата рождения: 1988-09-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Залеаскся Наталия Викторовна'}, {'from': 'gpt', 'value': 'Залеаскся Наталия Викторовна работает в организации Волшебный Доктор на должности\n fКоординатор сервисных работ, в филиале Ф05 (Отдел административно-хозяйственного и технического обеспечения, Административный департамент). Дата начала работы в организации: 2021-01-19. Дата рождения: 1963-03-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Занумхалова Лейла Залимхановна'}, {'from': 'gpt', 'value': 'Занумхалова Лейла Залимхановна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2022-10-03. Дата рождения: 1997-07-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Заняткима Ирина Александровна'}, {'from': 'gpt', 'value': 'Заняткима Ирина Александровна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2018-03-21. Дата рождения: 1989-05-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ззноаин Антон Александрович'}, {'from': 'gpt', 'value': 'Ззноаин Антон Александрович работает в организации Волшебный Доктор на должности\n fВрач-травматолог-ортопед, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-02-15. Дата рождения: 1993-10-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Запольнова Марина Алексеевна'}, {'from': 'gpt', 'value': 'Запольнова Марина Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2007-04-24. Дата рождения: 1970-08-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зарекцая Елена Вячеславовна'}, {'from': 'gpt', 'value': 'Зарекцая Елена Вячеславовна работает в организации Волшебный Доктор на должности\n fСестра-хозяйка, в филиале ГЦ-Ф15 (Отдел хозобеспечения, Стационар). Дата начала работы в организации: 2015-05-19. Дата рождения: 1977-06-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Заруцакя Ольга Николаевна'}, {'from': 'gpt', 'value': 'Заруцакя Ольга Николаевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2018-09-03. Дата рождения: 1961-06-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Заханерко Андрей Алексеевич'}, {'from': 'gpt', 'value': 'Заханерко Андрей Алексеевич работает в организации Волшебный Доктор на должности\n fЗаведующий отделением скорой и неотложной медицинской помощи и приемным покоем - врач общей практики, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2021-12-13. Дата рождения: 1987-06-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зхааров Алексей Александрович'}, {'from': 'gpt', 'value': 'Зхааров Алексей Александрович работает в организации Волшебный Доктор на должности\n fГлавный врач Госпитального центра, в филиале ГЦ-Ф15 (Администрация, Стационар). Дата начала работы в организации: 2020-09-01. Дата рождения: 1978-02-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Захаорв Денис Викторович'}, {'from': 'gpt', 'value': 'Захаорв Денис Викторович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2024-08-15. Дата рождения: 1997-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Заоархв Павел Александрович'}, {'from': 'gpt', 'value': 'Заоархв Павел Александрович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2018-10-15. Дата рождения: 1996-08-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Захаорва Людмила Ивановна'}, {'from': 'gpt', 'value': 'Захаорва Людмила Ивановна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами, в филиале Управление (Отдел госпитальных продаж, Коммерческий департамент). Дата начала работы в организации: 2014-09-25. Дата рождения: 1980-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Заоарцхв Александр Витальевич'}, {'from': 'gpt', 'value': 'Заоарцхв Александр Витальевич работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-07-22. Дата рождения: 1995-04-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зацариыннй Владимир Викторович'}, {'from': 'gpt', 'value': 'Зацариыннй Владимир Викторович работает в организации Волшебный Доктор на должности\n fЗаведующий хирургическим отделением, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2022-04-25. Дата рождения: 1986-03-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Звдаа Арина Владиславовна'}, {'from': 'gpt', 'value': 'Звдаа Арина Владиславовна работает в организации Волшебный Доктор на должности\n fВрач-ординатор, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2024-09-02. Дата рождения: 1998-02-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зиарвч Леонид Степанович'}, {'from': 'gpt', 'value': 'Зиарвч Леонид Степанович работает в организации Волшебный Доктор на должности\n fВедущий врач-пластический хирург, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2023-06-05. Дата рождения: 1977-09-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Звездинова Юлия Вячеславовна'}, {'from': 'gpt', 'value': 'Звездинова Юлия Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2018-10-01. Дата рождения: 1992-01-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Званодич Сергей Федорович'}, {'from': 'gpt', 'value': 'Званодич Сергей Федорович работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2009-09-01. Дата рождения: 1959-11-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Знилаев Тимур Сагиевич'}, {'from': 'gpt', 'value': 'Знилаев Тимур Сагиевич работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2022-07-18. Дата рождения: 1987-03-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Золотова Людмила Николаевна'}, {'from': 'gpt', 'value': 'Золотова Людмила Николаевна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2023-08-14. Дата рождения: 1976-10-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Звльнооа Анна Аркадьевна'}, {'from': 'gpt', 'value': 'Звльнооа Анна Аркадьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по физиотерапии, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2024-11-25. Дата рождения: 1966-02-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зуобва Елена Альбертовна'}, {'from': 'gpt', 'value': 'Зуобва Елена Альбертовна работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2015-05-25. Дата рождения: 1965-03-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зурбицкая Наталия Александровна'}, {'from': 'gpt', 'value': 'Зурбицкая Наталия Александровна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2019-08-19. Дата рождения: 1967-06-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зунида Мария Валерьевна'}, {'from': 'gpt', 'value': 'Зунида Мария Валерьевна работает в организации Волшебный Доктор на должности\n fГлавная медицинская сестра, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2008-02-26. Дата рождения: 1972-05-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Звеуа Юлия Владимировна'}, {'from': 'gpt', 'value': 'Звеуа Юлия Владимировна работает в организации Волшебный Доктор на должности\n fСпециалист центра клиентского сервиса, в филиале Управление (Центр клиентского сервиса, Коммерческий департамент). Дата начала работы в организации: 2019-11-06. Дата рождения: 1977-03-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зуексвая Ксения Александровна'}, {'from': 'gpt', 'value': 'Зуексвая Ксения Александровна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-11-18. Дата рождения: 1994-11-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зуйквоа Надежда Леонидовна'}, {'from': 'gpt', 'value': 'Зуйквоа Надежда Леонидовна работает в организации Волшебный Доктор на должности\n fВрач-психотерапевт, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2021-06-01. Дата рождения: 1959-10-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зурндажи Елена Вячеславовна'}, {'from': 'gpt', 'value': 'Зурндажи Елена Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф14 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2006-03-22. Дата рождения: 1965-05-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Зкыов Андрей Валентинович'}, {'from': 'gpt', 'value': 'Зкыов Андрей Валентинович работает в организации Волшебный Доктор на должности\n fВрач-травматолог-ортопед, в филиале Ф01 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-02-13. Дата рождения: 1974-01-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ибваов Хаким Махачович'}, {'from': 'gpt', 'value': 'Ибваов Хаким Махачович работает в организации Волшебный Доктор на должности\n fФельдшер скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2021-02-24. Дата рождения: 1991-05-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ивалинова Ираида Геннадьевна'}, {'from': 'gpt', 'value': 'Ивалинова Ираида Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2025-02-17. Дата рождения: 1966-10-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Инавов Александр Петрович'}, {'from': 'gpt', 'value': 'Инавов Александр Петрович работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2020-02-17. Дата рождения: 1977-01-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иавнов Анатолий Сергеевич'}, {'from': 'gpt', 'value': 'Иавнов Анатолий Сергеевич работает в организации Волшебный Доктор на должности\n fРуководитель центра корпоративного обучения врачей, в филиале Управление (Центр корпоративного обучения врачей, Департамент Специализированных проектов). Дата начала работы в организации: 2021-11-08. Дата рождения: 1988-05-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ивнаов Андрей Николаевич'}, {'from': 'gpt', 'value': 'Ивнаов Андрей Николаевич работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале ГЦ-Ф15 (Отделение ВОП/педиатрии, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2025-02-24. Дата рождения: 1971-12-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иавнов Илья Юрьевич'}, {'from': 'gpt', 'value': 'Иавнов Илья Юрьевич работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф01 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-12-14. Дата рождения: 1968-05-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ивнаов Павел Викторович'}, {'from': 'gpt', 'value': 'Ивнаов Павел Викторович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-хирург, в филиале Ф16 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2017-02-01. Дата рождения: 1990-07-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Инавов Сергей Александрович'}, {'from': 'gpt', 'value': 'Инавов Сергей Александрович работает в организации Волшебный Доктор на должности\n fЗаместитель руководителя отдела продаж, в филиале Управление (Отдел по работе с физическими лицами, Коммерческий департамент). Дата начала работы в организации: 2023-05-02. Дата рождения: 1986-07-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ивнаов Ярослав Игоревич'}, {'from': 'gpt', 'value': 'Ивнаов Ярослав Игоревич работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф02 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2022-02-14. Дата рождения: 1992-02-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иввноаа Ангелина Юрьевна'}, {'from': 'gpt', 'value': 'Иввноаа Ангелина Юрьевна работает в организации Волшебный Доктор на должности\n fВрач клинической лабораторной диагностики, в филиале Лаборатория (Администрация, Администрация). Дата начала работы в организации: 2019-04-22. Дата рождения: 1987-06-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иванвоа Галина Владимировна'}, {'from': 'gpt', 'value': 'Иванвоа Галина Владимировна работает в организации Волшебный Доктор на должности\n fСтарший оператор, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2020-07-23. Дата рождения: 1977-10-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иавнова Зинаида Александровна'}, {'from': 'gpt', 'value': 'Иавнова Зинаида Александровна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф12 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2014-08-04. Дата рождения: 1955-02-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иваонва Ирина Евгеньевна'}, {'from': 'gpt', 'value': 'Иваонва Ирина Евгеньевна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2019-03-27. Дата рождения: 1958-09-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иоанвва Ирина Николаевна'}, {'from': 'gpt', 'value': 'Иоанвва Ирина Николаевна работает в организации Волшебный Доктор на должности\n fВторой заместитель генерального директора, в филиале Управление (Аппарат управления, Аппарат управления). Дата начала работы в организации: 2007-09-01. Дата рождения: 1974-06-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Инавова Наталья Владимировна'}, {'from': 'gpt', 'value': 'Инавова Наталья Владимировна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф01 (Отделение педиатрии, Отделение педиатрии). Дата начала работы в организации: 2010-10-06. Дата рождения: 1969-12-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Инавова Ольга Александровна'}, {'from': 'gpt', 'value': 'Инавова Ольга Александровна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2013-03-28. Дата рождения: 1958-07-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иваонва Ольга Сергеевна'}, {'from': 'gpt', 'value': 'Иваонва Ольга Сергеевна работает в организации Волшебный Доктор на должности\n fДиректор департамента организационного развития и контроля, в филиале Управление (Департамент организационного развития и контроля, Департамент организационного развития и контроля). Дата начала работы в организации: 2008-09-15. Дата рождения: 1983-07-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ивавона Юлия Владимировна'}, {'from': 'gpt', 'value': 'Ивавона Юлия Владимировна работает в организации Волшебный Доктор на должности\n fВрач-детский эндокринолог, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2021-06-01. Дата рождения: 1975-06-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ивщаенко Анна Владимировна'}, {'from': 'gpt', 'value': 'Ивщаенко Анна Владимировна работает в организации Волшебный Доктор на должности\n fЗаместитель директора медицинского департамента, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2018-09-12. Дата рождения: 1990-06-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Илвев Илья Сергеевич'}, {'from': 'gpt', 'value': 'Илвев Илья Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2023-08-07. Дата рождения: 1992-06-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иншива Жанна Андреевна'}, {'from': 'gpt', 'value': 'Иншива Жанна Андреевна работает в организации Волшебный Доктор на должности\n fСпециалист отдела подбора и адаптации, в филиале Управление (Отдел подбора и адаптации, Департамент по работе с персоналом). Дата начала работы в организации: 2025-02-17. Дата рождения: 2001-08-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Игватона Наталья Витальевна'}, {'from': 'gpt', 'value': 'Игватона Наталья Витальевна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-01-11. Дата рождения: 1978-04-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Игантьева Ксения Григорьевна'}, {'from': 'gpt', 'value': 'Игантьева Ксения Григорьевна работает в организации Волшебный Доктор на должности\n fВрач-сексолог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2024-08-05. Дата рождения: 1998-05-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иеажва Зухра Абдуловна'}, {'from': 'gpt', 'value': 'Иеажва Зухра Абдуловна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2020-08-10. Дата рождения: 1979-11-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Илиьна Наталья Геннадьевна'}, {'from': 'gpt', 'value': 'Илиьна Наталья Геннадьевна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2022-02-14. Дата рождения: 1995-11-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ильюшкниа Валерия Олеговна'}, {'from': 'gpt', 'value': 'Ильюшкниа Валерия Олеговна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2021-08-30. Дата рождения: 2001-08-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иаммова Мадина Муратовна'}, {'from': 'gpt', 'value': 'Иаммова Мадина Муратовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф16 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-10-28. Дата рождения: 2005-04-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Инемных Маргарита Владимировна'}, {'from': 'gpt', 'value': 'Инемных Маргарита Владимировна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2023-01-30. Дата рождения: 1972-08-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иокдо Ярослав Владимирович'}, {'from': 'gpt', 'value': 'Иокдо Ярослав Владимирович работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале ГЦ-Ф15 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-10-17. Дата рождения: 2003-05-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Исеав Алексей Владимирович'}, {'from': 'gpt', 'value': 'Исеав Алексей Владимирович работает в организации Волшебный Доктор на должности\n fПерсональный водитель автомобиля руководителя, в филиале Управление (Автопарк дирекции, Аппарат дирекции). Дата начала работы в организации: 2009-08-03. Дата рождения: 1970-06-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Иваеса Алла Семеновна'}, {'from': 'gpt', 'value': 'Иваеса Алла Семеновна работает в организации Волшебный Доктор на должности\n fВрач функциональной диагностики, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2018-10-29. Дата рождения: 1962-01-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Икайсина Татьяна Сергеевна'}, {'from': 'gpt', 'value': 'Икайсина Татьяна Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра кабинета подологии, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2024-03-25. Дата рождения: 1979-06-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Исанечко Татьяна Викторовна'}, {'from': 'gpt', 'value': 'Исанечко Татьяна Викторовна работает в организации Волшебный Доктор на должности\n fМенеджер по плановой госпитализации сменный, в филиале ГЦ-Ф15 (Отдел госпитализаций и сервиса, Коммерческий департамент ГЦ). Дата начала работы в организации: 2022-12-12. Дата рождения: 1983-05-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Имсаилов Абдулла Исмаилович'}, {'from': 'gpt', 'value': 'Имсаилов Абдулла Исмаилович работает в организации Волшебный Доктор на должности\n fВрач-травматолог-ортопед, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-08-12. Дата рождения: 1992-06-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Исамилова Марьям Гусейн-Магомедовна'}, {'from': 'gpt', 'value': 'Исамилова Марьям Гусейн-Магомедовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф10 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-09-09. Дата рождения: 2002-12-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кабавеа Мария Сергеевна'}, {'from': 'gpt', 'value': 'Кабавеа Мария Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф03 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-01-25. Дата рождения: 1979-01-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Каблаин Михаил Анатольевич'}, {'from': 'gpt', 'value': 'Каблаин Михаил Анатольевич работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу участковая, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2015-05-06. Дата рождения: 1983-10-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кадыквоа Людмила Викторовна'}, {'from': 'gpt', 'value': 'Кадыквоа Людмила Викторовна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2023-10-12. Дата рождения: 1973-02-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кадырвоа Евгения Владимировна'}, {'from': 'gpt', 'value': 'Кадырвоа Евгения Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф02 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-08-26. Дата рождения: 2000-09-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Казвкоаа Кристина Валерьевна'}, {'from': 'gpt', 'value': 'Казвкоаа Кристина Валерьевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-04-01. Дата рождения: 1982-01-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Каазнцев Сергей Викторович'}, {'from': 'gpt', 'value': 'Каазнцев Сергей Викторович работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-01-11. Дата рождения: 1961-03-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кзаанцева Алёна Юрьевна'}, {'from': 'gpt', 'value': 'Кзаанцева Алёна Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2023-08-14. Дата рождения: 1994-05-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кезанцава Оксана Александровна'}, {'from': 'gpt', 'value': 'Кезанцава Оксана Александровна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2019-12-12. Дата рождения: 1975-03-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Криаов Заур Маремович'}, {'from': 'gpt', 'value': 'Криаов Заур Маремович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-07-08. Дата рождения: 1990-08-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кайномова Надежда Спиридоновна'}, {'from': 'gpt', 'value': 'Кайномова Надежда Спиридоновна работает в организации Волшебный Доктор на должности\n fПомощник менеджера стоматологического центра, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2025-02-03. Дата рождения: 1975-08-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кайтуквоа Зарина Владимировна'}, {'from': 'gpt', 'value': 'Кайтуквоа Зарина Владимировна работает в организации Волшебный Доктор на должности\n fРуководитель отдела закупок, в филиале ГЦ-Ф15 (Отдел хозобеспечения, Стационар). Дата начала работы в организации: 2019-03-14. Дата рождения: 1980-10-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Калишнаков Валерий Игоревич'}, {'from': 'gpt', 'value': 'Калишнаков Валерий Игоревич работает в организации Волшебный Доктор на должности\n fВрач-инфекционист, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-03-23. Дата рождения: 1986-11-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кглаанова Юлия Константиновна'}, {'from': 'gpt', 'value': 'Кглаанова Юлия Константиновна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-05-02. Дата рождения: 2000-08-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Калннииа Виктория Валерьевна'}, {'from': 'gpt', 'value': 'Калннииа Виктория Валерьевна работает в организации Волшебный Доктор на должности\n fМенеджер по мониторингу, в филиале Управление (Отдел мониторинга, Департамент мониторинга и внутреннего контроля). Дата начала работы в организации: 2007-09-18. Дата рождения: 1986-08-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Киланина Елена Викторовна'}, {'from': 'gpt', 'value': 'Киланина Елена Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по физиотерапии, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2024-10-07. Дата рождения: 1980-03-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Канилина Кристина Владимировна'}, {'from': 'gpt', 'value': 'Канилина Кристина Владимировна работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Ф02 (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2010-11-29. Дата рождения: 1985-08-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Калымков Дмитрий Алексеевич'}, {'from': 'gpt', 'value': 'Калымков Дмитрий Алексеевич работает в организации Волшебный Доктор на должности\n fКлинический психолог, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2023-10-02. Дата рождения: 1995-03-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Калкымова Александра Михайловна'}, {'from': 'gpt', 'value': 'Калкымова Александра Михайловна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф02 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-11-05. Дата рождения: 1998-09-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кулагина Галина Яковлевна'}, {'from': 'gpt', 'value': 'Кулагина Галина Яковлевна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф07 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1951-02-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Калуигна Екатерина Викторовна'}, {'from': 'gpt', 'value': 'Калуигна Екатерина Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-10-01. Дата рождения: 1981-07-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кагялин Данил Александрович'}, {'from': 'gpt', 'value': 'Кагялин Данил Александрович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2016-12-12. Дата рождения: 1986-11-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Квмалоаа Ольга Вячеславовна'}, {'from': 'gpt', 'value': 'Квмалоаа Ольга Вячеславовна работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача, в филиале Ф01 (Администрация, Администрация). Дата начала работы в организации: 2012-10-01. Дата рождения: 1972-06-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кмаенева Ирина Викторовна'}, {'from': 'gpt', 'value': 'Кмаенева Ирина Викторовна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф07 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1961-05-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кыманина Анастасия Викторовна'}, {'from': 'gpt', 'value': 'Кыманина Анастасия Викторовна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2023-09-04. Дата рождения: 1997-04-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Каннвчеако Ирина Егоровна'}, {'from': 'gpt', 'value': 'Каннвчеако Ирина Егоровна работает в организации Волшебный Доктор на должности\n fБуфетчица, в филиале ГЦ-Ф15 (Служба сервиса, Стационар). Дата начала работы в организации: 2020-10-15. Дата рождения: 1962-11-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Квниаец Алла Викторовна'}, {'from': 'gpt', 'value': 'Квниаец Алла Викторовна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2020-09-02. Дата рождения: 1976-12-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Капитоонва Оксана Викторовна'}, {'from': 'gpt', 'value': 'Капитоонва Оксана Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2018-09-19. Дата рождения: 1981-04-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Катуспина Ирина Петровна'}, {'from': 'gpt', 'value': 'Катуспина Ирина Петровна работает в организации Волшебный Доктор на должности\n fСпециалист Отдела кадров, в филиале Управление (Отдел кадров, Департамент по работе с персоналом). Дата начала работы в организации: 2013-08-06. Дата рождения: 1974-10-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Каупстина Ольга Сергеевна'}, {'from': 'gpt', 'value': 'Каупстина Ольга Сергеевна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2025-01-23. Дата рождения: 1985-10-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Капуснита Светлана Олеговна'}, {'from': 'gpt', 'value': 'Капуснита Светлана Олеговна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2020-02-26. Дата рождения: 1980-07-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Краа Юлия Алексеевна'}, {'from': 'gpt', 'value': 'Краа Юлия Алексеевна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2022-03-14. Дата рождения: 1996-12-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Каоакрй Елизавета Александровна'}, {'from': 'gpt', 'value': 'Каоакрй Елизавета Александровна работает в организации Волшебный Доктор на должности\n fСекретарь дирекции, в филиале Управление (Секретариат, Аппарат дирекции). Дата начала работы в организации: 2021-09-27. Дата рождения: 1995-10-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Каряпетан Элина Арутюновна'}, {'from': 'gpt', 'value': 'Каряпетан Элина Арутюновна работает в организации Волшебный Доктор на должности\n fСтарший менеджер по плановой госпитализации, в филиале ГЦ-Ф15 (Отдел госпитализаций и сервиса, Коммерческий департамент ГЦ). Дата начала работы в организации: 2013-01-10. Дата рождения: 1976-07-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кавера Людмила Вацлавовна'}, {'from': 'gpt', 'value': 'Кавера Людмила Вацлавовна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2023-11-20. Дата рождения: 1967-08-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кавера Оксана Александровна'}, {'from': 'gpt', 'value': 'Кавера Оксана Александровна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2023-08-21. Дата рождения: 1990-01-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Краелина Анна Валерьевна'}, {'from': 'gpt', 'value': 'Краелина Анна Валерьевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2016-02-15. Дата рождения: 1983-06-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Краимова Гюзаль Ильдаровна'}, {'from': 'gpt', 'value': 'Краимова Гюзаль Ильдаровна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2012-11-21. Дата рождения: 1973-04-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Корпав Андрей Владимирович'}, {'from': 'gpt', 'value': 'Корпав Андрей Владимирович работает в организации Волшебный Доктор на должности\n fКопирайтер, в филиале Управление (Отдел рекламы, Департамент маркетинга и рекламы). Дата начала работы в организации: 2012-01-10. Дата рождения: 1969-07-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кврпоаа Ирина Юрьевна'}, {'from': 'gpt', 'value': 'Кврпоаа Ирина Юрьевна работает в организации Волшебный Доктор на должности\n fГлавный бухгалтер, в филиале Управление (Отдел учета расходов, Финансовый департамент). Дата начала работы в организации: 2024-05-02. Дата рождения: 1988-11-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Картавцвеа Мария Витальевна'}, {'from': 'gpt', 'value': 'Картавцвеа Мария Витальевна работает в организации Волшебный Доктор на должности\n fВедущий менеджер интернет-проектов, в филиале Управление (Отдел маркетинга специализированных центров, Департамент Специализированных проектов). Дата начала работы в организации: 2024-11-14. Дата рождения: 2001-01-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Картлеь Людмила Алексеевна'}, {'from': 'gpt', 'value': 'Картлеь Людмила Алексеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2016-02-03. Дата рождения: 1957-07-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кврцоаа Евгения Сергеевна'}, {'from': 'gpt', 'value': 'Кврцоаа Евгения Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-ординатор, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2025-01-20. Дата рождения: 1998-07-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кштеаова Анна Германовна'}, {'from': 'gpt', 'value': 'Кштеаова Анна Германовна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф17-ДЦ (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2021-12-20. Дата рождения: 2001-09-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Катгара Олеся Олеговна'}, {'from': 'gpt', 'value': 'Катгара Олеся Олеговна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф16 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2025-02-03. Дата рождения: 1999-07-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Куатер Ольга Леонидовна'}, {'from': 'gpt', 'value': 'Куатер Ольга Леонидовна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф09 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2014-04-07. Дата рождения: 1973-01-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Каченва Ольга Александровна'}, {'from': 'gpt', 'value': 'Каченва Ольга Александровна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2021-10-21. Дата рождения: 1973-01-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Качруина Наталья Викторовна'}, {'from': 'gpt', 'value': 'Качруина Наталья Викторовна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-11-21. Дата рождения: 1979-10-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Квашнина Алеся Александровна'}, {'from': 'gpt', 'value': 'Квашнина Алеся Александровна работает в организации Волшебный Доктор на должности\n fРуководитель службы поддержки клиентов, в филиале Управление (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2021-12-22. Дата рождения: 1982-06-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Квдроеа Наталья Васильевна'}, {'from': 'gpt', 'value': 'Квдроеа Наталья Васильевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2011-09-14. Дата рождения: 1979-10-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кибьлачич Наталья Владимировна'}, {'from': 'gpt', 'value': 'Кибьлачич Наталья Владимировна работает в организации Волшебный Доктор на должности\n fФельдшер скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2018-08-01. Дата рождения: 1970-02-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Квио Михаил Андреевич'}, {'from': 'gpt', 'value': 'Квио Михаил Андреевич работает в организации Волшебный Доктор на должности\n fСтарший специалист по тестированию ПО, в филиале Управление (Отдел тестирования, Центр разработки ПО). Дата начала работы в организации: 2025-02-10. Дата рождения: 2004-10-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Киколиаившли Мелисса Зурабовна'}, {'from': 'gpt', 'value': 'Киколиаившли Мелисса Зурабовна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-03-11. Дата рождения: 1996-08-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ким Светлана Ильинична'}, {'from': 'gpt', 'value': 'Ким Светлана Ильинична работает в организации Волшебный Доктор на должности\n fМенеджер по экстренной госпитализации, в филиале ГЦ-Ф15 (Отдел мониторинга и экстренной госпитализации, Коммерческий департамент ГЦ). Дата начала работы в организации: 2014-02-11. Дата рождения: 1980-09-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кинеикезв Юрий Баграмович'}, {'from': 'gpt', 'value': 'Кинеикезв Юрий Баграмович работает в организации Волшебный Доктор на должности\n fМедицинский регистратор, в филиале Лаборатория (Экспресс-лаборатория, Экспресс-лаборатория). Дата начала работы в организации: 2022-04-04. Дата рождения: 1989-08-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кипвоа Лариса Султановна'}, {'from': 'gpt', 'value': 'Кипвоа Лариса Султановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2017-07-10. Дата рождения: 1974-07-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коракисян Кристине Гришаевна'}, {'from': 'gpt', 'value': 'Коракисян Кристине Гришаевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2022-03-21. Дата рождения: 1989-12-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Клриилов Максим Александрович'}, {'from': 'gpt', 'value': 'Клриилов Максим Александрович работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2025-03-03. Дата рождения: 1967-07-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Клрилиова Людмила Анатольевна'}, {'from': 'gpt', 'value': 'Клрилиова Людмила Анатольевна работает в организации Волшебный Доктор на должности\n fВедущий бизнес-аналитик, в филиале Управление (Отдел автоматизации и проектирования бизнес-процессов, Отдел автоматизации и проектирования бизнес-процессов). Дата начала работы в организации: 2023-11-20. Дата рождения: 1996-05-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кирилквсоий Алексей Александрович'}, {'from': 'gpt', 'value': 'Кирилквсоий Алексей Александрович работает в организации Волшебный Доктор на должности\n fЗаведующий клинико-диагностическим отделением, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2022-04-25. Дата рождения: 1995-03-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кнричеико Лариса Алексеевна'}, {'from': 'gpt', 'value': 'Кнричеико Лариса Алексеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра кабинета функциональной диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-07-01. Дата рождения: 1991-03-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кнрсаиова Ксения Олеговна'}, {'from': 'gpt', 'value': 'Кнрсаиова Ксения Олеговна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2017-10-18. Дата рождения: 1991-01-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Касиков Владимир Васильевич'}, {'from': 'gpt', 'value': 'Касиков Владимир Васильевич работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2023-12-04. Дата рождения: 1972-04-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Киселева Любовь Сергеевна'}, {'from': 'gpt', 'value': 'Киселева Любовь Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2021-06-02. Дата рождения: 1997-09-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ктиков Александр Владимирович'}, {'from': 'gpt', 'value': 'Ктиков Александр Владимирович работает в организации Волшебный Доктор на должности\n fТехник, в филиале Ф06 (АХО, Вспомогательная служба). Дата начала работы в организации: 2021-06-01. Дата рождения: 1985-06-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Киамшрия Индира Нодариевна'}, {'from': 'gpt', 'value': 'Киамшрия Индира Нодариевна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2024-01-18. Дата рождения: 1981-04-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Клиемнко Екатерина Анатольевна'}, {'from': 'gpt', 'value': 'Клиемнко Екатерина Анатольевна работает в организации Волшебный Доктор на должности\n fГлавный оператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2012-08-21. Дата рождения: 1977-12-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Килмина Эльвира Маратовна'}, {'from': 'gpt', 'value': 'Килмина Эльвира Маратовна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале ГЦ-Ф15 (Центр гастроэнтерологии, Центр гастроэнтерологии). Дата начала работы в организации: 2024-12-02. Дата рождения: 1968-07-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Клиомв Сергей Александрович'}, {'from': 'gpt', 'value': 'Клиомв Сергей Александрович работает в организации Волшебный Доктор на должности\n fСтарший программист, в филиале Управление (Отдел разработки корпоративных систем, Центр разработки ПО). Дата начала работы в организации: 2022-12-05. Дата рождения: 1989-07-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кливома Елена Юрьевна'}, {'from': 'gpt', 'value': 'Кливома Елена Юрьевна работает в организации Волшебный Доктор на должности\n fСтарший бухгалтер по расчету заработной платы, в филиале Управление (Отдел учета расходов, Финансовый департамент). Дата начала работы в организации: 2021-03-16. Дата рождения: 1987-04-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Клкчоов Виктор Владимирович'}, {'from': 'gpt', 'value': 'Клкчоов Виктор Владимирович работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2024-04-08. Дата рождения: 1971-12-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Клкбюо Ольга Васильевна'}, {'from': 'gpt', 'value': 'Клкбюо Ольга Васильевна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале ГЦ-Ф15 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2018-09-20. Дата рождения: 1979-12-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Киюковклна Елена Викторовна'}, {'from': 'gpt', 'value': 'Киюковклна Елена Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2019-09-16. Дата рождения: 1992-10-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кзянева Надежда Александровна'}, {'from': 'gpt', 'value': 'Кзянева Надежда Александровна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф11 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2025-03-03. Дата рождения: 1974-07-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кбоец Татьяна Александровна'}, {'from': 'gpt', 'value': 'Кбоец Татьяна Александровна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2016-10-03. Дата рождения: 1965-05-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кобоикзва Наталия Аркадьевна'}, {'from': 'gpt', 'value': 'Кобоикзва Наталия Аркадьевна работает в организации Волшебный Доктор на должности\n fОператор, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2021-04-14. Дата рождения: 1994-08-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коавлев Петр Андреевич'}, {'from': 'gpt', 'value': 'Коавлев Петр Андреевич работает в организации Волшебный Доктор на должности\n fЮрист-эксперт, в филиале Управление (Юридический отдел, Коммерческий департамент). Дата начала работы в организации: 2024-05-13. Дата рождения: 1989-03-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коеалвва Елизавета Андреевна'}, {'from': 'gpt', 'value': 'Коеалвва Елизавета Андреевна работает в организации Волшебный Доктор на должности\n fСистемный аналитик, в филиале Управление (Отдел разработки корпоративных систем, Центр разработки ПО). Дата начала работы в организации: 2023-09-18. Дата рождения: 1998-11-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Колавева Ирина Николаевна'}, {'from': 'gpt', 'value': 'Колавева Ирина Николаевна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2013-09-16. Дата рождения: 1965-07-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ковалева Наталья Александровна'}, {'from': 'gpt', 'value': 'Ковалева Наталья Александровна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2009-06-02. Дата рождения: 1983-05-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ковалевский Кирилл Игоревич'}, {'from': 'gpt', 'value': 'Ковалевский Кирилл Игоревич работает в организации Волшебный Доктор на должности\n fМенеджер по работе с социальными сетями, в филиале Управление (Отдел рекламы, Департамент маркетинга и рекламы). Дата начала работы в организации: 2021-12-09. Дата рождения: 1992-01-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Клваоенко Любовь Петровна'}, {'from': 'gpt', 'value': 'Клваоенко Любовь Петровна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф03 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1953-03-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кпвоий Юлия Владимировна'}, {'from': 'gpt', 'value': 'Кпвоий Юлия Владимировна работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача по акушерству и гинекологии, в филиале ГЦ-Ф15 (Администрация, Стационар). Дата начала работы в организации: 2022-05-16. Дата рождения: 1975-01-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кгоай Ольга Александровна'}, {'from': 'gpt', 'value': 'Кгоай Ольга Александровна работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Ф06 (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2015-11-03. Дата рождения: 1988-06-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коагн Анна Сергеевна'}, {'from': 'gpt', 'value': 'Коагн Анна Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2024-02-05. Дата рождения: 1995-08-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кодзоев Казбек Нурудинович'}, {'from': 'gpt', 'value': 'Кодзоев Казбек Нурудинович работает в организации Волшебный Доктор на должности\n fВрач-онколог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-12-16. Дата рождения: 1996-09-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Козлнеко Дмитрий Валерьевич'}, {'from': 'gpt', 'value': 'Козлнеко Дмитрий Валерьевич работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2015-11-11. Дата рождения: 1974-12-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коолзв Дмитрий Николаевич'}, {'from': 'gpt', 'value': 'Коолзв Дмитрий Николаевич работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2023-10-16. Дата рождения: 1976-04-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кзолов Максим Вячеславович'}, {'from': 'gpt', 'value': 'Кзолов Максим Вячеславович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2023-02-27. Дата рождения: 1986-08-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Козолв Сергей Александрович'}, {'from': 'gpt', 'value': 'Козолв Сергей Александрович работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2009-02-19. Дата рождения: 1964-02-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Козлова Алина Викторовна'}, {'from': 'gpt', 'value': 'Козлова Алина Викторовна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2025-03-06. Дата рождения: 2004-07-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Крзыоева Александра Юрьевна'}, {'from': 'gpt', 'value': 'Крзыоева Александра Юрьевна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2021-09-09. Дата рождения: 1981-01-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Колнсеик Василий Ярославович'}, {'from': 'gpt', 'value': 'Колнсеик Василий Ярославович работает в организации Волшебный Доктор на должности\n fВрач - пластический хирург, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2024-10-14. Дата рождения: 1999-02-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кслеоников Никита Петрович'}, {'from': 'gpt', 'value': 'Кслеоников Никита Петрович работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-08-26. Дата рождения: 1998-08-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Колескинова Ольга Александровна'}, {'from': 'gpt', 'value': 'Колескинова Ольга Александровна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2018-02-19. Дата рождения: 1985-11-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Клоесникова Ульяна Васильевна'}, {'from': 'gpt', 'value': 'Клоесникова Ульяна Васильевна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2021-05-11. Дата рождения: 1990-12-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Колетоинвв Николай Викторович'}, {'from': 'gpt', 'value': 'Колетоинвв Николай Викторович работает в организации Волшебный Доктор на должности\n fВрач функциональной диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2023-12-25. Дата рождения: 1994-04-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коллкооов Всеволод Викторович'}, {'from': 'gpt', 'value': 'Коллкооов Всеволод Викторович работает в организации Волшебный Доктор на должности\n fСтарший программист, в филиале Управление (Отдел разработки интернет-проектов, Центр разработки ПО). Дата начала работы в организации: 2022-06-01. Дата рождения: 2001-08-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кплоакова Людмила Федоровна'}, {'from': 'gpt', 'value': 'Кплоакова Людмила Федоровна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф11 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-10. Дата рождения: 1952-04-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Колтуонв Андрей Сергеевич'}, {'from': 'gpt', 'value': 'Колтуонв Андрей Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2022-12-05. Дата рождения: 1986-09-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Конпамеец Анна Алексеевна'}, {'from': 'gpt', 'value': 'Конпамеец Анна Алексеевна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф14 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2014-06-24. Дата рождения: 1981-09-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ктндраоьева Анна Петровна'}, {'from': 'gpt', 'value': 'Ктндраоьева Анна Петровна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2023-06-26. Дата рождения: 1989-08-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коьдратнева Вероника Алексеевна'}, {'from': 'gpt', 'value': 'Коьдратнева Вероника Алексеевна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2023-11-02. Дата рождения: 2001-07-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коенва Виктория Владимировна'}, {'from': 'gpt', 'value': 'Коенва Виктория Владимировна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2019-08-26. Дата рождения: 1993-12-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кановолова Мария Леонидовна'}, {'from': 'gpt', 'value': 'Кановолова Мария Леонидовна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2020-10-20. Дата рождения: 1992-10-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коненонко Мария Викторовна'}, {'from': 'gpt', 'value': 'Коненонко Мария Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2020-09-24. Дата рождения: 1981-07-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коноплкциая Эльвира Александровна'}, {'from': 'gpt', 'value': 'Коноплкциая Эльвира Александровна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2019-05-27. Дата рождения: 1985-06-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Когчунов Роман Юрьевич'}, {'from': 'gpt', 'value': 'Когчунов Роман Юрьевич работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2010-03-02. Дата рождения: 1979-08-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кепаова Галина Моисеевна'}, {'from': 'gpt', 'value': 'Кепаова Галина Моисеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2023-11-29. Дата рождения: 1972-02-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Конабельриков Александр Игоревич'}, {'from': 'gpt', 'value': 'Конабельриков Александр Игоревич работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф14 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2023-07-10. Дата рождения: 1994-12-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Колаберьников Иван Михайлович'}, {'from': 'gpt', 'value': 'Колаберьников Иван Михайлович работает в организации Волшебный Доктор на должности\n fМенеджер отдела маркетинга и рекламы, в филиале Управление (Отдел рекламы, Департамент маркетинга и рекламы). Дата начала работы в организации: 2024-09-23. Дата рождения: 1983-05-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кброова Яна Андреевна'}, {'from': 'gpt', 'value': 'Кброова Яна Андреевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф07 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2022-01-10. Дата рождения: 1985-04-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Крокмазова Лейла Барадиновна'}, {'from': 'gpt', 'value': 'Крокмазова Лейла Барадиновна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2024-01-15. Дата рождения: 1992-11-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кореенв Дмитрий Артурович'}, {'from': 'gpt', 'value': 'Кореенв Дмитрий Артурович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-12-16. Дата рождения: 1967-09-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кброока Ксения Витальевна'}, {'from': 'gpt', 'value': 'Кброока Ксения Витальевна работает в организации Волшебный Доктор на должности\n fСпециалист по тестированию ПО, в филиале Управление (Центр разработки ПО, Центр разработки ПО). Дата начала работы в организации: 2017-02-21. Дата рождения: 1993-03-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коорбко Ольга Константиновна'}, {'from': 'gpt', 'value': 'Коорбко Ольга Константиновна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2023-12-27. Дата рождения: 1984-12-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Корбоков Григорий Евгеньевич'}, {'from': 'gpt', 'value': 'Корбоков Григорий Евгеньевич работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача по административно-хозяйственной части, в филиале ГЦ-Ф15 (Администрация, Администрация). Дата начала работы в организации: 2018-12-06. Дата рождения: 1964-02-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коробквоа Анна Михайловна'}, {'from': 'gpt', 'value': 'Коробквоа Анна Михайловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-02-15. Дата рождения: 1971-10-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Короокбва Ирина Григорьевна'}, {'from': 'gpt', 'value': 'Короокбва Ирина Григорьевна работает в организации Волшебный Доктор на должности\n fЗаведующий центром психического здоровья - врач-психиатр, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2005-01-01. Дата рождения: 1988-10-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Корвбкооа Марина Вячеславовна'}, {'from': 'gpt', 'value': 'Корвбкооа Марина Вячеславовна работает в организации Волшебный Доктор на должности\n fЗаместитель генерального директора, в филиале Управление (Аппарат управления, Аппарат управления). Дата начала работы в организации: 2004-03-29. Дата рождения: 1964-03-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Короубшкина Наталья Валерьевна'}, {'from': 'gpt', 'value': 'Короубшкина Наталья Валерьевна работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2023-09-12. Дата рождения: 1974-06-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Королев Вячеслав Владимирович'}, {'from': 'gpt', 'value': 'Королев Вячеслав Владимирович работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2016-03-02. Дата рождения: 1978-04-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коорлева Елена Андреевна'}, {'from': 'gpt', 'value': 'Коорлева Елена Андреевна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф01 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2021-07-19. Дата рождения: 1981-08-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Корелова Ксения Юрьевна'}, {'from': 'gpt', 'value': 'Корелова Ксения Юрьевна работает в организации Волшебный Доктор на должности\n fВрач - пластический хирург, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2024-01-09. Дата рождения: 1990-04-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коровела Татьяна Витальевна'}, {'from': 'gpt', 'value': 'Коровела Татьяна Витальевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф12 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-11-13. Дата рождения: 1994-08-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коретаова Алина Эдуардовна'}, {'from': 'gpt', 'value': 'Коретаова Алина Эдуардовна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2023-11-27. Дата рождения: 1993-09-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коротоноикжна Виктория Валерьевна'}, {'from': 'gpt', 'value': 'Коротоноикжна Виктория Валерьевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2024-10-29. Дата рождения: 1987-09-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коршвкоиа Анджела Викторовна'}, {'from': 'gpt', 'value': 'Коршвкоиа Анджела Викторовна работает в организации Волшебный Доктор на должности\n fЗаведующий педиатрическим отделением, в филиале ГЦ-Ф15 (Педиатрическое отделение, Стационар). Дата начала работы в организации: 2023-03-06. Дата рождения: 1993-05-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кошрунов Кирилл Анатольевич'}, {'from': 'gpt', 'value': 'Кошрунов Кирилл Анатольевич работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф01 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-04-24. Дата рождения: 1994-09-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Косолапвоа Наталья Михайловна'}, {'from': 'gpt', 'value': 'Косолапвоа Наталья Михайловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2018-07-09. Дата рождения: 1988-08-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Костиыкрна Екатерина Олеговна'}, {'from': 'gpt', 'value': 'Костиыкрна Екатерина Олеговна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2019-01-09. Дата рождения: 1996-12-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коотюксва Елена Александровна'}, {'from': 'gpt', 'value': 'Коотюксва Елена Александровна работает в организации Волшебный Доктор на должности\n fНачальник отдела по работе с физическими лицами в специализированных центрах, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2013-11-22. Дата рождения: 1990-02-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кнстюшиоа Ирина Александровна'}, {'from': 'gpt', 'value': 'Кнстюшиоа Ирина Александровна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2023-10-23. Дата рождения: 1985-10-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коснльуикова Ольга Павловна'}, {'from': 'gpt', 'value': 'Коснльуикова Ольга Павловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по физиотерапии, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2019-10-02. Дата рождения: 1968-03-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кнсьмиоова Марина Александровна'}, {'from': 'gpt', 'value': 'Кнсьмиоова Марина Александровна работает в организации Волшебный Доктор на должности\n fМенеджер по сервису, в филиале ГЦ-Ф15 (Служба сервиса, Стационар). Дата начала работы в организации: 2024-12-19. Дата рождения: 1970-08-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кооьянсва Ксения Сергеевна'}, {'from': 'gpt', 'value': 'Кооьянсва Ксения Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2014-02-17. Дата рождения: 1985-11-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Колетьников Николай Андреевич'}, {'from': 'gpt', 'value': 'Колетьников Николай Андреевич работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2019-06-24. Дата рождения: 1993-11-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Котоква Елена Вениаминовна'}, {'from': 'gpt', 'value': 'Котоква Елена Вениаминовна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2016-11-14. Дата рождения: 1980-04-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ковлярота Виктория Геннадьевна'}, {'from': 'gpt', 'value': 'Ковлярота Виктория Геннадьевна работает в организации Волшебный Доктор на должности\n fНачальник отдела по работе с физическими лицами, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2014-07-22. Дата рождения: 1972-01-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ковота Елена Сергеевна'}, {'from': 'gpt', 'value': 'Ковота Елена Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог (хирург), в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-03-11. Дата рождения: 1994-01-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кочванока Алина Равилевна'}, {'from': 'gpt', 'value': 'Кочванока Алина Равилевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра эндоскопии, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-09-05. Дата рождения: 2002-01-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кашевоя Наталия Руслановна'}, {'from': 'gpt', 'value': 'Кашевоя Наталия Руслановна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2022-12-05. Дата рождения: 1993-07-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кошелева Алла Владимировна'}, {'from': 'gpt', 'value': 'Кошелева Алла Владимировна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф17-ДЦ (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-12-04. Дата рождения: 1988-11-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Квшелеоа Ксения Викторовна'}, {'from': 'gpt', 'value': 'Квшелеоа Ксения Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра эндоскопии, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2022-10-20. Дата рождения: 2000-02-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ккшоина Юлия Викторовна'}, {'from': 'gpt', 'value': 'Ккшоина Юлия Викторовна работает в организации Волшебный Доктор на должности\n fВрач клинической лабораторной диагностики, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2007-08-10. Дата рождения: 1980-05-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Котшура Дарья Владимировна'}, {'from': 'gpt', 'value': 'Котшура Дарья Владимировна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-02-01. Дата рождения: 1989-08-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Квасавцера Виктория Игоревна'}, {'from': 'gpt', 'value': 'Квасавцера Виктория Игоревна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2016-11-16. Дата рождения: 1996-05-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Красильоикнва Екатерина Владимировна'}, {'from': 'gpt', 'value': 'Красильоикнва Екатерина Владимировна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2021-12-20. Дата рождения: 1993-03-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Карснова Ксения Владимировна'}, {'from': 'gpt', 'value': 'Карснова Ксения Владимировна работает в организации Волшебный Доктор на должности\n fВрач-эндоскопист, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-07-01. Дата рождения: 1997-02-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Крикоротенво Виктория Вадимовна'}, {'from': 'gpt', 'value': 'Крикоротенво Виктория Вадимовна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф16 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2025-02-27. Дата рождения: 1973-08-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кртоов Александр Игоревич'}, {'from': 'gpt', 'value': 'Кртоов Александр Игоревич работает в организации Волшебный Доктор на должности\n fВедущий системный инженер, в филиале Управление (Отдел разработки корпоративных систем, Центр разработки ПО). Дата начала работы в организации: 2023-09-25. Дата рождения: 1984-04-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Крутиквоа Татьяна Тимофеевна'}, {'from': 'gpt', 'value': 'Крутиквоа Татьяна Тимофеевна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф01 (Администрация, Администрация). Дата начала работы в организации: 2011-03-01. Дата рождения: 1965-02-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Киутрна Елена Исмагзамовна'}, {'from': 'gpt', 'value': 'Киутрна Елена Исмагзамовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по физиотерапии, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2016-09-07. Дата рождения: 1972-12-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Коылрв Сергей Анатольевич'}, {'from': 'gpt', 'value': 'Коылрв Сергей Анатольевич работает в организации Волшебный Доктор на должности\n fВедущий программист, в филиале Управление (Отдел разработки корпоративных систем, Центр разработки ПО). Дата начала работы в организации: 2009-03-03. Дата рождения: 1970-09-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кркюов Дмитрий Александрович'}, {'from': 'gpt', 'value': 'Кркюов Дмитрий Александрович работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2025-01-16. Дата рождения: 1981-07-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Крювока Юлия Николаевна'}, {'from': 'gpt', 'value': 'Крювока Юлия Николаевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф16 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2022-11-21. Дата рождения: 1987-03-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Крюченоква Наталия Владимировна'}, {'from': 'gpt', 'value': 'Крюченоква Наталия Владимировна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2010-11-08. Дата рождения: 1974-09-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кседнзова Юлия Вячеславовна'}, {'from': 'gpt', 'value': 'Кседнзова Юлия Вячеславовна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2023-12-21. Дата рождения: 1975-06-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Квензоса Ольга Николаевна'}, {'from': 'gpt', 'value': 'Квензоса Ольга Николаевна работает в организации Волшебный Доктор на должности\n fЛогопед, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2007-09-05. Дата рождения: 1978-05-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Крдуя Андрей Евгеньевич'}, {'from': 'gpt', 'value': 'Крдуя Андрей Евгеньевич работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф17-ДЦ (Администрация, Администрация). Дата начала работы в организации: 2017-12-18. Дата рождения: 1974-08-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кзуиева Камила Баходуровна'}, {'from': 'gpt', 'value': 'Кзуиева Камила Баходуровна работает в организации Волшебный Доктор на должности\n fРуководитель поликлинической педиатрии и контрактной службы, в филиале Управление (Отдел поликлинической педиатрии и контрактной службы, Медицинский департамент). Дата начала работы в организации: 2013-04-03. Дата рождения: 1983-05-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кзуин Роман Сергеевич'}, {'from': 'gpt', 'value': 'Кзуин Роман Сергеевич работает в организации Волшебный Доктор на должности\n fТехник, в филиале Ф05 (Отдел административно-хозяйственного и технического обеспечения, Административный департамент). Дата начала работы в организации: 2022-09-19. Дата рождения: 1983-01-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кзуина Елена Валерьевна'}, {'from': 'gpt', 'value': 'Кзуина Елена Валерьевна работает в организации Волшебный Доктор на должности\n fМенеджер по экстренной госпитализации, в филиале ГЦ-Ф15 (Отдел мониторинга и экстренной госпитализации, Коммерческий департамент ГЦ). Дата начала работы в организации: 2016-04-19. Дата рождения: 1979-11-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кезнуцов Евгений Александрович'}, {'from': 'gpt', 'value': 'Кезнуцов Евгений Александрович работает в организации Волшебный Доктор на должности\n fСистемный администратор, в филиале Управление (Отдел системного администрирования, Департамент ИТ). Дата начала работы в организации: 2025-01-23. Дата рождения: 1984-07-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кузноцева Анастасия Вадимовна'}, {'from': 'gpt', 'value': 'Кузноцева Анастасия Вадимовна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2024-09-09. Дата рождения: 1998-11-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кузнвцоеа Анна Владимировна'}, {'from': 'gpt', 'value': 'Кузнвцоеа Анна Владимировна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2022-10-24. Дата рождения: 1982-05-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кознецува Виктория Владимировна'}, {'from': 'gpt', 'value': 'Кознецува Виктория Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-04-25. Дата рождения: 2001-05-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Куензцова Евгения Владимировна'}, {'from': 'gpt', 'value': 'Куензцова Евгения Владимировна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2019-03-11. Дата рождения: 1986-04-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Куцнезова Наталья Александровна'}, {'from': 'gpt', 'value': 'Куцнезова Наталья Александровна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф05 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-05. Дата рождения: 1961-03-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кнзуецова Наталья Валерьевна'}, {'from': 'gpt', 'value': 'Кнзуецова Наталья Валерьевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2025-03-06. Дата рождения: 1979-10-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кузенцова Оксана Викторовна'}, {'from': 'gpt', 'value': 'Кузенцова Оксана Викторовна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф03 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2014-04-02. Дата рождения: 1972-12-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кузнецвоа Светлана Владимировна'}, {'from': 'gpt', 'value': 'Кузнецвоа Светлана Владимировна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф03 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1965-03-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кцзнеуова Татьяна Николаевна'}, {'from': 'gpt', 'value': 'Кцзнеуова Татьяна Николаевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф07 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2017-04-17. Дата рождения: 1959-03-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Куиьмзна Анастасия Львовна'}, {'from': 'gpt', 'value': 'Куиьмзна Анастасия Львовна работает в организации Волшебный Доктор на должности\n fНачальник отдела автоматизации и проектирования бизнес-процессов, в филиале Управление (Отдел автоматизации и проектирования бизнес-процессов, Отдел автоматизации и проектирования бизнес-процессов). Дата начала работы в организации: 2017-08-07. Дата рождения: 1973-08-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кумьзина Анна Алексеевна'}, {'from': 'gpt', 'value': 'Кумьзина Анна Алексеевна работает в организации Волшебный Доктор на должности\n fМенеджер по плановой госпитализации, в филиале ГЦ-Ф15 (Отдел госпитализаций и сервиса, Коммерческий департамент ГЦ). Дата начала работы в организации: 2023-07-10. Дата рождения: 1974-07-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кмзьуиных Наталья Александровна'}, {'from': 'gpt', 'value': 'Кмзьуиных Наталья Александровна работает в организации Волшебный Доктор на должности\n fЗаместитель директора медицинского департамента, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2015-12-16. Дата рождения: 1987-08-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кйубида Олег Ростиславович'}, {'from': 'gpt', 'value': 'Кйубида Олег Ростиславович работает в организации Волшебный Доктор на должности\n fВрач-эндоскопист, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2011-12-15. Дата рождения: 1962-07-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Куккшуина Ольга Валериевна'}, {'from': 'gpt', 'value': 'Куккшуина Ольга Валериевна работает в организации Волшебный Доктор на должности\n fБухгалтер по расчетам с покупателями, в филиале Управление (Отдел учета реализации, Финансовый департамент). Дата начала работы в организации: 2022-11-08. Дата рождения: 1979-12-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кшлеуова Галина Степановна'}, {'from': 'gpt', 'value': 'Кшлеуова Галина Степановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2023-07-17. Дата рождения: 1977-09-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Клуикова Наталья Александровна'}, {'from': 'gpt', 'value': 'Клуикова Наталья Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2024-06-27. Дата рождения: 1970-06-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кулвкоиа Светлана Александровна'}, {'from': 'gpt', 'value': 'Кулвкоиа Светлана Александровна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2020-12-21. Дата рождения: 1972-03-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кулшиова Татьяна Викторовна'}, {'from': 'gpt', 'value': 'Кулшиова Татьяна Викторовна работает в организации Волшебный Доктор на должности\n fСекретарь-архивариус, в филиале Управление (Отдел корпоративной документации, Финансовый департамент). Дата начала работы в организации: 2018-04-20. Дата рождения: 1972-01-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Колува Жанна Борисовна'}, {'from': 'gpt', 'value': 'Колува Жанна Борисовна работает в организации Волшебный Доктор на должности\n fРуководитель направления дерматологии, косметологии, пластической хирургии и антивозрастной медицины, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2018-08-06. Дата рождения: 1969-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кумывеша Марина Анзоровна'}, {'from': 'gpt', 'value': 'Кумывеша Марина Анзоровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2019-06-17. Дата рождения: 1996-10-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Куноухдва Алана Александровна'}, {'from': 'gpt', 'value': 'Куноухдва Алана Александровна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф12 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2021-04-19. Дата рождения: 1995-12-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кнуин Евгений Валерьевич'}, {'from': 'gpt', 'value': 'Кнуин Евгений Валерьевич работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Управление (Отдел административно-хозяйственного и технического обеспечения, Административный департамент). Дата начала работы в организации: 2024-07-29. Дата рождения: 1974-11-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Купрвкоиа Наталия Владимировна'}, {'from': 'gpt', 'value': 'Купрвкоиа Наталия Владимировна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Отдел госпитальных продаж, Коммерческий департамент). Дата начала работы в организации: 2011-09-14. Дата рождения: 1979-05-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кнрамшиуа Лилия Фанусовна'}, {'from': 'gpt', 'value': 'Кнрамшиуа Лилия Фанусовна работает в организации Волшебный Доктор на должности\n fРуководитель Call-центра, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2007-10-01. Дата рождения: 1979-10-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Курбанвоа Айнагуль Файзуковна'}, {'from': 'gpt', 'value': 'Курбанвоа Айнагуль Файзуковна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2013-09-23. Дата рождения: 1972-08-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Курнсоова Фаина Алексеевна'}, {'from': 'gpt', 'value': 'Курнсоова Фаина Алексеевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале ГЦ-Ф15 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-03-18. Дата рождения: 1999-06-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Куорчкина Ольга Вячеславовна'}, {'from': 'gpt', 'value': 'Куорчкина Ольга Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2016-11-23. Дата рождения: 1971-05-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кврпякоуа Ольга Валерьевна'}, {'from': 'gpt', 'value': 'Кврпякоуа Ольга Валерьевна работает в организации Волшебный Доктор на должности\n fЗаведующий отделением врачей общей практики, в филиале Ф02 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2006-04-03. Дата рождения: 1981-03-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ксрукая Ирина Ивановна'}, {'from': 'gpt', 'value': 'Ксрукая Ирина Ивановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стерилизационной, в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2018-11-12. Дата рождения: 1964-02-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кирышкуна Алена Игоревна'}, {'from': 'gpt', 'value': 'Кирышкуна Алена Игоревна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф02 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2022-05-30. Дата рождения: 1995-11-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Косува Ольга Владимировна'}, {'from': 'gpt', 'value': 'Косува Ольга Владимировна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2021-11-15. Дата рождения: 1965-03-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кетатуладзе Эрика Николаевна'}, {'from': 'gpt', 'value': 'Кетатуладзе Эрика Николаевна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2023-04-03. Дата рождения: 1969-02-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Куханерко Оксана Сергеевна'}, {'from': 'gpt', 'value': 'Куханерко Оксана Сергеевна работает в организации Волшебный Доктор на должности\n fЗаведующий стоматологическим отделением, в филиале Ф09 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2016-12-12. Дата рождения: 1969-06-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Куцнеко Виктория Витальевна'}, {'from': 'gpt', 'value': 'Куцнеко Виктория Витальевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2021-10-18. Дата рождения: 1976-11-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кечуренко Екатерина Викторовна'}, {'from': 'gpt', 'value': 'Кечуренко Екатерина Викторовна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2022-02-24. Дата рождения: 1997-11-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кочерув Павел Владимирович'}, {'from': 'gpt', 'value': 'Кочерув Павел Владимирович работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф11 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2012-11-28. Дата рождения: 1982-03-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Кушнорив Александр Васильевич'}, {'from': 'gpt', 'value': 'Кушнорив Александр Васильевич работает в организации Волшебный Доктор на должности\n fНачальник отдела кадров, в филиале Управление (Отдел кадров, Департамент по работе с персоналом). Дата начала работы в организации: 2024-12-02. Дата рождения: 1977-05-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лаорунвва Татьяна Александровна'}, {'from': 'gpt', 'value': 'Лаорунвва Татьяна Александровна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2024-11-18. Дата рождения: 1992-04-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лаазрева Майя Аркадьевна'}, {'from': 'gpt', 'value': 'Лаазрева Майя Аркадьевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2023-11-13. Дата рождения: 1968-05-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лайапнова Амина Рамазановна'}, {'from': 'gpt', 'value': 'Лайапнова Амина Рамазановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2024-12-16. Дата рождения: 2003-07-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лайнапова Руфина Рамазановна'}, {'from': 'gpt', 'value': 'Лайнапова Руфина Рамазановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2024-09-09. Дата рождения: 2000-02-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ланпайова Халимат Магометовна'}, {'from': 'gpt', 'value': 'Ланпайова Халимат Магометовна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф06 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2023-06-26. Дата рождения: 1987-10-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лвмзоаа Юлия Викторовна'}, {'from': 'gpt', 'value': 'Лвмзоаа Юлия Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра эндоскопии, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-12-12. Дата рождения: 2002-05-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лднаа Галина Павловна'}, {'from': 'gpt', 'value': 'Лднаа Галина Павловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2016-10-03. Дата рождения: 1965-09-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лаипн Леонид Алексеевич'}, {'from': 'gpt', 'value': 'Лаипн Леонид Алексеевич работает в организации Волшебный Доктор на должности\n fМенеджер стоматологического центра, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2021-01-25. Дата рождения: 2001-04-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лапниа Ольга Николаевна'}, {'from': 'gpt', 'value': 'Лапниа Ольга Николаевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф02 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2024-12-16. Дата рождения: 1985-07-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лраин Александр Александрович'}, {'from': 'gpt', 'value': 'Лраин Александр Александрович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2024-06-10. Дата рождения: 1994-06-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лиран Алексей Владимирович'}, {'from': 'gpt', 'value': 'Лиран Алексей Владимирович работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача, в филиале Ф16 (Администрация, Администрация). Дата начала работы в организации: 2016-11-16. Дата рождения: 1977-02-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лнриаа Алиса Юрьевна'}, {'from': 'gpt', 'value': 'Лнриаа Алиса Юрьевна работает в организации Волшебный Доктор на должности\n fМедицинский оптик-оптометрист, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-12-12. Дата рождения: 2001-11-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лирана Людмила Павловна'}, {'from': 'gpt', 'value': 'Лирана Людмила Павловна работает в организации Волшебный Доктор на должности\n fРуководитель отдела закупок, в филиале Управление (Отдел закупок, Медицинский департамент). Дата начала работы в организации: 2009-03-03. Дата рождения: 1973-09-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ланиорова Юлия Олеговна'}, {'from': 'gpt', 'value': 'Ланиорова Юлия Олеговна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-07-22. Дата рождения: 1995-07-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лрацев Олег Дмитриевич'}, {'from': 'gpt', 'value': 'Лрацев Олег Дмитриевич работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-02-26. Дата рождения: 1993-02-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лебедева Екатерина Владимировна'}, {'from': 'gpt', 'value': 'Лебедева Екатерина Владимировна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2018-09-24. Дата рождения: 1983-06-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Леебдева Полина Сергеевна'}, {'from': 'gpt', 'value': 'Леебдева Полина Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2019-02-25. Дата рождения: 1991-11-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лесандоввкая Наталья Владленовна'}, {'from': 'gpt', 'value': 'Лесандоввкая Наталья Владленовна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2004-04-28. Дата рождения: 1966-09-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ленкива Наталья Витальевна'}, {'from': 'gpt', 'value': 'Ленкива Наталья Витальевна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2011-12-01. Дата рождения: 1963-10-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лвеченко Валерия Валериевна'}, {'from': 'gpt', 'value': 'Лвеченко Валерия Валериевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф06 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2023-11-20. Дата рождения: 1982-04-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Левихода Екатерина Сергеевна'}, {'from': 'gpt', 'value': 'Левихода Екатерина Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2025-01-20. Дата рождения: 1992-06-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лекужова Елена Геннадьевна'}, {'from': 'gpt', 'value': 'Лекужова Елена Геннадьевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2025-02-03. Дата рождения: 1967-10-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лемника Дарья Владимировна'}, {'from': 'gpt', 'value': 'Лемника Дарья Владимировна работает в организации Волшебный Доктор на должности\n fНачальник отдела корпоративной документации, в филиале Управление (Отдел корпоративной документации, Финансовый департамент). Дата начала работы в организации: 2023-08-01. Дата рождения: 1990-11-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лвоноеа Екатерина Александровна'}, {'from': 'gpt', 'value': 'Лвоноеа Екатерина Александровна работает в организации Волшебный Доктор на должности\n fДизайнер, в филиале Управление (Отдел рекламы, Департамент маркетинга и рекламы). Дата начала работы в организации: 2011-09-07. Дата рождения: 1981-10-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Леонова Марина Евгеньевна'}, {'from': 'gpt', 'value': 'Леонова Марина Евгеньевна работает в организации Волшебный Доктор на должности\n fГлавный оператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2017-07-18. Дата рождения: 1971-07-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Леонеьтв Станислав Игоревич'}, {'from': 'gpt', 'value': 'Леонеьтв Станислав Игоревич работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2025-02-10. Дата рождения: 1999-05-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лектина Алла Александровна'}, {'from': 'gpt', 'value': 'Лектина Алла Александровна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф14 (Администрация, Администрация). Дата начала работы в организации: 2018-04-23. Дата рождения: 1969-10-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Летигян Вадим Викторович'}, {'from': 'gpt', 'value': 'Летигян Вадим Викторович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2006-12-25. Дата рождения: 1981-07-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Леятгина Елена Владимировна'}, {'from': 'gpt', 'value': 'Леятгина Елена Владимировна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2007-08-28. Дата рождения: 1981-01-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лнваит Михаил Анатольевич'}, {'from': 'gpt', 'value': 'Лнваит Михаил Анатольевич работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2022-12-05. Дата рождения: 1993-04-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ливжиеда Елена Александровна'}, {'from': 'gpt', 'value': 'Ливжиеда Елена Александровна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2020-09-02. Дата рождения: 1981-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Линнкиова Марина Юрьевна'}, {'from': 'gpt', 'value': 'Линнкиова Марина Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2017-07-03. Дата рождения: 1991-04-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лициская Яна Александровна'}, {'from': 'gpt', 'value': 'Лициская Яна Александровна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф16 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2025-01-23. Дата рождения: 1979-11-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Летвининко Анна Александровна'}, {'from': 'gpt', 'value': 'Летвининко Анна Александровна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог детский, в филиале Ф17-ДЦ (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2020-02-04. Дата рождения: 1997-06-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Литнивова Марина Александровна'}, {'from': 'gpt', 'value': 'Литнивова Марина Александровна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф14 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1962-04-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ливтинчук Юрий Владимирович'}, {'from': 'gpt', 'value': 'Ливтинчук Юрий Владимирович работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2024-02-26. Дата рождения: 1987-03-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лхбаниоин Александр Алексеевич'}, {'from': 'gpt', 'value': 'Лхбаниоин Александр Алексеевич работает в организации Волшебный Доктор на должности\n fЗаместитель директора медицинского департамента, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2018-06-04. Дата рождения: 1987-01-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лвбанооа Наталия Анатольевна'}, {'from': 'gpt', 'value': 'Лвбанооа Наталия Анатольевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по физиотерапии, в филиале ГЦ-Ф15 (Отделение реабилитации, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2015-03-23. Дата рождения: 1958-02-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лообва Галина Михайловна'}, {'from': 'gpt', 'value': 'Лообва Галина Михайловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2024-10-03. Дата рождения: 1998-11-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лоцвова Анастасия Витальевна'}, {'from': 'gpt', 'value': 'Лоцвова Анастасия Витальевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2021-05-17. Дата рождения: 1996-10-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лигонова Анастасия Алексеевна'}, {'from': 'gpt', 'value': 'Лигонова Анастасия Алексеевна работает в организации Волшебный Доктор на должности\n fВедущий специалист отдела нормирования труда и мотивации, в филиале Управление (Отдел нормирования труда и мотивации персонала, Департамент по работе с персоналом). Дата начала работы в организации: 2020-12-22. Дата рождения: 1994-11-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лопатника Юлия Александровна'}, {'from': 'gpt', 'value': 'Лопатника Юлия Александровна работает в организации Волшебный Доктор на должности\n fЗаместитель руководителя службы поддержки клиентов, в филиале Управление (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2009-12-16. Дата рождения: 1979-04-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лопаечва Анна Владимировна'}, {'from': 'gpt', 'value': 'Лопаечва Анна Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2018-07-09. Дата рождения: 1991-07-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лощннииа Екатерина Сергеевна'}, {'from': 'gpt', 'value': 'Лощннииа Екатерина Сергеевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2025-02-17. Дата рождения: 1992-08-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Луганксая Екатерина Юрьевна'}, {'from': 'gpt', 'value': 'Луганксая Екатерина Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2024-10-21. Дата рождения: 1989-04-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лзуина Ирина Валерьевна'}, {'from': 'gpt', 'value': 'Лзуина Ирина Валерьевна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2025-02-03. Дата рождения: 1977-04-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лькуянов Алексей Олегович'}, {'from': 'gpt', 'value': 'Лькуянов Алексей Олегович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Управление (Отдел административно-хозяйственного и технического обеспечения, Административный департамент). Дата начала работы в организации: 2025-01-16. Дата рождения: 1992-08-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лкуьянова Екатерина Алексеевна'}, {'from': 'gpt', 'value': 'Лкуьянова Екатерина Алексеевна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Отдел госпитальных продаж, Коммерческий департамент). Дата начала работы в организации: 2024-05-13. Дата рождения: 1985-01-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лккьянчиуов Денис Владимирович'}, {'from': 'gpt', 'value': 'Лккьянчиуов Денис Владимирович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2007-02-01. Дата рождения: 1980-01-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лупеноква Юлия Александровна'}, {'from': 'gpt', 'value': 'Лупеноква Юлия Александровна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале ГЦ-Ф15 (Отдел административно-хозяйственного и технического обеспечения, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-10-28. Дата рождения: 1964-12-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Луецнко Лилия Николаевна'}, {'from': 'gpt', 'value': 'Луецнко Лилия Николаевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2013-08-21. Дата рождения: 1965-05-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Луцнеко Яна Викторовна'}, {'from': 'gpt', 'value': 'Луцнеко Яна Викторовна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф09 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2011-09-29. Дата рождения: 1978-07-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лучкниа Татьяна Владимировна'}, {'from': 'gpt', 'value': 'Лучкниа Татьяна Владимировна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф14 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2013-09-18. Дата рождения: 1988-09-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лбыа Ирина Викторовна'}, {'from': 'gpt', 'value': 'Лбыа Ирина Викторовна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2021-12-13. Дата рождения: 1964-04-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лыгниа Анна Сергеевна'}, {'from': 'gpt', 'value': 'Лыгниа Анна Сергеевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2023-08-07. Дата рождения: 1976-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лысвоа Юлия Викторовна'}, {'from': 'gpt', 'value': 'Лысвоа Юлия Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2015-10-28. Дата рождения: 1996-02-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лывеша Елизавета Сергеевна'}, {'from': 'gpt', 'value': 'Лывеша Елизавета Сергеевна работает в организации Волшебный Доктор на должности\n fБухгалтер по учету кассовых операций, в филиале Управление (Отдел учета расходов, Финансовый департамент). Дата начала работы в организации: 2024-08-19. Дата рождения: 2002-05-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Любниа Анастасия Сергеевна'}, {'from': 'gpt', 'value': 'Любниа Анастасия Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2022-08-15. Дата рождения: 1998-01-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Люиткова Евгения Викторовна'}, {'from': 'gpt', 'value': 'Люиткова Евгения Викторовна работает в организации Волшебный Доктор на должности\n fЮрист, в филиале Управление (Юридический отдел, Коммерческий департамент). Дата начала работы в организации: 2022-11-01. Дата рождения: 1989-10-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ляодва Ольга Викторовна'}, {'from': 'gpt', 'value': 'Ляодва Ольга Викторовна работает в организации Волшебный Доктор на должности\n fДизайнер, в филиале Управление (Отдел рекламы, Департамент маркетинга и рекламы). Дата начала работы в организации: 2024-08-29. Дата рождения: 1998-10-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лвкишеяа Ирина Юрьевна'}, {'from': 'gpt', 'value': 'Лвкишеяа Ирина Юрьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра кабинета подологии, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2022-03-21. Дата рождения: 1968-09-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лиляна Ольга Дмитриевна'}, {'from': 'gpt', 'value': 'Лиляна Ольга Дмитриевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-05-27. Дата рождения: 1997-03-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Лялхюина Анна Николаевна'}, {'from': 'gpt', 'value': 'Лялхюина Анна Николаевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2019-08-12. Дата рождения: 1976-09-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Магоррамева Нино Омаровна'}, {'from': 'gpt', 'value': 'Магоррамева Нино Омаровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2017-11-15. Дата рождения: 1980-07-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Магомодев Минатулла Камалутдинович'}, {'from': 'gpt', 'value': 'Магомодев Минатулла Камалутдинович работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф09 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2024-10-07. Дата рождения: 1998-02-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Майбдрооа Василий Игоревич'}, {'from': 'gpt', 'value': 'Майбдрооа Василий Игоревич работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф17-ДЦ (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-09-12. Дата рождения: 1985-11-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Маакров Сергей Владимирович'}, {'from': 'gpt', 'value': 'Маакров Сергей Владимирович работает в организации Волшебный Доктор на должности\n fЗаместитель директора административного департамента, в филиале Управление (Административный департамент, Административный департамент). Дата начала работы в организации: 2012-11-12. Дата рождения: 1972-01-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Макарвоа Анастасия Вячеславовна'}, {'from': 'gpt', 'value': 'Макарвоа Анастасия Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач клинической лабораторной диагностики, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2023-10-23. Дата рождения: 1994-03-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Маракова Мария Дмитриевна'}, {'from': 'gpt', 'value': 'Маракова Мария Дмитриевна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2024-10-01. Дата рождения: 1995-05-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Макарова Наталья Анатольевна'}, {'from': 'gpt', 'value': 'Макарова Наталья Анатольевна работает в организации Волшебный Доктор на должности\n fМенеджер по плановой госпитализации сменный, в филиале ГЦ-Ф15 (Отдел госпитализаций и сервиса, Коммерческий департамент ГЦ). Дата начала работы в организации: 2015-09-22. Дата рождения: 1975-04-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Маоарква Ольга Викторовна'}, {'from': 'gpt', 'value': 'Маоарква Ольга Викторовна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф06 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-03. Дата рождения: 1972-10-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Макршааипова Карина Багомедовна'}, {'from': 'gpt', 'value': 'Макршааипова Карина Багомедовна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале ГЦ-Ф15 (Администрация, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2019-08-22. Дата рождения: 1998-04-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Макикнео Ольга Николаевна'}, {'from': 'gpt', 'value': 'Макикнео Ольга Николаевна работает в организации Волшебный Доктор на должности\n fВрач-генетик, в филиале ГЦ-Ф15 (Отделение ВОП/педиатрии, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2022-09-19. Дата рождения: 1964-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Маклевоца Виолетта Сергеевна'}, {'from': 'gpt', 'value': 'Маклевоца Виолетта Сергеевна работает в организации Волшебный Доктор на должности\n fМенеджер центра косметологии и пластической хирургии, в филиале Управление (Центр косметологии и пластической хирургии, Коммерческий департамент). Дата начала работы в организации: 2024-05-06. Дата рождения: 2002-07-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мшкуакин Андрей Васильевич'}, {'from': 'gpt', 'value': 'Мшкуакин Андрей Васильевич работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2021-11-15. Дата рождения: 1968-11-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Малхаова Алина Игоревна'}, {'from': 'gpt', 'value': 'Малхаова Алина Игоревна работает в организации Волшебный Доктор на должности\n fЗаместитель начальника отдела по работе с физическими лицами, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2008-02-04. Дата рождения: 1983-11-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Малхаова Наталья Сергеевна'}, {'from': 'gpt', 'value': 'Малхаова Наталья Сергеевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Ф01 (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2021-01-14. Дата рождения: 1976-01-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Малокива Жанна Авгатовна'}, {'from': 'gpt', 'value': 'Малокива Жанна Авгатовна работает в организации Волшебный Доктор на должности\n fВрач-нефролог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-01-13. Дата рождения: 1989-03-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Малкиова Светлана Николаевна'}, {'from': 'gpt', 'value': 'Малкиова Светлана Николаевна работает в организации Волшебный Доктор на должности\n fДизайнер-верстальщик, в филиале Управление (Отдел рекламы, Департамент маркетинга и рекламы). Дата начала работы в организации: 2019-02-27. Дата рождения: 1991-05-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мавлаела Хадижат Зайнудиновна'}, {'from': 'gpt', 'value': 'Мавлаела Хадижат Зайнудиновна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-12-19. Дата рождения: 1996-02-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Млаов Евгений Викторович'}, {'from': 'gpt', 'value': 'Млаов Евгений Викторович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф01 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-02-08. Дата рождения: 1971-06-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Маолва Инна Викторовна'}, {'from': 'gpt', 'value': 'Маолва Инна Викторовна работает в организации Волшебный Доктор на должности\n fВедущий специалист по охране труда, в филиале Управление (Отдел охраны труда, Департамент по работе с персоналом). Дата начала работы в организации: 2013-04-04. Дата рождения: 1966-06-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мылах Ирина Леонидовна'}, {'from': 'gpt', 'value': 'Мылах Ирина Леонидовна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2014-11-12. Дата рождения: 1959-03-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мальсагвоа Фариза Ибрагимовна'}, {'from': 'gpt', 'value': 'Мальсагвоа Фариза Ибрагимовна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2024-11-12. Дата рождения: 1998-02-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мальецв Сергей Сергеевич'}, {'from': 'gpt', 'value': 'Мальецв Сергей Сергеевич работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2023-11-02. Дата рождения: 1998-01-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мацьлев Станислав Васильевич'}, {'from': 'gpt', 'value': 'Мацьлев Станислав Васильевич работает в организации Волшебный Доктор на должности\n fИнженер по эксплуатации медицинского оборудования сменный, в филиале Управление (Отдел эксплуатации медицинского оборудования, Медицинский департамент). Дата начала работы в организации: 2022-11-24. Дата рождения: 1978-01-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мвляаина Анастасия Александровна'}, {'from': 'gpt', 'value': 'Мвляаина Анастасия Александровна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2017-09-25. Дата рождения: 1990-08-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Маялр Надежда Евгеньевна'}, {'from': 'gpt', 'value': 'Маялр Надежда Евгеньевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2025-02-20. Дата рождения: 2000-04-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мамаев Константин Борисович'}, {'from': 'gpt', 'value': 'Мамаев Константин Борисович работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2019-09-30. Дата рождения: 1979-11-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ммааев Темирлан Уллубиевич'}, {'from': 'gpt', 'value': 'Ммааев Темирлан Уллубиевич работает в организации Волшебный Доктор на должности\n fСпециалист по поддержке пользователей (сменный), в филиале Управление (Отдел технической поддержки, Департамент ИТ). Дата начала работы в организации: 2023-08-21. Дата рождения: 2001-04-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мамглиаа Вера Игнатовна'}, {'from': 'gpt', 'value': 'Мамглиаа Вера Игнатовна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2016-04-11. Дата рождения: 1962-01-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мянукан Лиана Микаеловна'}, {'from': 'gpt', 'value': 'Мянукан Лиана Микаеловна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-12-16. Дата рождения: 1986-01-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Манцруова Анна Владимировна'}, {'from': 'gpt', 'value': 'Манцруова Анна Владимировна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2002-03-01. Дата рождения: 1969-01-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Маринина Татьяна Леонидовна'}, {'from': 'gpt', 'value': 'Маринина Татьяна Леонидовна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2009-03-05. Дата рождения: 1962-07-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Млркеаова Вера Александровна'}, {'from': 'gpt', 'value': 'Млркеаова Вера Александровна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2023-05-22. Дата рождения: 1987-03-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мавкора Елена Александровна'}, {'from': 'gpt', 'value': 'Мавкора Елена Александровна работает в организации Волшебный Доктор на должности\n fЭстетист, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2024-09-12. Дата рождения: 1974-10-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мавкора Ирина Владимировна'}, {'from': 'gpt', 'value': 'Мавкора Ирина Владимировна работает в организации Волшебный Доктор на должности\n fВедущий специалист Call-центра, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2008-07-24. Дата рождения: 1976-11-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Моркава Ольга Владимировна'}, {'from': 'gpt', 'value': 'Моркава Ольга Владимировна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф05 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2015-08-10. Дата рождения: 1969-11-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Маокрвская Марина Васильевна'}, {'from': 'gpt', 'value': 'Маокрвская Марина Васильевна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2022-01-13. Дата рождения: 1966-11-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Марткненыо Александра Александровна'}, {'from': 'gpt', 'value': 'Марткненыо Александра Александровна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф07 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2024-05-27. Дата рождения: 1976-08-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Масвееа Галина Львовна'}, {'from': 'gpt', 'value': 'Масвееа Галина Львовна работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2022-06-27. Дата рождения: 1968-10-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Маолсв Александр Александрович'}, {'from': 'gpt', 'value': 'Маолсв Александр Александрович работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф12 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1953-02-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мсалова Наталия Борисовна'}, {'from': 'gpt', 'value': 'Мсалова Наталия Борисовна работает в организации Волшебный Доктор на должности\n fСекретарь-делопроизводитель, в филиале Управление (Отдел корпоративной документации, Финансовый департамент). Дата начала работы в организации: 2024-08-26. Дата рождения: 1992-03-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Матянин Елена Викторовна'}, {'from': 'gpt', 'value': 'Матянин Елена Викторовна работает в организации Волшебный Доктор на должности\n fСтарший менеджер по плановой госпитализации, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-02-22. Дата рождения: 1971-01-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Матюнниа Юлия Владимировна'}, {'from': 'gpt', 'value': 'Матюнниа Юлия Владимировна работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф09 (Администрация, Администрация). Дата начала работы в организации: 2024-03-26. Дата рождения: 1980-10-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Митюхан Александр Сергеевич'}, {'from': 'gpt', 'value': 'Митюхан Александр Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-пульмонолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-09-23. Дата рождения: 1983-12-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Махмотува Екатерина Шамилевна'}, {'from': 'gpt', 'value': 'Махмотува Екатерина Шамилевна работает в организации Волшебный Доктор на должности\n fГлавный специалист центра клиентского сервиса, в филиале Управление (Центр клиентского сервиса, Коммерческий департамент). Дата начала работы в организации: 2025-02-24. Дата рождения: 1999-12-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Маицяс Дарья Владимировна'}, {'from': 'gpt', 'value': 'Маицяс Дарья Владимировна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-12-09. Дата рождения: 1995-01-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мачуеаква Анжела Мовдановна'}, {'from': 'gpt', 'value': 'Мачуеаква Анжела Мовдановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2014-09-24. Дата рождения: 1971-08-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мкшаова Яна Александровна'}, {'from': 'gpt', 'value': 'Мкшаова Яна Александровна работает в организации Волшебный Доктор на должности\n fГлавная медицинская сестра, в филиале ГЦ-Ф15 (Администрация, Администрация). Дата начала работы в организации: 2019-01-14. Дата рождения: 1979-12-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мдеведева Инна Николаевна'}, {'from': 'gpt', 'value': 'Мдеведева Инна Николаевна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-12-17. Дата рождения: 1995-11-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Медвеведа Марина Илларионовна'}, {'from': 'gpt', 'value': 'Медвеведа Марина Илларионовна работает в организации Волшебный Доктор на должности\n fМенеджер по экстренной госпитализации, в филиале ГЦ-Ф15 (Отдел мониторинга и экстренной госпитализации, Коммерческий департамент ГЦ). Дата начала работы в организации: 2014-03-20. Дата рождения: 1971-09-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мижнена Ирина Васильевна'}, {'from': 'gpt', 'value': 'Мижнена Ирина Васильевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф09 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2021-07-01. Дата рождения: 1980-12-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мезнеева Ольга Владимировна'}, {'from': 'gpt', 'value': 'Мезнеева Ольга Владимировна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф01 (Отделение педиатрии, Отделение педиатрии). Дата начала работы в организации: 2004-03-10. Дата рождения: 1976-07-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мнзеецев Максим Юрьевич'}, {'from': 'gpt', 'value': 'Мнзеецев Максим Юрьевич работает в организации Волшебный Доктор на должности\n fВедущий специалист интернет-проектов, в филиале Управление (Отдел интернет-проектов, Департамент маркетинга и рекламы). Дата начала работы в организации: 2025-02-17. Дата рождения: 1994-01-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Млеикян Сабина Седраковна'}, {'from': 'gpt', 'value': 'Млеикян Сабина Седраковна работает в организации Волшебный Доктор на должности\n fСпециалист отдела подбора и адаптации, в филиале Управление (Отдел подбора и адаптации, Департамент по работе с персоналом). Дата начала работы в организации: 2024-10-10. Дата рождения: 2000-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мхлиеова Валентина Николаевна'}, {'from': 'gpt', 'value': 'Мхлиеова Валентина Николаевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2023-05-15. Дата рождения: 1979-06-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мельнивока Ольга Викторовна'}, {'from': 'gpt', 'value': 'Мельнивока Ольга Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2014-12-24. Дата рождения: 1972-03-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мееьничлнко Наталья Викторовна'}, {'from': 'gpt', 'value': 'Мееьничлнко Наталья Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2024-09-02. Дата рождения: 1977-01-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мельинчук Татьяна Александровна'}, {'from': 'gpt', 'value': 'Мельинчук Татьяна Александровна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2012-11-06. Дата рождения: 1974-12-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Меекенва Анна Петровна'}, {'from': 'gpt', 'value': 'Меекенва Анна Петровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Педиатрическое отделение, Стационар). Дата начала работы в организации: 2019-09-30. Дата рождения: 1968-07-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Меронива Екатерина Андреевна'}, {'from': 'gpt', 'value': 'Меронива Екатерина Андреевна работает в организации Волшебный Доктор на должности\n fВрач клинической лабораторной диагностики, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2019-11-11. Дата рождения: 1988-03-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мекшова Ксения Сергеевна'}, {'from': 'gpt', 'value': 'Мекшова Ксения Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2024-08-29. Дата рождения: 2000-09-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мевкоша Оксана Алексеевна'}, {'from': 'gpt', 'value': 'Мевкоша Оксана Алексеевна работает в организации Волшебный Доктор на должности\n fЗаведующий центром женского здоровья и косметологии, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2018-08-29. Дата рождения: 1988-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мещеноква Анна Анатольевна'}, {'from': 'gpt', 'value': 'Мещеноква Анна Анатольевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф01 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-01-22. Дата рождения: 1997-05-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Миезв Алексей Александрович'}, {'from': 'gpt', 'value': 'Миезв Алексей Александрович работает в организации Волшебный Доктор на должности\n fНачальник отдела системного администрирования, в филиале Управление (Отдел системного администрирования, Департамент ИТ). Дата начала работы в организации: 2012-12-06. Дата рождения: 1974-05-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Микинетко Ольга Николаевна'}, {'from': 'gpt', 'value': 'Микинетко Ольга Николаевна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф09 (Администрация, Администрация). Дата начала работы в организации: 2024-11-05. Дата рождения: 1983-09-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Млиевский Александр Сергеевич'}, {'from': 'gpt', 'value': 'Млиевский Александр Сергеевич работает в организации Волшебный Доктор на должности\n fТехник, в филиале Ф16 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-02-08. Дата рождения: 1976-10-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мивишникола Татьяна Романовна'}, {'from': 'gpt', 'value': 'Мивишникола Татьяна Романовна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф16 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2019-09-02. Дата рождения: 1987-11-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мильчокава Наталья Ивановна'}, {'from': 'gpt', 'value': 'Мильчокава Наталья Ивановна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф12 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2002-09-05. Дата рождения: 1953-12-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Миникнов Александр Сергеевич'}, {'from': 'gpt', 'value': 'Миникнов Александр Сергеевич работает в организации Волшебный Доктор на должности\n fСпециалист по контролю качества, в филиале Лаборатория (Администрация, Администрация). Дата начала работы в организации: 2017-12-20. Дата рождения: 1982-10-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мироненко Марина Николаевна'}, {'from': 'gpt', 'value': 'Мироненко Марина Николаевна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф16 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-24. Дата рождения: 1957-08-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Миронова Валентина Степановна'}, {'from': 'gpt', 'value': 'Миронова Валентина Степановна работает в организации Волшебный Доктор на должности\n fВрач клинической лабораторной диагностики, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2012-11-14. Дата рождения: 1954-02-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мирноова Елена Николаевна'}, {'from': 'gpt', 'value': 'Мирноова Елена Николаевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2013-09-11. Дата рождения: 1984-07-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Миронова Людмила Николаевна'}, {'from': 'gpt', 'value': 'Миронова Людмила Николаевна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2020-02-13. Дата рождения: 1974-07-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Митроаольскпя Инга Альбертовна'}, {'from': 'gpt', 'value': 'Митроаольскпя Инга Альбертовна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2019-03-13. Дата рождения: 1966-10-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Митшоренков Пётр Павлович'}, {'from': 'gpt', 'value': 'Митшоренков Пётр Павлович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-хирург, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2022-02-01. Дата рождения: 1991-01-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мртиошина Людмила Владимировна'}, {'from': 'gpt', 'value': 'Мртиошина Людмила Владимировна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2025-01-16. Дата рождения: 1974-04-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Михйалова Зоя Игоревна'}, {'from': 'gpt', 'value': 'Михйалова Зоя Игоревна работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2021-09-16. Дата рождения: 1990-11-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мхиайловская Людмила Георгиевна'}, {'from': 'gpt', 'value': 'Мхиайловская Людмила Георгиевна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2004-09-22. Дата рождения: 1959-02-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Михнлиаа Светлана Владимировна'}, {'from': 'gpt', 'value': 'Михнлиаа Светлана Владимировна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф03 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2019-02-11. Дата рождения: 1983-11-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мехеива Наталья Владимировна'}, {'from': 'gpt', 'value': 'Мехеива Наталья Владимировна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2017-11-20. Дата рождения: 1974-08-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Миишн Сергей Сергеевич'}, {'from': 'gpt', 'value': 'Миишн Сергей Сергеевич работает в организации Волшебный Доктор на должности\n fМенеджер проектов, в филиале Управление (Департамент управления проектами, Департамент управления проектами). Дата начала работы в организации: 2016-10-27. Дата рождения: 1990-02-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мнщеико Жанна Викторовна'}, {'from': 'gpt', 'value': 'Мнщеико Жанна Викторовна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф16 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2021-04-26. Дата рождения: 1996-09-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мищкнео Наталья Сергеевна'}, {'from': 'gpt', 'value': 'Мищкнео Наталья Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2007-09-01. Дата рождения: 1971-02-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мощенкив Константин Витальевич'}, {'from': 'gpt', 'value': 'Мощенкив Константин Витальевич работает в организации Волшебный Доктор на должности\n fСпециалист отдела внешнего обучения, оценки и развития, в филиале Управление (Учебный центр/Отдел внешнего обучения, оценки и развития, Департамент по работе с персоналом). Дата начала работы в организации: 2023-04-06. Дата рождения: 2001-04-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мкщениова Елена Андреевна'}, {'from': 'gpt', 'value': 'Мкщениова Елена Андреевна работает в организации Волшебный Доктор на должности\n fРуководитель учебного центра, в филиале Управление (Учебный центр, Департамент по работе с персоналом). Дата начала работы в организации: 2020-03-30. Дата рождения: 1977-04-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Млинниквоа Наталья Викторовна'}, {'from': 'gpt', 'value': 'Млинниквоа Наталья Викторовна работает в организации Волшебный Доктор на должности\n fВрач клинической лабораторной диагностики, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2020-12-22. Дата рождения: 1972-10-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мчвоан Юлия Олеговна'}, {'from': 'gpt', 'value': 'Мчвоан Юлия Олеговна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Экспресс-лаборатория, Экспресс-лаборатория). Дата начала работы в организации: 2024-09-26. Дата рождения: 1999-05-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Моажева Татьяна Андреевна'}, {'from': 'gpt', 'value': 'Моажева Татьяна Андреевна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2022-05-16. Дата рождения: 1986-11-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Моцодлова Светлана Николаевна'}, {'from': 'gpt', 'value': 'Моцодлова Светлана Николаевна работает в организации Волшебный Доктор на должности\n fМенеджер по сопровождению, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2025-02-17. Дата рождения: 1973-09-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Моиочнлков Сергей Викторович'}, {'from': 'gpt', 'value': 'Моиочнлков Сергей Викторович работает в организации Волшебный Доктор на должности\n fТехник, в филиале Ф07 (АХО, Вспомогательная служба). Дата начала работы в организации: 2017-07-27. Дата рождения: 1970-04-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Млочанов Алексей Валерьевич'}, {'from': 'gpt', 'value': 'Млочанов Алексей Валерьевич работает в организации Волшебный Доктор на должности\n fЗаместитель руководителя центра разработки ПО, в филиале Управление (Центр разработки ПО, Центр разработки ПО). Дата начала работы в организации: 2011-06-20. Дата рождения: 1982-06-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мноах Алексей Владимирович'}, {'from': 'gpt', 'value': 'Мноах Алексей Владимирович работает в организации Волшебный Доктор на должности\n fСтарший техник сменный, в филиале Управление (Отдел административно-хозяйственного и технического обеспечения, Административный департамент). Дата начала работы в организации: 2018-02-06. Дата рождения: 1975-03-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мотне Николас Андреавич'}, {'from': 'gpt', 'value': 'Мотне Николас Андреавич работает в организации Волшебный Доктор на должности\n fНачальник отдела маркетинга специализированных центров, в филиале Управление (Отдел маркетинга специализированных центров, Департамент Специализированных проектов). Дата начала работы в организации: 2024-10-07. Дата рождения: 1990-04-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Морновида Анна Павловна'}, {'from': 'gpt', 'value': 'Морновида Анна Павловна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф07 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2020-03-23. Дата рождения: 1969-09-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Моорв Юрий Александрович'}, {'from': 'gpt', 'value': 'Моорв Юрий Александрович работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2013-12-16. Дата рождения: 1963-02-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мовозора Анна Станиславовна'}, {'from': 'gpt', 'value': 'Мовозора Анна Станиславовна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф14 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2024-11-11. Дата рождения: 1980-08-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мзрооова Вера Олеговна'}, {'from': 'gpt', 'value': 'Мзрооова Вера Олеговна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2021-12-06. Дата рождения: 1983-05-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Моровоза Надежда Александровна'}, {'from': 'gpt', 'value': 'Моровоза Надежда Александровна работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2016-06-13. Дата рождения: 1963-11-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мнскалеоко Сергей Анатольевич'}, {'from': 'gpt', 'value': 'Мнскалеоко Сергей Анатольевич работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2018-02-14. Дата рождения: 1986-01-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мксолоова Мария Геннадьевна'}, {'from': 'gpt', 'value': 'Мксолоова Мария Геннадьевна работает в организации Волшебный Доктор на должности\n fВедущий специалист по воинскому учету, в филиале Управление (Отдел кадров, Департамент по работе с персоналом). Дата начала работы в организации: 2019-02-19. Дата рождения: 1990-10-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мосолвоа Евгения Александровна'}, {'from': 'gpt', 'value': 'Мосолвоа Евгения Александровна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2025-02-10. Дата рождения: 1977-10-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Моволоса Маргарита Юрьевна'}, {'from': 'gpt', 'value': 'Моволоса Маргарита Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2020-02-10. Дата рождения: 1995-06-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Моситев Заур Эдуардович'}, {'from': 'gpt', 'value': 'Моситев Заур Эдуардович работает в организации Волшебный Доктор на должности\n fГигиенист стоматологический, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-03-04. Дата рождения: 1990-10-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мусор Анна Владленовна'}, {'from': 'gpt', 'value': 'Мусор Анна Владленовна работает в организации Волшебный Доктор на должности\n fЗаведующий отделением врачей общей практики, в филиале Ф06 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2023-08-28. Дата рождения: 1995-02-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Муборекшоав Муборакшо Шорухович'}, {'from': 'gpt', 'value': 'Муборекшоав Муборакшо Шорухович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2007-02-01. Дата рождения: 1962-10-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мобуракшоева Майгун Шоруховна'}, {'from': 'gpt', 'value': 'Мобуракшоева Майгун Шоруховна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2009-04-23. Дата рождения: 1965-08-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мудунгосанав Шамиль Магомедович'}, {'from': 'gpt', 'value': 'Мудунгосанав Шамиль Магомедович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-07-08. Дата рождения: 1990-05-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Муравицкий Юрий Леонидович'}, {'from': 'gpt', 'value': 'Муравицкий Юрий Леонидович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2020-09-14. Дата рождения: 1978-12-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Меравлува Юлия Геннадьевна'}, {'from': 'gpt', 'value': 'Меравлува Юлия Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач функциональной диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2022-11-07. Дата рождения: 1970-01-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мшрауова Наталья Николаевна'}, {'from': 'gpt', 'value': 'Мшрауова Наталья Николаевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2007-03-19. Дата рождения: 1967-02-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Муртзаин Руслан Няилевич'}, {'from': 'gpt', 'value': 'Муртзаин Руслан Няилевич работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале Ф12 (Отделение анестезиологии и реанимации, Отделение анестезиологии и реанимации). Дата начала работы в организации: 2021-07-12. Дата рождения: 1994-10-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Муснльяац Рубен Владимирович'}, {'from': 'gpt', 'value': 'Муснльяац Рубен Владимирович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф10 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2013-03-15. Дата рождения: 1967-09-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мсуина Регина Ринатовна'}, {'from': 'gpt', 'value': 'Мсуина Регина Ринатовна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2018-11-19. Дата рождения: 1994-12-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мсутафина Светлана Игоревна'}, {'from': 'gpt', 'value': 'Мсутафина Светлана Игоревна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф03 (Администрация, Администрация). Дата начала работы в организации: 2018-10-10. Дата рождения: 1983-07-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мутхарова Марьям Мансуровна'}, {'from': 'gpt', 'value': 'Мутхарова Марьям Мансуровна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2021-09-20. Дата рождения: 1979-07-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Муакчева Александра Михайловна'}, {'from': 'gpt', 'value': 'Муакчева Александра Михайловна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф16 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2014-03-07. Дата рождения: 1964-11-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мизын Артем Алексеевич'}, {'from': 'gpt', 'value': 'Мизын Артем Алексеевич работает в организации Волшебный Доктор на должности\n fДизайнер интерфейсов, в филиале Управление (Отдел маркетинга специализированных центров, Департамент Специализированных проектов). Дата начала работы в организации: 2024-11-14. Дата рождения: 1989-06-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Мягоква Оксана Николаевна'}, {'from': 'gpt', 'value': 'Мягоква Оксана Николаевна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2020-08-24. Дата рождения: 1971-02-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Наврузошева Фируза Довутшоевна'}, {'from': 'gpt', 'value': 'Наврузошева Фируза Довутшоевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2021-05-17. Дата рождения: 1983-04-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Надгораня Светлана Викторовна'}, {'from': 'gpt', 'value': 'Надгораня Светлана Викторовна работает в организации Волшебный Доктор на должности\n fДиректор центра клиентского сервиса, в филиале Управление (Центр клиентского сервиса, Коммерческий департамент). Дата начала работы в организации: 2017-04-03. Дата рождения: 1977-01-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Накеждина Екатерина Викторовна'}, {'from': 'gpt', 'value': 'Накеждина Екатерина Викторовна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2014-10-04. Дата рождения: 1983-01-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Навароза Екатерина Дмитриевна'}, {'from': 'gpt', 'value': 'Навароза Екатерина Дмитриевна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Отдел госпитальных продаж, Коммерческий департамент). Дата начала работы в организации: 2024-11-14. Дата рождения: 2002-03-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Низамов Константин Владимирович'}, {'from': 'gpt', 'value': 'Низамов Константин Владимирович работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-10-21. Дата рождения: 1960-08-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Надйенова Ирина Николаевна'}, {'from': 'gpt', 'value': 'Надйенова Ирина Николаевна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2018-11-14. Дата рождения: 1966-12-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нлкуаян Лидия Оганесовна'}, {'from': 'gpt', 'value': 'Нлкуаян Лидия Оганесовна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2017-08-21. Дата рождения: 1979-01-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нарунжых Николай Викторович'}, {'from': 'gpt', 'value': 'Нарунжых Николай Викторович работает в организации Волшебный Доктор на должности\n fВрач-эндоскопист, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-06-24. Дата рождения: 1988-04-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Наесдкина Галина Григорьевна'}, {'from': 'gpt', 'value': 'Наесдкина Галина Григорьевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2011-10-25. Дата рождения: 1974-07-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нсаырова Наталья Васильевна'}, {'from': 'gpt', 'value': 'Нсаырова Наталья Васильевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2023-10-12. Дата рождения: 1978-03-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нмуаенко Юлия Владимировна'}, {'from': 'gpt', 'value': 'Нмуаенко Юлия Владимировна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале ГЦ-Ф15 (Отделение ВОП/педиатрии, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-09-23. Дата рождения: 2000-05-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ныбесней Дмитрий Эдуардович'}, {'from': 'gpt', 'value': 'Ныбесней Дмитрий Эдуардович работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2023-12-29. Дата рождения: 1994-05-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Невесылей Владимир Владимирович'}, {'from': 'gpt', 'value': 'Невесылей Владимир Владимирович работает в организации Волшебный Доктор на должности\n fПродуктовый дизайнер, в филиале Управление (Отдел разработки интернет-проектов, Центр разработки ПО). Дата начала работы в организации: 2024-07-01. Дата рождения: 2000-04-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Неодшивкина Елена Вячеславовна'}, {'from': 'gpt', 'value': 'Неодшивкина Елена Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2023-11-07. Дата рождения: 1994-02-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нмеилостива Елена Алексеевна'}, {'from': 'gpt', 'value': 'Нмеилостива Елена Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-эксперт-инфекционист, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-05-16. Дата рождения: 1967-10-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нерозанк Ирина Сергеевна'}, {'from': 'gpt', 'value': 'Нерозанк Ирина Сергеевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2016-11-01. Дата рождения: 1985-07-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нектеренсо Кристина Григорьевна'}, {'from': 'gpt', 'value': 'Нектеренсо Кристина Григорьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2012-09-03. Дата рождения: 1987-12-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нейумина Татьяна Валерьевна'}, {'from': 'gpt', 'value': 'Нейумина Татьяна Валерьевна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2021-09-20. Дата рождения: 1973-03-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нефвдоеа Оксана Николаевна'}, {'from': 'gpt', 'value': 'Нефвдоеа Оксана Николаевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2023-08-07. Дата рождения: 1979-07-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нелтулфаев Максим Зейнуллаевич'}, {'from': 'gpt', 'value': 'Нелтулфаев Максим Зейнуллаевич работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф07 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-05-06. Дата рождения: 1998-05-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Начеев Михаил Иванович'}, {'from': 'gpt', 'value': 'Начеев Михаил Иванович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф06 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2008-10-20. Дата рождения: 1957-02-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нигмаиуллтна Карина Ильяровна'}, {'from': 'gpt', 'value': 'Нигмаиуллтна Карина Ильяровна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2024-07-08. Дата рождения: 1979-07-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Низаомв Руслан Ильдусович'}, {'from': 'gpt', 'value': 'Низаомв Руслан Ильдусович работает в организации Волшебный Доктор на должности\n fВодитель скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2024-02-01. Дата рождения: 1993-06-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Никинита Анна Александровна'}, {'from': 'gpt', 'value': 'Никинита Анна Александровна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф01 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-03-11. Дата рождения: 1991-07-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Никинита Ника Александровна'}, {'from': 'gpt', 'value': 'Никинита Ника Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-09-26. Дата рождения: 1999-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Никитинская Елена Владимировна'}, {'from': 'gpt', 'value': 'Никитинская Елена Владимировна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф02 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2015-07-16. Дата рождения: 1977-08-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Никиворофа Юлия Олеговна'}, {'from': 'gpt', 'value': 'Никиворофа Юлия Олеговна работает в организации Волшебный Доктор на должности\n fСпециалист отдела дистанционного обучения, в филиале Управление (Учебный центр/Отдел дистанционного обучения, Департамент по работе с персоналом). Дата начала работы в организации: 2025-02-03. Дата рождения: 1992-03-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Никвлаеоа Анна Викторовна'}, {'from': 'gpt', 'value': 'Никвлаеоа Анна Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стерилизационной, в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2023-02-13. Дата рождения: 1991-02-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Николвеаа Елена Геннадьевна'}, {'from': 'gpt', 'value': 'Николвеаа Елена Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2025-02-10. Дата рождения: 1987-10-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Никоваела Любовь Петровна'}, {'from': 'gpt', 'value': 'Никоваела Любовь Петровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2019-12-03. Дата рождения: 1988-04-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Николеава Оксана Анатольевна'}, {'from': 'gpt', 'value': 'Николеава Оксана Анатольевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2011-12-15. Дата рождения: 1976-05-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Никалоенко Юлия Юрьевна'}, {'from': 'gpt', 'value': 'Никалоенко Юлия Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2025-01-22. Дата рождения: 1986-06-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Никоолва Маргарита Сергеевна'}, {'from': 'gpt', 'value': 'Никоолва Маргарита Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2023-08-14. Дата рождения: 1974-02-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нилокьский Роман Борисович'}, {'from': 'gpt', 'value': 'Нилокьский Роман Борисович работает в организации Волшебный Доктор на должности\n fТехник, в филиале Ф02 (АХО, Вспомогательная служба). Дата начала работы в организации: 2011-04-25. Дата рождения: 1977-01-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Никоонва Ксения Вячеславовна'}, {'from': 'gpt', 'value': 'Никоонва Ксения Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф05 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2024-07-15. Дата рождения: 1995-04-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нкиулина Ирина Сергеевна'}, {'from': 'gpt', 'value': 'Нкиулина Ирина Сергеевна работает в организации Волшебный Доктор на должности\n fГлавный оператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2019-07-25. Дата рождения: 1990-06-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Намиев Доржинима Бадмаевич'}, {'from': 'gpt', 'value': 'Намиев Доржинима Бадмаевич работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2012-02-07. Дата рождения: 1982-09-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нтсиор Дарина Виталиевна'}, {'from': 'gpt', 'value': 'Нтсиор Дарина Виталиевна работает в организации Волшебный Доктор на должности\n fСтарший менеджер центра косметологии и пластической хирургии, в филиале Управление (Центр косметологии и пластической хирургии, Коммерческий департамент). Дата начала работы в организации: 2025-01-23. Дата рождения: 1992-04-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Нвоиков Илья Вадимович'}, {'from': 'gpt', 'value': 'Нвоиков Илья Вадимович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2024-11-05. Дата рождения: 2000-08-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Новикова Елена Викторовна'}, {'from': 'gpt', 'value': 'Новикова Елена Викторовна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф02 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2014-05-27. Дата рождения: 1970-06-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Новвкоиа Елена Николаевна'}, {'from': 'gpt', 'value': 'Новвкоиа Елена Николаевна работает в организации Волшебный Доктор на должности\n fОператор, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2024-04-15. Дата рождения: 1977-07-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Новикова Ольга Игоревна'}, {'from': 'gpt', 'value': 'Новикова Ольга Игоревна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2017-04-27. Дата рождения: 1969-05-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Новёсолов Алексей Викторович'}, {'from': 'gpt', 'value': 'Новёсолов Алексей Викторович работает в организации Волшебный Доктор на должности\n fСтарший менеджер стоматологического центра, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2015-02-18. Дата рождения: 1988-05-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ноеосвлова Валерия Юрьевна'}, {'from': 'gpt', 'value': 'Ноеосвлова Валерия Юрьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-07-18. Дата рождения: 2003-09-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Носоква Анастасия Витальевна'}, {'from': 'gpt', 'value': 'Носоква Анастасия Витальевна работает в организации Волшебный Доктор на должности\n fГлавный специалист центра клиентского сервиса, в филиале Управление (Центр клиентского сервиса, Коммерческий департамент). Дата начала работы в организации: 2020-09-24. Дата рождения: 1994-07-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Норкыв Данила Олегович'}, {'from': 'gpt', 'value': 'Норкыв Данила Олегович работает в организации Волшебный Доктор на должности\n fСпециалист по поддержке пользователей (сменный), в филиале Управление (Отдел технической поддержки, Департамент ИТ). Дата начала работы в организации: 2024-09-09. Дата рождения: 2001-04-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Обушенвока Наталья Константиновна'}, {'from': 'gpt', 'value': 'Обушенвока Наталья Константиновна работает в организации Волшебный Доктор на должности\n fБухгалтер по расчету заработной платы, в филиале Управление (Отдел учета расходов, Финансовый департамент). Дата начала работы в организации: 2020-08-25. Дата рождения: 1980-09-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Оёыдбннова Анна Викторовна'}, {'from': 'gpt', 'value': 'Оёыдбннова Анна Викторовна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф09 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2023-10-02. Дата рождения: 1997-12-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Овечикн Степан Анатольевич'}, {'from': 'gpt', 'value': 'Овечикн Степан Анатольевич работает в организации Волшебный Доктор на должности\n fАналитик бизнес и трудовых процессов, в филиале Управление (Отдел бизнес и трудовых процессов, Департамент организационного развития и контроля). Дата начала работы в организации: 2021-01-25. Дата рождения: 1994-11-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Овечикна Татьяна Владимировна'}, {'from': 'gpt', 'value': 'Овечикна Татьяна Владимировна работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф01 (Администрация, Администрация). Дата начала работы в организации: 2016-07-11. Дата рождения: 1973-03-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Овсянниквоа Анастасия Сергеевна'}, {'from': 'gpt', 'value': 'Овсянниквоа Анастасия Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра эндоскопии, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2021-08-23. Дата рождения: 1996-07-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Овчонникив Вадим Юрьевич'}, {'from': 'gpt', 'value': 'Овчонникив Вадим Юрьевич работает в организации Волшебный Доктор на должности\n fВрач-колопроктолог, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-12-09. Дата рождения: 1980-09-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Оагнезов Дмитрий Георгиевич'}, {'from': 'gpt', 'value': 'Оагнезов Дмитрий Георгиевич работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф09 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2025-02-10. Дата рождения: 1997-09-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Огансеян Татевик Тиграновна'}, {'from': 'gpt', 'value': 'Огансеян Татевик Тиграновна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале ГЦ-Ф15 (Отделение гинекологии №1, Стационар). Дата начала работы в организации: 2024-11-25. Дата рождения: 1990-04-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Оганинсян Эдгар Артурович'}, {'from': 'gpt', 'value': 'Оганинсян Эдгар Артурович работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале ГЦ-Ф15 (Отделение гинекологии №1, Стационар). Дата начала работы в организации: 2023-09-25. Дата рождения: 1992-07-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ооаркгв Максим Александрович'}, {'from': 'gpt', 'value': 'Ооаркгв Максим Александрович работает в организации Волшебный Доктор на должности\n fФельдшер скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2023-07-24. Дата рождения: 1992-06-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Огвена Марина Алексеевна'}, {'from': 'gpt', 'value': 'Огвена Марина Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2019-12-02. Дата рождения: 1968-02-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Огёнва Ольга Владимировна'}, {'from': 'gpt', 'value': 'Огёнва Ольга Владимировна работает в организации Волшебный Доктор на должности\n fВрач-эндоскопист, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2024-09-02. Дата рождения: 1984-05-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Озеива Лиза Магомет-Таировна'}, {'from': 'gpt', 'value': 'Озеива Лиза Магомет-Таировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2022-01-20. Дата рождения: 1991-02-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Оозл Юлия Александровна'}, {'from': 'gpt', 'value': 'Оозл Юлия Александровна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2022-02-24. Дата рождения: 1968-12-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Олкйниеов Андрей Валентинович'}, {'from': 'gpt', 'value': 'Олкйниеов Андрей Валентинович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф12 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2013-10-21. Дата рождения: 1983-03-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Оморава Наталья Витальевна'}, {'from': 'gpt', 'value': 'Оморава Наталья Витальевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2019-04-10. Дата рождения: 1974-04-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Омельяноивч Ольга Олеговна'}, {'from': 'gpt', 'value': 'Омельяноивч Ольга Олеговна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2024-03-28. Дата рождения: 1979-09-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Оунчин Михаил Витальевич'}, {'from': 'gpt', 'value': 'Оунчин Михаил Витальевич работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача, в филиале Ф05 (Администрация, Администрация). Дата начала работы в организации: 2023-04-10. Дата рождения: 1993-11-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Опраин Никита Сергеевич'}, {'from': 'gpt', 'value': 'Опраин Никита Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф10 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2019-10-14. Дата рождения: 1992-12-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Оревоха Оксана Владимировна'}, {'from': 'gpt', 'value': 'Оревоха Оксана Владимировна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф14 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2021-10-04. Дата рождения: 2000-10-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Окешнирова Мария Владимировна'}, {'from': 'gpt', 'value': 'Окешнирова Мария Владимировна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2008-01-30. Дата рождения: 1974-04-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Олров Александр Сергеевич'}, {'from': 'gpt', 'value': 'Олров Александр Сергеевич работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2014-04-14. Дата рождения: 1982-11-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Орвола Алина Рауфовна'}, {'from': 'gpt', 'value': 'Орвола Алина Рауфовна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2013-09-13. Дата рождения: 1986-11-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Оролва Светлана Николаевна'}, {'from': 'gpt', 'value': 'Оролва Светлана Николаевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф17-ДЦ (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2009-07-08. Дата рождения: 1974-01-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Осадчая Лариса Владимировна'}, {'from': 'gpt', 'value': 'Осадчая Лариса Владимировна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2025-02-06. Дата рождения: 1969-08-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Оуадчск Елена Витальевна'}, {'from': 'gpt', 'value': 'Оуадчск Елена Витальевна работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2008-06-10. Дата рождения: 1976-03-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Отсанин Андрей Владимирович'}, {'from': 'gpt', 'value': 'Отсанин Андрей Владимирович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф10 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2021-11-08. Дата рождения: 1995-11-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Осншыый Николай Иванович'}, {'from': 'gpt', 'value': 'Осншыый Николай Иванович работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-09-09. Дата рождения: 1991-05-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Оаффн Юлия Сергеевна'}, {'from': 'gpt', 'value': 'Оаффн Юлия Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-колопроктолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-11-08. Дата рождения: 1987-04-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пквалааи Виктория Викторовна'}, {'from': 'gpt', 'value': 'Пквалааи Виктория Викторовна работает в организации Волшебный Доктор на должности\n fВрач-ординатор, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2024-09-02. Дата рождения: 1999-10-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пваелюк Олеся Васильевна'}, {'from': 'gpt', 'value': 'Пваелюк Олеся Васильевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2007-08-27. Дата рождения: 1986-03-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Паволв Алексей Валериянович'}, {'from': 'gpt', 'value': 'Паволв Алексей Валериянович работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф02 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-04-22. Дата рождения: 1995-08-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Паволв Сергей Михайлович'}, {'from': 'gpt', 'value': 'Паволв Сергей Михайлович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2011-12-01. Дата рождения: 1962-06-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Павнючелко Надежда Сергеевна'}, {'from': 'gpt', 'value': 'Павнючелко Надежда Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2015-10-05. Дата рождения: 1989-06-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Паоердва Альфия Наильевна'}, {'from': 'gpt', 'value': 'Паоердва Альфия Наильевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2008-12-10. Дата рождения: 1967-11-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Паливдоа Филипп Александрович'}, {'from': 'gpt', 'value': 'Паливдоа Филипп Александрович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2019-02-18. Дата рождения: 1988-07-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Панавёча Валентина Петровна'}, {'from': 'gpt', 'value': 'Панавёча Валентина Петровна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2015-10-07. Дата рождения: 1963-06-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Павона Ксения Владимировна'}, {'from': 'gpt', 'value': 'Павона Ксения Владимировна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-05-14. Дата рождения: 1988-08-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Плнфиаова Светлана Викторовна'}, {'from': 'gpt', 'value': 'Плнфиаова Светлана Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-10-07. Дата рождения: 1968-01-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пкпиаян Лика Валерьевна'}, {'from': 'gpt', 'value': 'Пкпиаян Лика Валерьевна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог детский, в филиале Ф17-ДЦ (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2019-07-19. Дата рождения: 1990-02-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Параомнова Екатерина Геннадьевна'}, {'from': 'gpt', 'value': 'Параомнова Екатерина Геннадьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2020-11-26. Дата рождения: 1999-10-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Парфнеов Андрей Владимирович'}, {'from': 'gpt', 'value': 'Парфнеов Андрей Владимирович работает в организации Волшебный Доктор на должности\n fПервый заместитель генерального директора, в филиале Управление (Аппарат управления, Аппарат управления). Дата начала работы в организации: 2007-09-01. Дата рождения: 1973-02-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Паофенрва Ольга Викторовна'}, {'from': 'gpt', 'value': 'Паофенрва Ольга Викторовна работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2018-09-24. Дата рождения: 1987-09-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Писхана Ульяна Александровна'}, {'from': 'gpt', 'value': 'Писхана Ульяна Александровна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2016-10-24. Дата рождения: 1988-09-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Палтатый Кирилл Эдуардович'}, {'from': 'gpt', 'value': 'Палтатый Кирилл Эдуардович работает в организации Волшебный Доктор на должности\n fИнженер по эксплуатации медицинского оборудования сменный, в филиале Управление (Отдел эксплуатации медицинского оборудования, Медицинский департамент). Дата начала работы в организации: 2024-10-14. Дата рождения: 1998-01-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пахмоова Регина Александровна'}, {'from': 'gpt', 'value': 'Пахмоова Регина Александровна работает в организации Волшебный Доктор на должности\n fВедущий врач-пластический хирург, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2021-12-20. Дата рождения: 1980-08-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Памохова Софья Станиславовна'}, {'from': 'gpt', 'value': 'Памохова Софья Станиславовна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение микробиологии, Отделение микробиологии). Дата начала работы в организации: 2024-11-05. Дата рождения: 2001-12-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пелвхоиа Мария Андреевна'}, {'from': 'gpt', 'value': 'Пелвхоиа Мария Андреевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2018-02-21. Дата рождения: 1993-01-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Переведеецнва Марина Геннадьевна'}, {'from': 'gpt', 'value': 'Переведеецнва Марина Геннадьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2025-02-17. Дата рождения: 2002-04-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пееевррзев Игорь Александрович'}, {'from': 'gpt', 'value': 'Пееевррзев Игорь Александрович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2018-10-17. Дата рождения: 1968-01-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Перапичей Екатерина Николаевна'}, {'from': 'gpt', 'value': 'Перапичей Екатерина Николаевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф12 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2022-11-21. Дата рождения: 1980-12-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пегесов Станислав Вадимович'}, {'from': 'gpt', 'value': 'Пегесов Станислав Вадимович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2025-01-27. Дата рождения: 1980-02-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пртеова Евгения Юрьевна'}, {'from': 'gpt', 'value': 'Пртеова Евгения Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2016-11-14. Дата рождения: 1987-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Петвора Елена Андреевна'}, {'from': 'gpt', 'value': 'Петвора Елена Андреевна работает в организации Волшебный Доктор на должности\n fКонтент-менеджер, в филиале Управление (Отдел рекламы, Департамент маркетинга и рекламы). Дата начала работы в организации: 2025-03-06. Дата рождения: 1989-05-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Потрева Марина Николаевна'}, {'from': 'gpt', 'value': 'Потрева Марина Николаевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф17-ДЦ (Отделение педиатрии, Отделение педиатрии). Дата начала работы в организации: 2009-02-24. Дата рождения: 1978-11-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Потрева Ольга Николаевна'}, {'from': 'gpt', 'value': 'Потрева Ольга Николаевна работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале Ф02 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2014-12-01. Дата рождения: 1965-06-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Петросняц Лариса Еремяевна'}, {'from': 'gpt', 'value': 'Петросняц Лариса Еремяевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале ГЦ-Ф15 (Отделение ВОП/педиатрии, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2018-04-16. Дата рождения: 1967-11-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пеуртк Марина Клавдиевна'}, {'from': 'gpt', 'value': 'Пеуртк Марина Клавдиевна работает в организации Волшебный Доктор на должности\n fКоординатор сервисных работ, в филиале ГЦ-Ф15 (Служба сервиса, Стационар). Дата начала работы в организации: 2020-09-22. Дата рождения: 1965-10-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пешктеова Ольга Анатольевна'}, {'from': 'gpt', 'value': 'Пешктеова Ольга Анатольевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф01 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2023-10-02. Дата рождения: 1985-09-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пвикина Галина Алексеевна'}, {'from': 'gpt', 'value': 'Пвикина Галина Алексеевна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф05 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1962-02-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пивовавора Екатерина Александровна'}, {'from': 'gpt', 'value': 'Пивовавора Екатерина Александровна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф07 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-12-25. Дата рождения: 1983-04-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Палит Людмила Николаевна'}, {'from': 'gpt', 'value': 'Палит Людмила Николаевна работает в организации Волшебный Доктор на должности\n fВрач-физиотерапевт, в филиале ГЦ-Ф15 (Отделение реабилитации, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2009-10-20. Дата рождения: 1969-09-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пигорова Ирина Юрьевна'}, {'from': 'gpt', 'value': 'Пигорова Ирина Юрьевна работает в организации Волшебный Доктор на должности\n fРуководитель центра гастроэнтерологии, врач-гастроэнтеролог, в филиале ГЦ-Ф15 (Центр гастроэнтерологии, Центр гастроэнтерологии). Дата начала работы в организации: 2023-08-07. Дата рождения: 1964-05-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пкавслая Нина Викторовна'}, {'from': 'gpt', 'value': 'Пкавслая Нина Викторовна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2024-07-09. Дата рождения: 1975-08-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Плотонав Александр Викторович'}, {'from': 'gpt', 'value': 'Плотонав Александр Викторович работает в организации Волшебный Доктор на должности\n fРуководитель направления функциональной диагностики, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2019-03-27. Дата рождения: 1984-01-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Палтонов Антон Викторович'}, {'from': 'gpt', 'value': 'Палтонов Антон Викторович работает в организации Волшебный Доктор на должности\n fВрач-травматолог-ортопед, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2015-02-09. Дата рождения: 1984-07-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Платонвоа Ксения Анатольевна'}, {'from': 'gpt', 'value': 'Платонвоа Ксения Анатольевна работает в организации Волшебный Доктор на должности\n fРуководитель секретариата Дирекции, в филиале Управление (Секретариат, Аппарат дирекции). Дата начала работы в организации: 2009-01-19. Дата рождения: 1975-03-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пеилва Фатима Магамет-Башировна'}, {'from': 'gpt', 'value': 'Пеилва Фатима Магамет-Башировна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2023-10-02. Дата рождения: 1997-12-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Плоинткова Анастасия Сергеевна'}, {'from': 'gpt', 'value': 'Плоинткова Анастасия Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2024-01-22. Дата рождения: 1995-11-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Плвтникооа Галина Ивановна'}, {'from': 'gpt', 'value': 'Плвтникооа Галина Ивановна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2022-04-11. Дата рождения: 1971-03-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Прваонина Диана Ильинична'}, {'from': 'gpt', 'value': 'Прваонина Диана Ильинична работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2024-09-11. Дата рождения: 1962-04-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пооосгва Светлана Игоревна'}, {'from': 'gpt', 'value': 'Пооосгва Светлана Игоревна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2020-12-21. Дата рождения: 1979-09-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пгоосян Полина Сергеевна'}, {'from': 'gpt', 'value': 'Пгоосян Полина Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2012-12-10. Дата рождения: 1982-04-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Поивдцкая Ольга Александровна'}, {'from': 'gpt', 'value': 'Поивдцкая Ольга Александровна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф09 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2022-06-20. Дата рождения: 1973-07-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Поргоднов Александр Алексеевич'}, {'from': 'gpt', 'value': 'Поргоднов Александр Алексеевич работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2022-11-21. Дата рождения: 1998-04-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пздоорова Людмила Геннадьевна'}, {'from': 'gpt', 'value': 'Пздоорова Людмила Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2017-03-20. Дата рождения: 1966-11-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пядолокин Николай Васильевич'}, {'from': 'gpt', 'value': 'Пядолокин Николай Васильевич работает в организации Волшебный Доктор на должности\n fВодитель скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2020-05-27. Дата рождения: 1975-11-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Подымова Анна Александровна'}, {'from': 'gpt', 'value': 'Подымова Анна Александровна работает в организации Волшебный Доктор на должности\n fГлавный врач, в филиале Ф12 (Администрация, Администрация). Дата начала работы в организации: 2015-11-17. Дата рождения: 1985-01-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Поздоякнва Ольга Игоревна'}, {'from': 'gpt', 'value': 'Поздоякнва Ольга Игоревна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф03 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2007-08-01. Дата рождения: 1978-07-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Покодива Анастасия Сергеевна'}, {'from': 'gpt', 'value': 'Покодива Анастасия Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф10 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2024-06-17. Дата рождения: 1978-12-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Полявока Александра Денисовна'}, {'from': 'gpt', 'value': 'Полявока Александра Денисовна работает в организации Волшебный Доктор на должности\n fВрач-ортодонт, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-07-22. Дата рождения: 1999-11-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Поиарнна Анастасия Викторовна'}, {'from': 'gpt', 'value': 'Поиарнна Анастасия Викторовна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2021-10-25. Дата рождения: 1996-01-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пноомарев Владимир Борисович'}, {'from': 'gpt', 'value': 'Пноомарев Владимир Борисович работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2021-08-26. Дата рождения: 2001-01-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пеномарова Алевтина Владимировна'}, {'from': 'gpt', 'value': 'Пеномарова Алевтина Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра эндоскопии, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2023-09-01. Дата рождения: 1975-01-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Поаомнрева Екатерина Вячеславовна'}, {'from': 'gpt', 'value': 'Поаомнрева Екатерина Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф07 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2017-03-13. Дата рождения: 1990-05-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ппоов Илья Валентинович'}, {'from': 'gpt', 'value': 'Ппоов Илья Валентинович работает в организации Волшебный Доктор на должности\n fРуководитель проектов по технической поддержке информационно-коммуникационных систем, в филиале Управление (Отдел технической поддержки, Департамент ИТ). Дата начала работы в организации: 2010-09-20. Дата рождения: 1982-06-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Попвоа Анжелика Викторовна'}, {'from': 'gpt', 'value': 'Попвоа Анжелика Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2015-08-03. Дата рождения: 1969-04-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Повопа Ирина Алексеевна'}, {'from': 'gpt', 'value': 'Повопа Ирина Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-координатор сменный, в филиале ГЦ-Ф15 (Медицинский пульт, Коммерческий департамент ГЦ). Дата начала работы в организации: 2024-10-28. Дата рождения: 1995-01-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Попвоа Наталья Михайловна'}, {'from': 'gpt', 'value': 'Попвоа Наталья Михайловна работает в организации Волшебный Доктор на должности\n fЗаведующий отделением, в филиале ГЦ-Ф15 (Отделение гинекологии №1, Стационар). Дата начала работы в организации: 2020-12-07. Дата рождения: 1976-01-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Попвоа Светлана Александровна'}, {'from': 'gpt', 'value': 'Попвоа Светлана Александровна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2020-11-16. Дата рождения: 1990-10-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Повопа Светлана Юрьевна'}, {'from': 'gpt', 'value': 'Повопа Светлана Юрьевна работает в организации Волшебный Доктор на должности\n fРуководитель педиатрии и выездной помощи, в филиале Управление (Отдел выездной помощи и педиатрии, Медицинский департамент). Дата начала работы в организации: 2009-07-14. Дата рождения: 1975-03-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Поопва Юнна Рафиковна'}, {'from': 'gpt', 'value': 'Поопва Юнна Рафиковна работает в организации Волшебный Доктор на должности\n fВрач-бактериолог, в филиале Лаборатория (Отделение микробиологии, Отделение микробиологии). Дата начала работы в организации: 2021-09-27. Дата рождения: 1985-08-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Порошкнео Мария Валерьевна'}, {'from': 'gpt', 'value': 'Порошкнео Мария Валерьевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф09 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2011-08-22. Дата рождения: 1980-09-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Протарескул Наталья Васильевна'}, {'from': 'gpt', 'value': 'Протарескул Наталья Васильевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Ф12 (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2019-11-19. Дата рождения: 1980-03-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Поснтых Елена Владимировна'}, {'from': 'gpt', 'value': 'Поснтых Елена Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф05 (Центр косметологии, Центр косметологии). Дата начала работы в организации: 2018-10-22. Дата рождения: 1968-01-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Посысоева Маргарита Алексеевна'}, {'from': 'gpt', 'value': 'Посысоева Маргарита Алексеевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2014-12-24. Дата рождения: 1959-05-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Посесоыва Ольга Юрьевна'}, {'from': 'gpt', 'value': 'Посесоыва Ольга Юрьевна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф12 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2025-02-24. Дата рождения: 1983-10-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Понайтова Дарья Александровна'}, {'from': 'gpt', 'value': 'Понайтова Дарья Александровна работает в организации Волшебный Доктор на должности\n fСпециалист отдела нормирования труда и мотивации, в филиале Управление (Отдел нормирования труда и мотивации персонала, Департамент по работе с персоналом). Дата начала работы в организации: 2024-09-26. Дата рождения: 2002-08-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Потапенквоа Евгения Юрьевна'}, {'from': 'gpt', 'value': 'Потапенквоа Евгения Юрьевна работает в организации Волшебный Доктор на должности\n fЗаместитель начальника отдела подбора и адаптации персонала, в филиале Управление (Отдел подбора и адаптации, Департамент по работе с персоналом). Дата начала работы в организации: 2014-08-28. Дата рождения: 1987-06-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пречав Даниил Сергеевич'}, {'from': 'gpt', 'value': 'Пречав Даниил Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-ординатор, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2024-12-09. Дата рождения: 1999-06-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Прцлуикая Наталья Борисовна'}, {'from': 'gpt', 'value': 'Прцлуикая Наталья Борисовна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф10 (Администрация, Администрация). Дата начала работы в организации: 2022-05-12. Дата рождения: 1970-07-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пирщепа Анна Олеговна'}, {'from': 'gpt', 'value': 'Пирщепа Анна Олеговна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2024-06-24. Дата рождения: 1988-04-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Проскруина Анна Ильинична'}, {'from': 'gpt', 'value': 'Проскруина Анна Ильинична работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-09-20. Дата рождения: 1973-12-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Провтокоса Ольга Юрьевна'}, {'from': 'gpt', 'value': 'Провтокоса Ольга Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-10-12. Дата рождения: 1962-06-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пророхов Андрей Валерьевич'}, {'from': 'gpt', 'value': 'Пророхов Андрей Валерьевич работает в организации Волшебный Доктор на должности\n fЗаместитель руководителя отдела разработки интернет-проектов, в филиале Управление (Отдел разработки интернет-проектов, Центр разработки ПО). Дата начала работы в организации: 2020-02-10. Дата рождения: 1996-11-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Прудниоква Татьяна Васильевна'}, {'from': 'gpt', 'value': 'Прудниоква Татьяна Васильевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2018-08-08. Дата рождения: 1984-03-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Прдяко Георгий Борисович'}, {'from': 'gpt', 'value': 'Прдяко Георгий Борисович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2020-12-01. Дата рождения: 1979-05-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Прчнияникова Ольга Андреевна'}, {'from': 'gpt', 'value': 'Прчнияникова Ольга Андреевна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2022-06-06. Дата рождения: 1994-03-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пуагчев Виталий Олегович'}, {'from': 'gpt', 'value': 'Пуагчев Виталий Олегович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2015-12-07. Дата рождения: 1993-03-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пузвноаа Александра Викторовна'}, {'from': 'gpt', 'value': 'Пузвноаа Александра Викторовна работает в организации Волшебный Доктор на должности\n fОператор, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2019-10-24. Дата рождения: 1992-02-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пнзеуко Екатерина Анатольевна'}, {'from': 'gpt', 'value': 'Пнзеуко Екатерина Анатольевна работает в организации Волшебный Доктор на должности\n fДиректор по маркетингу и рекламе, в филиале Управление (Департамент маркетинга и рекламы, Департамент маркетинга и рекламы). Дата начала работы в организации: 2017-11-13. Дата рождения: 1976-11-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пулниа Анастасия Викторовна'}, {'from': 'gpt', 'value': 'Пулниа Анастасия Викторовна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф01 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2025-02-10. Дата рождения: 1997-10-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пукьно Дарья Сергеевна'}, {'from': 'gpt', 'value': 'Пукьно Дарья Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф02 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-01-09. Дата рождения: 1995-06-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Пунилита Ксения Павловна'}, {'from': 'gpt', 'value': 'Пунилита Ксения Павловна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф07 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-04-22. Дата рождения: 2001-01-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рвгимоаа Зара Эюбовна'}, {'from': 'gpt', 'value': 'Рвгимоаа Зара Эюбовна работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача, в филиале Ф01 (Администрация, Администрация). Дата начала работы в организации: 2016-06-01. Дата рождения: 1980-09-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ридыгана Елена Анатольевна'}, {'from': 'gpt', 'value': 'Ридыгана Елена Анатольевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2025-03-06. Дата рождения: 1971-11-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Равбулаежа Анжела Мусаевна'}, {'from': 'gpt', 'value': 'Равбулаежа Анжела Мусаевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2023-11-07. Дата рождения: 1987-10-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Раазкова Лилия Анвяровна'}, {'from': 'gpt', 'value': 'Раазкова Лилия Анвяровна работает в организации Волшебный Доктор на должности\n fВрач-ортодонт, в филиале Ф07 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-10-28. Дата рождения: 1997-01-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Развмоуский Алексей Сергеевич'}, {'from': 'gpt', 'value': 'Развмоуский Алексей Сергеевич работает в организации Волшебный Доктор на должности\n fВодитель легкового автомобиля, в филиале Управление (Автопарк дирекции, Аппарат дирекции). Дата начала работы в организации: 2015-06-09. Дата рождения: 1983-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рвмазаноаа Патимат Зайпулаевна'}, {'from': 'gpt', 'value': 'Рвмазаноаа Патимат Зайпулаевна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2021-01-19. Дата рождения: 1994-08-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рсакин Сергей Юрьевич'}, {'from': 'gpt', 'value': 'Рсакин Сергей Юрьевич работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф06 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2020-10-19. Дата рождения: 1982-02-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рзее Андрей Геннадьевич'}, {'from': 'gpt', 'value': 'Рзее Андрей Геннадьевич работает в организации Волшебный Доктор на должности\n fЗаместитель генерального директора по науке и образованию, в филиале Управление (Аппарат управления, Аппарат управления). Дата начала работы в организации: 2024-05-17. Дата рождения: 1973-04-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рейизн Андрей Борисович'}, {'from': 'gpt', 'value': 'Рейизн Андрей Борисович работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2011-04-14. Дата рождения: 1961-02-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Реитнский Ярослав Борисович'}, {'from': 'gpt', 'value': 'Реитнский Ярослав Борисович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-10-07. Дата рождения: 1999-04-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ризниа Диляра Равилевна'}, {'from': 'gpt', 'value': 'Ризниа Диляра Равилевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-12-19. Дата рождения: 1989-09-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ридона Эржена Владимировна'}, {'from': 'gpt', 'value': 'Ридона Эржена Владимировна работает в организации Волшебный Доктор на должности\n fВедущий специалист Call-центра, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2011-06-15. Дата рождения: 1972-08-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Родиоонва Анастасия Николаевна'}, {'from': 'gpt', 'value': 'Родиоонва Анастасия Николаевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2021-01-25. Дата рождения: 1984-04-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ржоаева Ольга Константиновна'}, {'from': 'gpt', 'value': 'Ржоаева Ольга Константиновна работает в организации Волшебный Доктор на должности\n fБиолог, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2014-09-22. Дата рождения: 1963-07-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ржоко Анастасия Дмитриевна'}, {'from': 'gpt', 'value': 'Ржоко Анастасия Дмитриевна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2023-02-06. Дата рождения: 1994-05-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рвжкооа Елена Владимировна'}, {'from': 'gpt', 'value': 'Рвжкооа Елена Владимировна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-12-12. Дата рождения: 1983-04-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ржоок Алисия Игоревна'}, {'from': 'gpt', 'value': 'Ржоок Алисия Игоревна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-05-06. Дата рождения: 1997-09-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Розенберг Алексей Александрович'}, {'from': 'gpt', 'value': 'Розенберг Алексей Александрович работает в организации Волшебный Доктор на должности\n fВрач скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2017-04-17. Дата рождения: 1970-10-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Розноова Юнна Игоревна'}, {'from': 'gpt', 'value': 'Розноова Юнна Игоревна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф11 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2020-03-10. Дата рождения: 1986-06-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рзоумный Илья Аркадьевич'}, {'from': 'gpt', 'value': 'Рзоумный Илья Аркадьевич работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2019-11-18. Дата рождения: 1988-09-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рмоаненко Алексей Николаевич'}, {'from': 'gpt', 'value': 'Рмоаненко Алексей Николаевич работает в организации Волшебный Доктор на должности\n fВрач-колопроктолог, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2020-11-23. Дата рождения: 1978-09-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Роканенмов Лев Олегович'}, {'from': 'gpt', 'value': 'Роканенмов Лев Олегович работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф12 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-12-04. Дата рождения: 1996-05-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ромонав Андрей Витальевич'}, {'from': 'gpt', 'value': 'Ромонав Андрей Витальевич работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале Ф02 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2025-02-13. Дата рождения: 1997-01-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ромнаов Иван Андреевич'}, {'from': 'gpt', 'value': 'Ромнаов Иван Андреевич работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале Ф12 (Отделение анестезиологии и реанимации, Отделение анестезиологии и реанимации). Дата начала работы в организации: 2023-07-17. Дата рождения: 1995-12-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ронамова Марина Ивановна'}, {'from': 'gpt', 'value': 'Ронамова Марина Ивановна работает в организации Волшебный Доктор на должности\n fЗаместитель заведующего клинико-диагностической лабораторией, в филиале Лаборатория (Администрация, Администрация). Дата начала работы в организации: 2013-01-09. Дата рождения: 1965-05-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ромавона Милана Казбековна'}, {'from': 'gpt', 'value': 'Ромавона Милана Казбековна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф12 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-10-28. Дата рождения: 2003-07-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ронамова Яна Юрьевна'}, {'from': 'gpt', 'value': 'Ронамова Яна Юрьевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф11 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-01-11. Дата рождения: 1978-04-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ромначенко Юлия Васильевна'}, {'from': 'gpt', 'value': 'Ромначенко Юлия Васильевна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф16 (Администрация, Администрация). Дата начала работы в организации: 2009-04-24. Дата рождения: 1986-03-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ромакчинов Александр Васильевич'}, {'from': 'gpt', 'value': 'Ромакчинов Александр Васильевич работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф16 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1947-08-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ромшакина Анастасия Сергеевна'}, {'from': 'gpt', 'value': 'Ромшакина Анастасия Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-07-04. Дата рождения: 2003-12-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рубоцв Валерий Дмитриевич'}, {'from': 'gpt', 'value': 'Рубоцв Валерий Дмитриевич работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2018-04-04. Дата рождения: 1963-12-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рунедко Хельга Андреевна'}, {'from': 'gpt', 'value': 'Рунедко Хельга Андреевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2008-07-25. Дата рождения: 1981-07-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рувела Светлана Михайловна'}, {'from': 'gpt', 'value': 'Рувела Светлана Михайловна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2016-06-06. Дата рождения: 1969-07-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Руменцява Мария Юрьевна'}, {'from': 'gpt', 'value': 'Руменцява Мария Юрьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по физиотерапии, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-12-12. Дата рождения: 1989-07-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Росакува Татьяна Петровна'}, {'from': 'gpt', 'value': 'Росакува Татьяна Петровна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф02 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1949-04-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рыбкнсиая Вера Валентиновна'}, {'from': 'gpt', 'value': 'Рыбкнсиая Вера Валентиновна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2025-02-17. Дата рождения: 1979-11-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рыжвока Елена Валерьевна'}, {'from': 'gpt', 'value': 'Рыжвока Елена Валерьевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф06 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2018-04-09. Дата рождения: 1985-02-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рсыь Анна Алексеевна'}, {'from': 'gpt', 'value': 'Рсыь Анна Алексеевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2023-09-18. Дата рождения: 1979-09-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Рящниа Галина Васильевна'}, {'from': 'gpt', 'value': 'Рящниа Галина Васильевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по физиотерапии, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2015-10-22. Дата рождения: 1984-07-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сибинскай Алексей Евгеньевич'}, {'from': 'gpt', 'value': 'Сибинскай Алексей Евгеньевич работает в организации Волшебный Доктор на должности\n fВодитель скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2020-10-15. Дата рождения: 1975-12-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саввативома Софья Владимировна'}, {'from': 'gpt', 'value': 'Саввативома Софья Владимировна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф14 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2019-07-15. Дата рождения: 1985-08-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саввниа Оксана Юрьевна'}, {'from': 'gpt', 'value': 'Саввниа Оксана Юрьевна работает в организации Волшебный Доктор на должности\n fБуфетчица, в филиале ГЦ-Ф15 (Служба сервиса, Стационар). Дата начала работы в организации: 2023-11-20. Дата рождения: 1971-05-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Санкива Алена Александровна'}, {'from': 'gpt', 'value': 'Санкива Алена Александровна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Лаборатория (Консультативно-диагностическое отделение, Консультативно-диагностическое отделение). Дата начала работы в организации: 2009-10-08. Дата рождения: 1988-06-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Савлеав Ацамаз Савельевич'}, {'from': 'gpt', 'value': 'Савлеав Ацамаз Савельевич работает в организации Волшебный Доктор на должности\n fВедущий врач-рентгенолог (КТ), в филиале ГЦ-Ф15 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2018-02-12. Дата рождения: 1989-01-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Совасин Дмитрий Васильевич'}, {'from': 'gpt', 'value': 'Совасин Дмитрий Васильевич работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2020-04-20. Дата рождения: 1978-08-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саечвнко Артём Григорьевич'}, {'from': 'gpt', 'value': 'Саечвнко Артём Григорьевич работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2015-02-09. Дата рождения: 1987-06-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сдаиков Туймахмад Холович'}, {'from': 'gpt', 'value': 'Сдаиков Туймахмад Холович работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2014-11-05. Дата рождения: 1962-01-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саодвая Мария Александровна'}, {'from': 'gpt', 'value': 'Саодвая Мария Александровна работает в организации Волшебный Доктор на должности\n fВрач-рефлексотерапевт, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2021-11-15. Дата рождения: 1990-12-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сдаои Нана Джакевна'}, {'from': 'gpt', 'value': 'Сдаои Нана Джакевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф04А (Консультативно-диагностическое отделение, Консультативно-диагностическое отделение). Дата начала работы в организации: 2024-04-18. Дата рождения: 2001-03-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Садокыва Луиза Гавхаровна'}, {'from': 'gpt', 'value': 'Садокыва Луиза Гавхаровна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале ГЦ-Ф15 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2019-07-09. Дата рождения: 1979-07-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Созонав Сергей Васильевич'}, {'from': 'gpt', 'value': 'Созонав Сергей Васильевич работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2019-08-26. Дата рождения: 1961-10-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сиадова Диана Магомедкамиловна'}, {'from': 'gpt', 'value': 'Сиадова Диана Магомедкамиловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф11 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-08-22. Дата рождения: 1997-07-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саланида Елена Владимировна'}, {'from': 'gpt', 'value': 'Саланида Елена Владимировна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2024-01-25. Дата рождения: 1965-05-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Салаомва Хава Алиевна'}, {'from': 'gpt', 'value': 'Салаомва Хава Алиевна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2024-08-05. Дата рождения: 1995-10-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Снлангиаа Ольга Геннадьевна'}, {'from': 'gpt', 'value': 'Снлангиаа Ольга Геннадьевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф14 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2017-08-29. Дата рождения: 1969-12-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Скленао Алексей Игорьевич'}, {'from': 'gpt', 'value': 'Скленао Алексей Игорьевич работает в организации Волшебный Доктор на должности\n fТехник сменный, в филиале ГЦ-Ф15 (Отдел административно-хозяйственного и технического обеспечения, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-09-28. Дата рождения: 1984-06-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саоихлв Магомедсаид Алигаджиевич'}, {'from': 'gpt', 'value': 'Саоихлв Магомедсаид Алигаджиевич работает в организации Волшебный Доктор на должности\n fВрач скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2018-04-09. Дата рождения: 1984-07-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сяланкина Ольга Юрьевна'}, {'from': 'gpt', 'value': 'Сяланкина Ольга Юрьевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2020-11-03. Дата рождения: 1969-08-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Самадова Ирина Владимировна'}, {'from': 'gpt', 'value': 'Самадова Ирина Владимировна работает в организации Волшебный Доктор на должности\n fМенеджер по сервису, в филиале ГЦ-Ф15 (Служба сервиса, Стационар). Дата начала работы в организации: 2023-05-29. Дата рождения: 1973-08-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Самородсикй Николай Александрович'}, {'from': 'gpt', 'value': 'Самородсикй Николай Александрович работает в организации Волшебный Доктор на должности\n fВрач-ортодонт, в филиале Ф02 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-11-25. Дата рождения: 1993-04-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Савохмалова Нина Игорьевна'}, {'from': 'gpt', 'value': 'Савохмалова Нина Игорьевна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале ГЦ-Ф15 (Центр гастроэнтерологии, Центр гастроэнтерологии). Дата начала работы в организации: 2013-08-07. Дата рождения: 1974-08-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сднау Светлана Валерьевна'}, {'from': 'gpt', 'value': 'Сднау Светлана Валерьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по физиотерапии, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2025-02-24. Дата рождения: 1983-08-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Садныкова Софья Максимовна'}, {'from': 'gpt', 'value': 'Садныкова Софья Максимовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-03-11. Дата рождения: 2003-04-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Скноцаий Игорь Леонидович'}, {'from': 'gpt', 'value': 'Скноцаий Игорь Леонидович работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-02-08. Дата рождения: 2003-01-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саоожникпва Анастасия Викторовна'}, {'from': 'gpt', 'value': 'Саоожникпва Анастасия Викторовна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2018-10-09. Дата рождения: 1992-06-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саповникожа Светлана Витальевна'}, {'from': 'gpt', 'value': 'Саповникожа Светлана Витальевна работает в организации Волшебный Доктор на должности\n fРуководитель отдела госпитализаций, в филиале ГЦ-Ф15 (Отдел госпитализаций, Коммерческий департамент ГЦ). Дата начала работы в организации: 2015-12-22. Дата рождения: 1970-02-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Санарцева Олеся Геннадьевна'}, {'from': 'gpt', 'value': 'Санарцева Олеся Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2024-02-26. Дата рождения: 1994-09-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сариебкян Виктория Артуровна'}, {'from': 'gpt', 'value': 'Сариебкян Виктория Артуровна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2025-02-17. Дата рождения: 1997-08-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Соттарава Ольга Геннадьевна'}, {'from': 'gpt', 'value': 'Соттарава Ольга Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф07 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2004-07-01. Дата рождения: 1968-08-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Савтиеуа Марьем Хамурзаевна'}, {'from': 'gpt', 'value': 'Савтиеуа Марьем Хамурзаевна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2013-08-14. Дата рождения: 1962-06-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сафниа Екатерина Игоревна'}, {'from': 'gpt', 'value': 'Сафниа Екатерина Игоревна работает в организации Волшебный Доктор на должности\n fМедицинский регистратор, в филиале Лаборатория (Экспресс-лаборатория, Экспресс-лаборатория). Дата начала работы в организации: 2024-08-08. Дата рождения: 2006-03-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сафонова Евгения Анатольевна'}, {'from': 'gpt', 'value': 'Сафонова Евгения Анатольевна работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2017-09-25. Дата рождения: 1987-11-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саоонфва Екатерина Вячеславовна'}, {'from': 'gpt', 'value': 'Саоонфва Екатерина Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2024-08-05. Дата рождения: 1998-08-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саоархв Евгений Андреевич'}, {'from': 'gpt', 'value': 'Саоархв Евгений Андреевич работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2023-11-13. Дата рождения: 1991-04-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Снхао Галина Владимировна'}, {'from': 'gpt', 'value': 'Снхао Галина Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф09 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2021-01-25. Дата рождения: 1972-04-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Саянипа Татьяна Федоровна'}, {'from': 'gpt', 'value': 'Саянипа Татьяна Федоровна работает в организации Волшебный Доктор на должности\n fМенеджер по работе со страховыми компаниями, в филиале Управление (Отдел учета реализации, Финансовый департамент). Дата начала работы в организации: 2022-02-24. Дата рождения: 1998-01-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Светлчиная Зоя Юрьевна'}, {'from': 'gpt', 'value': 'Светлчиная Зоя Юрьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу участковая, в филиале Ф01 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2020-11-02. Дата рождения: 1998-10-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сенидкин Алексей Анатольевич'}, {'from': 'gpt', 'value': 'Сенидкин Алексей Анатольевич работает в организации Волшебный Доктор на должности\n fВедущий врач-оториноларинголог, в филиале ГЦ-Ф15 (Отделение оторинолангологии, Стационар). Дата начала работы в организации: 2020-01-13. Дата рождения: 1973-05-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сийрек Виктор Викторович'}, {'from': 'gpt', 'value': 'Сийрек Виктор Викторович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2024-07-29. Дата рождения: 1983-07-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Снлезеёв Олег Геннадьевич'}, {'from': 'gpt', 'value': 'Снлезеёв Олег Геннадьевич работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Управление (Отдел административно-хозяйственного и технического обеспечения, Административный департамент). Дата начала работы в организации: 2020-07-21. Дата рождения: 1973-06-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Селованива Софья Михайловна'}, {'from': 'gpt', 'value': 'Селованива Софья Михайловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2024-10-30. Дата рождения: 2004-02-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Селивока Ирина Павловна'}, {'from': 'gpt', 'value': 'Селивока Ирина Павловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале ГЦ-Ф15 (Отделение реабилитации, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2018-02-05. Дата рождения: 1964-07-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сенемова Виктория Александровна'}, {'from': 'gpt', 'value': 'Сенемова Виктория Александровна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф05 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2020-02-17. Дата рождения: 1971-06-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Семвноёа Марина Павловна'}, {'from': 'gpt', 'value': 'Семвноёа Марина Павловна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф07 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2018-02-22. Дата рождения: 1970-11-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сесбркая Ирэна Юрьевна'}, {'from': 'gpt', 'value': 'Сесбркая Ирэна Юрьевна работает в организации Волшебный Доктор на должности\n fИнструктор-методист по лечебной физкультуре, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-11-18. Дата рождения: 1992-09-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Серегев Даниил Андреевич'}, {'from': 'gpt', 'value': 'Серегев Даниил Андреевич работает в организации Волшебный Доктор на должности\n fМастер по ремонту медицинского оборудования, в филиале Управление (Отдел эксплуатации медицинского оборудования, Медицинский департамент). Дата начала работы в организации: 2022-11-03. Дата рождения: 2001-09-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сергеева Анастасия Игоревна'}, {'from': 'gpt', 'value': 'Сергеева Анастасия Игоревна работает в организации Волшебный Доктор на должности\n fМенеджер по плановой госпитализации сменный, в филиале ГЦ-Ф15 (Отдел госпитализаций и сервиса, Коммерческий департамент ГЦ). Дата начала работы в организации: 2017-11-09. Дата рождения: 1995-03-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сергеева Анна Евгеньевна'}, {'from': 'gpt', 'value': 'Сергеева Анна Евгеньевна работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача, в филиале Ф14 (Администрация, Администрация). Дата начала работы в организации: 2023-11-07. Дата рождения: 1983-08-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Серюдк Ольга Николаевна'}, {'from': 'gpt', 'value': 'Серюдк Ольга Николаевна работает в организации Волшебный Доктор на должности\n fМенеджер по сервису, в филиале ГЦ-Ф15 (Служба сервиса, Стационар). Дата начала работы в организации: 2021-03-22. Дата рождения: 1967-09-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Середин Денис Владимирович'}, {'from': 'gpt', 'value': 'Середин Денис Владимирович работает в организации Волшебный Доктор на должности\n fВидеограф-монтажер, в филиале Управление (Отдел маркетинга специализированных центров, Департамент Специализированных проектов). Дата начала работы в организации: 2023-09-21. Дата рождения: 1984-03-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Срекова Марита Тикановна'}, {'from': 'gpt', 'value': 'Срекова Марита Тикановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра эндоскопии, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2017-08-02. Дата рождения: 1963-06-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сероенкчв Александр Алексеевич'}, {'from': 'gpt', 'value': 'Сероенкчв Александр Алексеевич работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2019-08-28. Дата рождения: 1994-08-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сгиитова Елена Витальевна'}, {'from': 'gpt', 'value': 'Сгиитова Елена Витальевна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-07-15. Дата рождения: 1993-09-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сидлеьникова Ольга Михайловна'}, {'from': 'gpt', 'value': 'Сидлеьникова Ольга Михайловна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2022-09-05. Дата рождения: 1966-08-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Синедко Татьяна Анатольевна'}, {'from': 'gpt', 'value': 'Синедко Татьяна Анатольевна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2016-11-07. Дата рождения: 1967-03-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сиодренко Виталий Николаевич'}, {'from': 'gpt', 'value': 'Сиодренко Виталий Николаевич работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение микробиологии, Отделение микробиологии). Дата начала работы в организации: 2018-02-26. Дата рождения: 1990-03-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сидокрина Людмила Васильевна'}, {'from': 'gpt', 'value': 'Сидокрина Людмила Васильевна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф01 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-29. Дата рождения: 1960-12-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сиодрова Елена Сергеевна'}, {'from': 'gpt', 'value': 'Сиодрова Елена Сергеевна работает в организации Волшебный Доктор на должности\n fСпециалист отдела автоматизации и проектирования бизнес-процессов, в филиале Управление (Отдел автоматизации и проектирования бизнес-процессов, Отдел автоматизации и проектирования бизнес-процессов). Дата начала работы в организации: 2024-12-19. Дата рождения: 1983-10-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сикоаскря Анна Николаевна'}, {'from': 'gpt', 'value': 'Сикоаскря Анна Николаевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2018-09-17. Дата рождения: 1989-01-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сннекииа Любовь Александровна'}, {'from': 'gpt', 'value': 'Сннекииа Любовь Александровна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2016-06-08. Дата рождения: 1957-07-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сиеотрнко Анастасия Владимировна'}, {'from': 'gpt', 'value': 'Сиеотрнко Анастасия Владимировна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2024-05-13. Дата рождения: 1995-11-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Срвикская Ольга Анатольевна'}, {'from': 'gpt', 'value': 'Срвикская Ольга Анатольевна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2019-09-02. Дата рождения: 1993-01-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Совкрцова Разиля Фаатовна'}, {'from': 'gpt', 'value': 'Совкрцова Разиля Фаатовна работает в организации Волшебный Доктор на должности\n fГлавный оператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2020-06-23. Дата рождения: 1975-09-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Скоеебв Игорь Георгиевич'}, {'from': 'gpt', 'value': 'Скоеебв Игорь Георгиевич работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2018-01-15. Дата рождения: 1955-04-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Скоропувопа Татьяна Вячеславовна'}, {'from': 'gpt', 'value': 'Скоропувопа Татьяна Вячеславовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра эндоскопии, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2025-02-13. Дата рождения: 1977-07-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Слепчинко Светлана Леонидовна'}, {'from': 'gpt', 'value': 'Слепчинко Светлана Леонидовна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2022-09-05. Дата рождения: 1960-05-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Снирмов Михаил Евгеньевич'}, {'from': 'gpt', 'value': 'Снирмов Михаил Евгеньевич работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2020-07-06. Дата рождения: 1964-06-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Снирмова Элисия Борисовна'}, {'from': 'gpt', 'value': 'Снирмова Элисия Борисовна работает в организации Волшебный Доктор на должности\n fЗаведующий педиатрическим отделением, в филиале Ф17-ДЦ (Отделение педиатрии, Отделение педиатрии). Дата начала работы в организации: 2021-05-11. Дата рождения: 1994-03-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Смионрва Ольга Владимировна'}, {'from': 'gpt', 'value': 'Смионрва Ольга Владимировна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф17-ДЦ (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1974-10-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Смивнора Светлана Васильевна'}, {'from': 'gpt', 'value': 'Смивнора Светлана Васильевна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2011-04-14. Дата рождения: 1981-08-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Синтко Евгений Анатольевич'}, {'from': 'gpt', 'value': 'Синтко Евгений Анатольевич работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2024-06-10. Дата рождения: 1988-12-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Снопнеко Виктория Олеговна'}, {'from': 'gpt', 'value': 'Снопнеко Виктория Олеговна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф16 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2023-12-04. Дата рождения: 1996-05-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Собовела Екатерина Алексеевна'}, {'from': 'gpt', 'value': 'Собовела Екатерина Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2019-09-09. Дата рождения: 1989-04-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Соволока Мария Игоревна'}, {'from': 'gpt', 'value': 'Соволока Мария Игоревна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2025-02-17. Дата рождения: 1984-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Соколова Оксана Ивановна'}, {'from': 'gpt', 'value': 'Соколова Оксана Ивановна работает в организации Волшебный Доктор на должности\n fМенеджер по продажам юридическим лицам, в филиале Управление (Центр корпоративных продаж, Коммерческий департамент). Дата начала работы в организации: 2014-09-25. Дата рождения: 1970-06-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Соколова Оксана Ростиславовна'}, {'from': 'gpt', 'value': 'Соколова Оксана Ростиславовна работает в организации Волшебный Доктор на должности\n fДиректор коммерческого департамента, в филиале Управление (Коммерческий департамент, Коммерческий департамент). Дата начала работы в организации: 2010-04-08. Дата рождения: 1988-10-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Соолвейчук Мария Евгеньевна'}, {'from': 'gpt', 'value': 'Соолвейчук Мария Евгеньевна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2024-05-23. Дата рождения: 1986-04-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Соловьев Владимир Сергеевич'}, {'from': 'gpt', 'value': 'Соловьев Владимир Сергеевич работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Экспресс-лаборатория, Экспресс-лаборатория). Дата начала работы в организации: 2017-07-24. Дата рождения: 1996-05-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Солевьова Анастасия Александровна'}, {'from': 'gpt', 'value': 'Солевьова Анастасия Александровна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог (КТ), в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2025-01-29. Дата рождения: 1990-06-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Соолвьёва София Алексеевна'}, {'from': 'gpt', 'value': 'Соолвьёва София Алексеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2024-10-30. Дата рождения: 2004-07-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Соолмин Алексей Сергеевич'}, {'from': 'gpt', 'value': 'Соолмин Алексей Сергеевич работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2024-12-05. Дата рождения: 2005-09-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Соланорь Василий Александрович'}, {'from': 'gpt', 'value': 'Соланорь Василий Александрович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф10 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2014-10-10. Дата рождения: 1960-07-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Силоноцын Александр Владимирович'}, {'from': 'gpt', 'value': 'Силоноцын Александр Владимирович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2023-11-23. Дата рождения: 1984-10-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сосновский Роман Олегович'}, {'from': 'gpt', 'value': 'Сосновский Роман Олегович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-хирург, в филиале Ф16 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2025-02-24. Дата рождения: 1995-02-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Соншева Наталья Сергеевна'}, {'from': 'gpt', 'value': 'Соншева Наталья Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2023-10-16. Дата рождения: 1997-02-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Спивидонора Ольга Николаевна'}, {'from': 'gpt', 'value': 'Спивидонора Ольга Николаевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф12 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2017-08-15. Дата рождения: 1972-01-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Снатишич Вероника Валерьевна'}, {'from': 'gpt', 'value': 'Снатишич Вероника Валерьевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф16 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2010-11-17. Дата рождения: 1980-09-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сторавойтова Светлана Александровна'}, {'from': 'gpt', 'value': 'Сторавойтова Светлана Александровна работает в организации Волшебный Доктор на должности\n fБухгалтер по расчетам с поставщиками, в филиале Управление (Отдел учета расходов, Финансовый департамент). Дата начала работы в организации: 2022-03-28. Дата рождения: 1986-01-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Стакшенро Валерия Васильевна'}, {'from': 'gpt', 'value': 'Стакшенро Валерия Васильевна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2017-08-16. Дата рождения: 1995-07-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Снепатов Сергей Владимирович'}, {'from': 'gpt', 'value': 'Снепатов Сергей Владимирович работает в организации Волшебный Доктор на должности\n fКоммерческий директор Госпитального центра, в филиале ГЦ-Ф15 (Администрация, Администрация). Дата начала работы в организации: 2007-09-12. Дата рождения: 1978-07-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ствпаноеа Татьяна Николаевна'}, {'from': 'gpt', 'value': 'Ствпаноеа Татьяна Николаевна работает в организации Волшебный Доктор на должности\n fГлавный оператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2009-01-29. Дата рождения: 1975-08-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сетц Алина Ивановна'}, {'from': 'gpt', 'value': 'Сетц Алина Ивановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-07-18. Дата рождения: 1999-11-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сцрелеткая Юлия Владиславовна'}, {'from': 'gpt', 'value': 'Сцрелеткая Юлия Владиславовна работает в организации Волшебный Доктор на должности\n fПомощник менеджера стоматологического центра, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-07-29. Дата рождения: 2004-11-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Снроготов Андрей Васильевич'}, {'from': 'gpt', 'value': 'Снроготов Андрей Васильевич работает в организации Волшебный Доктор на должности\n fЗаведующий терапевтическим отделением, в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2018-08-27. Дата рождения: 1980-09-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Строоква Татьяна Викторовна'}, {'from': 'gpt', 'value': 'Строоква Татьяна Викторовна работает в организации Волшебный Доктор на должности\n fВрач-эксперт-гастроэнтеролог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-07-31. Дата рождения: 1967-01-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Стчорек Дарья Сергеевна'}, {'from': 'gpt', 'value': 'Стчорек Дарья Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-ортодонт, в филиале Ф10 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-04-01. Дата рождения: 1998-02-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Студнникиеа Екатерина Сергеевна'}, {'from': 'gpt', 'value': 'Студнникиеа Екатерина Сергеевна работает в организации Волшебный Доктор на должности\n fМенеджер по продажам юридическим лицам, в филиале Управление (Центр корпоративных продаж, Коммерческий департамент). Дата начала работы в организации: 2022-02-14. Дата рождения: 1980-05-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Стулвкоиа Галина Валерьевна'}, {'from': 'gpt', 'value': 'Стулвкоиа Галина Валерьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф09 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2022-05-16. Дата рождения: 1967-04-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Субботин Николай Викторович'}, {'from': 'gpt', 'value': 'Субботин Николай Викторович работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2022-06-20. Дата рождения: 1993-03-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Субхнакулова Ольга Петровна'}, {'from': 'gpt', 'value': 'Субхнакулова Ольга Петровна работает в организации Волшебный Доктор на должности\n fДиректор департамента Финансов и Эксплуатации, в филиале Управление (Финансовый департамент, Финансовый департамент). Дата начала работы в организации: 2013-04-04. Дата рождения: 1975-01-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сувовора Галина Васильевна'}, {'from': 'gpt', 'value': 'Сувовора Галина Васильевна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2023-02-27. Дата рождения: 1988-09-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Срвоуова Надежда Алексеевна'}, {'from': 'gpt', 'value': 'Срвоуова Надежда Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-хирург, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-03-05. Дата рождения: 1987-04-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Суадкова Наталья Сергеевна'}, {'from': 'gpt', 'value': 'Суадкова Наталья Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2018-09-19. Дата рождения: 1992-07-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Суеовцрв Филипп Александрович'}, {'from': 'gpt', 'value': 'Суеовцрв Филипп Александрович работает в организации Волшебный Доктор на должности\n fДиректор департамента развития, в филиале Управление (Департамент развития, Департамент развития). Дата начала работы в организации: 2019-04-08. Дата рождения: 1984-04-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ссуакина Марианна Ивановна'}, {'from': 'gpt', 'value': 'Ссуакина Марианна Ивановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2024-03-21. Дата рождения: 1978-08-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Суахнова Татьяна Владимировна'}, {'from': 'gpt', 'value': 'Суахнова Татьяна Владимировна работает в организации Волшебный Доктор на должности\n fБухгалтер участка "Банк-клиент", в филиале Управление (Отдел учета расходов, Финансовый департамент). Дата начала работы в организации: 2025-03-03. Дата рождения: 1986-01-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Суохва Мария Павловна'}, {'from': 'gpt', 'value': 'Суохва Мария Павловна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф09 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-03-04. Дата рождения: 1994-08-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сукшов Михаил Евгеньевич'}, {'from': 'gpt', 'value': 'Сукшов Михаил Евгеньевич работает в организации Волшебный Доктор на должности\n fТехник, в филиале Ф11 (АХО, Вспомогательная служба). Дата начала работы в организации: 2022-06-20. Дата рождения: 1988-06-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Сошкува Ирина Алексеевна'}, {'from': 'gpt', 'value': 'Сошкува Ирина Алексеевна работает в организации Волшебный Доктор на должности\n fМедицинский оптик-оптометрист, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-02-12. Дата рождения: 1996-06-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тавжлоанская Татьяна Васильевна'}, {'from': 'gpt', 'value': 'Тавжлоанская Татьяна Васильевна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф12 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2007-10-09. Дата рождения: 1960-07-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тагрибегова Кизил Зиявудиновна'}, {'from': 'gpt', 'value': 'Тагрибегова Кизил Зиявудиновна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-04-04. Дата рождения: 2003-09-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тажонев Темерлан Нурланович'}, {'from': 'gpt', 'value': 'Тажонев Темерлан Нурланович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-10-01. Дата рождения: 2003-04-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тапосханава Светлана Алиевна'}, {'from': 'gpt', 'value': 'Тапосханава Светлана Алиевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2024-09-17. Дата рождения: 1970-06-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тараваеда Наталья Михайловна'}, {'from': 'gpt', 'value': 'Тараваеда Наталья Михайловна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-01-29. Дата рождения: 1991-02-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Твраеаа Анна Евгеньевна'}, {'from': 'gpt', 'value': 'Твраеаа Анна Евгеньевна работает в организации Волшебный Доктор на должности\n fСпециалист отдела финансового контроллинга, в филиале Управление (Отдел финансового контроллинга, Финансовый департамент). Дата начала работы в организации: 2023-07-13. Дата рождения: 1993-10-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Таарки Брешна Мирза'}, {'from': 'gpt', 'value': 'Таарки Брешна Мирза работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале ГЦ-Ф15 (Центр гастроэнтерологии, Центр гастроэнтерологии). Дата начала работы в организации: 2024-07-22. Дата рождения: 1998-01-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Таракенсо Ирина Ильинична'}, {'from': 'gpt', 'value': 'Таракенсо Ирина Ильинична работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2023-08-17. Дата рождения: 1976-09-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Таеасрнко Юлия Владимировна'}, {'from': 'gpt', 'value': 'Таеасрнко Юлия Владимировна работает в организации Волшебный Доктор на должности\n fДиректор центра корпоративных и госпитальных продаж, в филиале Управление (Центр корпоративных продаж, Коммерческий департамент). Дата начала работы в организации: 2016-11-15. Дата рождения: 1983-10-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тарасова Ольга Александровна'}, {'from': 'gpt', 'value': 'Тарасова Ольга Александровна работает в организации Волшебный Доктор на должности\n fОператор, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2021-04-07. Дата рождения: 1978-12-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тарсаова Ольга Дмитриевна'}, {'from': 'gpt', 'value': 'Тарсаова Ольга Дмитриевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф17-ДЦ (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2021-08-23. Дата рождения: 1983-10-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Таиелкрн Алексей Валерьевич'}, {'from': 'gpt', 'value': 'Таиелкрн Алексей Валерьевич работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2009-11-17. Дата рождения: 1985-05-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Таоусрв Андрей Васильевич'}, {'from': 'gpt', 'value': 'Таоусрв Андрей Васильевич работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф16 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2015-03-11. Дата рождения: 1965-09-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Татерчанко Валерия Владимировна'}, {'from': 'gpt', 'value': 'Татерчанко Валерия Владимировна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-11-18. Дата рождения: 1995-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Теерхин Сергей Анатольевич'}, {'from': 'gpt', 'value': 'Теерхин Сергей Анатольевич работает в организации Волшебный Доктор на должности\n fЗаведующий клинико-диагностической лабораторией, в филиале Лаборатория (Администрация, Администрация). Дата начала работы в организации: 2007-05-15. Дата рождения: 1983-05-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Теерхов Дмитрий Сергеевич'}, {'from': 'gpt', 'value': 'Теерхов Дмитрий Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2020-02-10. Дата рождения: 1992-03-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Телроев Имран Маулович'}, {'from': 'gpt', 'value': 'Телроев Имран Маулович работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале ГЦ-Ф15 (Отделение оторинолангологии, Стационар). Дата начала работы в организации: 2022-06-14. Дата рождения: 1994-01-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Теаскря Валерия Алексеевна'}, {'from': 'gpt', 'value': 'Теаскря Валерия Алексеевна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2022-10-24. Дата рождения: 2001-02-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тивикова Алена Юрьевна'}, {'from': 'gpt', 'value': 'Тивикова Алена Юрьевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале ГЦ-Ф15 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2023-06-26. Дата рождения: 1995-03-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тилеоква Айзанат Ильясовна'}, {'from': 'gpt', 'value': 'Тилеоква Айзанат Ильясовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2023-08-17. Дата рождения: 1998-12-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тимофеев Михаил Евгеньевич'}, {'from': 'gpt', 'value': 'Тимофеев Михаил Евгеньевич работает в организации Волшебный Доктор на должности\n fВрач-эксперт-эндоскопист, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-03-19. Дата рождения: 1964-10-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тимефеова Вероника Леонидовна'}, {'from': 'gpt', 'value': 'Тимефеова Вероника Леонидовна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2009-11-17. Дата рождения: 1968-02-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тимоеефва Ольга Владимировна'}, {'from': 'gpt', 'value': 'Тимоеефва Ольга Владимировна работает в организации Волшебный Доктор на должности\n fМенеджер по работе с пациентами сменный, в филиале Управление (Отдел госпитальных продаж, Коммерческий департамент). Дата начала работы в организации: 2017-04-06. Дата рождения: 1975-05-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Темофеива Ольга Николаевна'}, {'from': 'gpt', 'value': 'Темофеива Ольга Николаевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2011-09-13. Дата рождения: 1955-02-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тииохмн Владислав Анатольевич'}, {'from': 'gpt', 'value': 'Тииохмн Владислав Анатольевич работает в организации Волшебный Доктор на должности\n fПрезидент, в филиале Управление (Руководство, Руководство). Дата начала работы в организации: 2007-09-01. Дата рождения: 1965-11-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тимщоенко Татьяна Геннадьевна'}, {'from': 'gpt', 'value': 'Тимщоенко Татьяна Геннадьевна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф10 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1958-05-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тиотв Павел Константинович'}, {'from': 'gpt', 'value': 'Тиотв Павел Константинович работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф11 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2025-02-03. Дата рождения: 2004-04-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тиотва Елена Александровна'}, {'from': 'gpt', 'value': 'Тиотва Елена Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2017-08-07. Дата рождения: 1988-01-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ттиова Мария Валентиновна'}, {'from': 'gpt', 'value': 'Ттиова Мария Валентиновна работает в организации Волшебный Доктор на должности\n fМенеджер стоматологического отделения, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2021-04-12. Дата рождения: 1972-02-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тихоморива Екатерина Александровна'}, {'from': 'gpt', 'value': 'Тихоморива Екатерина Александровна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-03-11. Дата рождения: 1990-04-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тиохнов Александр Игоревич'}, {'from': 'gpt', 'value': 'Тиохнов Александр Игоревич работает в организации Волшебный Доктор на должности\n fНачальник отдела по эксплуатации медоборудования, в филиале Управление (Отдел эксплуатации медицинского оборудования, Медицинский департамент). Дата начала работы в организации: 2022-02-01. Дата рождения: 1984-09-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тинохова Татьяна Владимировна'}, {'from': 'gpt', 'value': 'Тинохова Татьяна Владимировна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале Ф17-ДЦ (Отделение педиатрии, Отделение педиатрии). Дата начала работы в организации: 2019-11-05. Дата рождения: 1972-12-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тищнеко Ирина Витальевна'}, {'from': 'gpt', 'value': 'Тищнеко Ирина Витальевна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2021-03-29. Дата рождения: 1985-02-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Такч Игорь Сергеевич'}, {'from': 'gpt', 'value': 'Такч Игорь Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2023-04-10. Дата рождения: 1983-03-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тквчёаа Инна Васильевна'}, {'from': 'gpt', 'value': 'Тквчёаа Инна Васильевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2024-08-06. Дата рождения: 1984-02-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Такчева Ксения Юрьевна'}, {'from': 'gpt', 'value': 'Такчева Ксения Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр, в филиале ГЦ-Ф15 (Педиатрическое отделение, Стационар). Дата начала работы в организации: 2024-02-19. Дата рождения: 1993-01-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тляумветоба Юлия Сагитовна'}, {'from': 'gpt', 'value': 'Тляумветоба Юлия Сагитовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф07 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2022-02-10. Дата рождения: 1993-06-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ток Анна Анатольевна'}, {'from': 'gpt', 'value': 'Ток Анна Анатольевна работает в организации Волшебный Доктор на должности\n fНачальник отдела корпоративной редакции, в филиале Управление (Учебный центр/Отдел корпоративной редакции, Департамент по работе с персоналом). Дата начала работы в организации: 2024-08-14. Дата рождения: 1977-04-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Токкамова Ирина Александровна'}, {'from': 'gpt', 'value': 'Токкамова Ирина Александровна работает в организации Волшебный Доктор на должности\n fРуководитель стоматологического центра, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2021-11-15. Дата рождения: 1976-04-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тблоузов Максим Александрович'}, {'from': 'gpt', 'value': 'Тблоузов Максим Александрович работает в организации Волшебный Доктор на должности\n fСистемный администратор баз данных, в филиале Управление (Отдел разработки корпоративных систем, Центр разработки ПО). Дата начала работы в организации: 2024-02-15. Дата рождения: 1984-03-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тлокунов Петр Вячеславович'}, {'from': 'gpt', 'value': 'Тлокунов Петр Вячеславович работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2020-12-14. Дата рождения: 1973-09-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Точмалев Иван Владимирович'}, {'from': 'gpt', 'value': 'Точмалев Иван Владимирович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2025-02-17. Дата рождения: 1998-09-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Темашова Лариса Юрьевна'}, {'from': 'gpt', 'value': 'Темашова Лариса Юрьевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-12-07. Дата рождения: 1963-11-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Трнбиеа Мария Игоревна'}, {'from': 'gpt', 'value': 'Трнбиеа Мария Игоревна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф03 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2014-07-07. Дата рождения: 1989-06-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тгерубов Сергей Сергеевич'}, {'from': 'gpt', 'value': 'Тгерубов Сергей Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2021-04-19. Дата рождения: 1988-04-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Треяьткова Елена Владимировна'}, {'from': 'gpt', 'value': 'Треяьткова Елена Владимировна работает в организации Волшебный Доктор на должности\n fОператор, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2023-07-27. Дата рождения: 1977-12-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тноилира Людмила Робертовна'}, {'from': 'gpt', 'value': 'Тноилира Людмила Робертовна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2018-10-01. Дата рождения: 1967-09-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Трофиомва Надежда Николаевна'}, {'from': 'gpt', 'value': 'Трофиомва Надежда Николаевна работает в организации Волшебный Доктор на должности\n fБуфетчица, в филиале ГЦ-Ф15 (Служба сервиса, Стационар). Дата начала работы в организации: 2023-11-07. Дата рождения: 1967-05-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Трофмиова Наталья Валентиновна'}, {'from': 'gpt', 'value': 'Трофмиова Наталья Валентиновна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2024-08-19. Дата рождения: 1973-02-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тровимофа Наталья Олеговна'}, {'from': 'gpt', 'value': 'Тровимофа Наталья Олеговна работает в организации Волшебный Доктор на должности\n fЗаместитель начальника отдела учета реализации, в филиале Управление (Отдел учета реализации, Финансовый департамент). Дата начала работы в организации: 2019-05-31. Дата рождения: 1991-06-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тношкира Анна Сергеевна'}, {'from': 'gpt', 'value': 'Тношкира Анна Сергеевна работает в организации Волшебный Доктор на должности\n fВедущий специалист отдела подбора и адаптации, в филиале Управление (Отдел подбора и адаптации, Департамент по работе с персоналом). Дата начала работы в организации: 2018-12-13. Дата рождения: 1994-07-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Туршин Николай Николаевич'}, {'from': 'gpt', 'value': 'Туршин Николай Николаевич работает в организации Волшебный Доктор на должности\n fНачальник отдела административно-хозяйственного и технического обеспечения, в филиале Управление (Отдел административно-хозяйственного и технического обеспечения, Административный департамент). Дата начала работы в организации: 2007-09-15. Дата рождения: 1971-11-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тиушкрн Руслан Николаевич'}, {'from': 'gpt', 'value': 'Тиушкрн Руслан Николаевич работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2022-03-02. Дата рождения: 1972-10-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тауева Анжелика Сергеевна'}, {'from': 'gpt', 'value': 'Тауева Анжелика Сергеевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2023-09-18. Дата рождения: 1967-09-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тнркиуа Екатерина Алексеевна'}, {'from': 'gpt', 'value': 'Тнркиуа Екатерина Алексеевна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2023-09-04. Дата рождения: 2001-05-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тичуч Светлана Николаевна'}, {'from': 'gpt', 'value': 'Тичуч Светлана Николаевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф06 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2018-11-22. Дата рождения: 1977-08-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тншиуа Ольга Евгеньевна'}, {'from': 'gpt', 'value': 'Тншиуа Ольга Евгеньевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф04А (Консультативно-диагностическое отделение, Консультативно-диагностическое отделение). Дата начала работы в организации: 2013-09-16. Дата рождения: 1972-09-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тымчекно Светлана Сергеевна'}, {'from': 'gpt', 'value': 'Тымчекно Светлана Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-терапевт, в филиале Ф03 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-09-09. Дата рождения: 1981-11-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тлюенев Сергей Николаевич'}, {'from': 'gpt', 'value': 'Тлюенев Сергей Николаевич работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2024-12-09. Дата рождения: 1975-07-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тянугов Антон Александрович'}, {'from': 'gpt', 'value': 'Тянугов Антон Александрович работает в организации Волшебный Доктор на должности\n fЗаведующий операционным блоком, в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2024-11-05. Дата рождения: 1992-10-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Тян Маргарита Петровна'}, {'from': 'gpt', 'value': 'Тян Маргарита Петровна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале ГЦ-Ф15 (Отделение ВОП/педиатрии, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2013-12-23. Дата рождения: 1969-05-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Уоарвва Мария Вячеславовна'}, {'from': 'gpt', 'value': 'Уоарвва Мария Вячеславовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2014-08-06. Дата рождения: 1993-11-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Удлаов Владимир Владимирович'}, {'from': 'gpt', 'value': 'Удлаов Владимир Владимирович работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2019-05-15. Дата рождения: 1990-09-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Удвлоаа Ольга Валерьевна'}, {'from': 'gpt', 'value': 'Удвлоаа Ольга Валерьевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф03 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2018-10-23. Дата рождения: 1978-05-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Уквлооа Анастасия Павловна'}, {'from': 'gpt', 'value': 'Уквлооа Анастасия Павловна работает в организации Волшебный Доктор на должности\n fВрач-психотерапевт, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2023-05-29. Дата рождения: 1990-04-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Уламьн Кристина Константиновна'}, {'from': 'gpt', 'value': 'Уламьн Кристина Константиновна работает в организации Волшебный Доктор на должности\n fМенеджер центра косметологии и пластической хирургии, в филиале Управление (Центр косметологии и пластической хирургии, Коммерческий департамент). Дата начала работы в организации: 2024-01-15. Дата рождения: 2000-05-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Умвеаа Диана Магомедмурзаевна'}, {'from': 'gpt', 'value': 'Умвеаа Диана Магомедмурзаевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2024-10-03. Дата рождения: 2002-08-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Умвтоеа Гульнара Баисовна'}, {'from': 'gpt', 'value': 'Умвтоеа Гульнара Баисовна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2021-09-09. Дата рождения: 1969-07-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Утраева Жанна Герасовна'}, {'from': 'gpt', 'value': 'Утраева Жанна Герасовна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2022-01-24. Дата рождения: 1970-06-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Уучрк Татьяна Васильевна'}, {'from': 'gpt', 'value': 'Уучрк Татьяна Васильевна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-04-03. Дата рождения: 1996-10-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Усвока Ольга Владимировна'}, {'from': 'gpt', 'value': 'Усвока Ольга Владимировна работает в организации Волшебный Доктор на должности\n fМенеджер стоматологического центра, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2015-11-17. Дата рождения: 1973-07-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Успеснкая Елена Алексеевна'}, {'from': 'gpt', 'value': 'Успеснкая Елена Алексеевна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2021-05-24. Дата рождения: 1976-03-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Утшеева Айдана Болатовна'}, {'from': 'gpt', 'value': 'Утшеева Айдана Болатовна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-12-09. Дата рождения: 1994-04-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Уитгалиев Диас Алексеевич'}, {'from': 'gpt', 'value': 'Уитгалиев Диас Алексеевич работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-ортопед, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-04-15. Дата рождения: 1995-01-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Утнасувона Амра Михайловна'}, {'from': 'gpt', 'value': 'Утнасувона Амра Михайловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф02 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2025-01-27. Дата рождения: 1980-11-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ушавока Диана Владимировна'}, {'from': 'gpt', 'value': 'Ушавока Диана Владимировна работает в организации Волшебный Доктор на должности\n fНачальник отдела учета реализации, в филиале Управление (Отдел учета реализации, Финансовый департамент). Дата начала работы в организации: 2008-06-05. Дата рождения: 1977-03-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фбйаушевич Марина Николаевна'}, {'from': 'gpt', 'value': 'Фбйаушевич Марина Николаевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2012-12-17. Дата рождения: 1960-08-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фелалаева Елена Максимовна'}, {'from': 'gpt', 'value': 'Фелалаева Елена Максимовна работает в организации Волшебный Доктор на должности\n fВрач-ординатор, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2024-09-04. Дата рождения: 1999-03-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фраахов Рифкат Равилович'}, {'from': 'gpt', 'value': 'Фраахов Рифкат Равилович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2023-04-17. Дата рождения: 1993-01-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фаирон Елена Юрьевна'}, {'from': 'gpt', 'value': 'Фаирон Елена Юрьевна работает в организации Волшебный Доктор на должности\n fЗаведующий отделением оториноларингологии, в филиале ГЦ-Ф15 (Отделение оторинолангологии, Стационар). Дата начала работы в организации: 2023-05-15. Дата рождения: 1978-01-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фвттахоаа Зульфия Тальгатевна'}, {'from': 'gpt', 'value': 'Фвттахоаа Зульфия Тальгатевна работает в организации Волшебный Доктор на должности\n fВрач скорой медицинской помощи, в филиале ГЦ-Ф15 (Отделение скорой и неотложной медицинской помощи и приемный покой, Стационар). Дата начала работы в организации: 2023-07-03. Дата рождения: 1984-09-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фатькниа Диана Викторовна'}, {'from': 'gpt', 'value': 'Фатькниа Диана Викторовна работает в организации Волшебный Доктор на должности\n fЗаместитель руководителя службы поддержки клиентов, в филиале Управление (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-07-01. Дата рождения: 1987-05-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фердедс Алексей Юрьевич'}, {'from': 'gpt', 'value': 'Фердедс Алексей Юрьевич работает в организации Волшебный Доктор на должности\n fСтарший программист, в филиале Управление (Отдел разработки интернет-проектов, Центр разработки ПО). Дата начала работы в организации: 2018-01-25. Дата рождения: 1992-05-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фдеин Алексей Юрьевич'}, {'from': 'gpt', 'value': 'Фдеин Алексей Юрьевич работает в организации Волшебный Доктор на должности\n fВодитель автомобиля, в филиале Ф05 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2011-01-13. Дата рождения: 1972-07-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Федесеов Александр Анатольевич'}, {'from': 'gpt', 'value': 'Федесеов Александр Анатольевич работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2023-12-18. Дата рождения: 1966-11-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фодетов Роман Русланович'}, {'from': 'gpt', 'value': 'Фодетов Роман Русланович работает в организации Волшебный Доктор на должности\n fВодитель, в филиале Ф01 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2023-04-03. Дата рождения: 1987-06-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фодотев Станислав Викторович'}, {'from': 'gpt', 'value': 'Фодотев Станислав Викторович работает в организации Волшебный Доктор на должности\n fВедущий врач-хирург, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2018-08-01. Дата рождения: 1980-04-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фодотева Анастасия Алексеевна'}, {'from': 'gpt', 'value': 'Фодотева Анастасия Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф12 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-10-01. Дата рождения: 1998-12-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фодетова Анна Сергеевна'}, {'from': 'gpt', 'value': 'Фодетова Анна Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-02-05. Дата рождения: 1982-06-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Федтоова Галина Викторовна'}, {'from': 'gpt', 'value': 'Федтоова Галина Викторовна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2007-05-07. Дата рождения: 1983-02-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фвдотоеа Екатерина Дмитриевна'}, {'from': 'gpt', 'value': 'Фвдотоеа Екатерина Дмитриевна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф12 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2023-11-13. Дата рождения: 1997-05-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Флдуеова Елена Владимировна'}, {'from': 'gpt', 'value': 'Флдуеова Елена Владимировна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф12 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2019-07-18. Дата рождения: 1977-11-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фееснко Татьяна Анатольевна'}, {'from': 'gpt', 'value': 'Фееснко Татьяна Анатольевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2018-11-26. Дата рождения: 1980-04-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Феецтр Денис Витальевич'}, {'from': 'gpt', 'value': 'Феецтр Денис Витальевич работает в организации Волшебный Доктор на должности\n fВедущий врач по рентгенэндоваскулярным диагностике и лечению, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2019-01-16. Дата рождения: 1979-02-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фещкнео Светлана Юрьевна'}, {'from': 'gpt', 'value': 'Фещкнео Светлана Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2018-02-05. Дата рождения: 1989-08-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фтлаиов Константин Борисович'}, {'from': 'gpt', 'value': 'Фтлаиов Константин Борисович работает в организации Волшебный Доктор на должности\n fСистемный администратор, в филиале Управление (Отдел системного администрирования, Департамент ИТ). Дата начала работы в организации: 2022-12-14. Дата рождения: 1992-03-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фнлимоиов Дмитрий Юрьевич'}, {'from': 'gpt', 'value': 'Фнлимоиов Дмитрий Юрьевич работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2015-01-02. Дата рождения: 1984-11-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Филвмоноиа Инна Геннадьевна'}, {'from': 'gpt', 'value': 'Филвмоноиа Инна Геннадьевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф11 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2013-09-13. Дата рождения: 1969-03-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Филимноова Наталья Андреевна'}, {'from': 'gpt', 'value': 'Филимноова Наталья Андреевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2024-08-05. Дата рождения: 1980-06-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фолиппива Анастасия Васильевна'}, {'from': 'gpt', 'value': 'Фолиппива Анастасия Васильевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу участковая, в филиале Ф17-ДЦ (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2022-03-24. Дата рождения: 1984-01-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Филиоппва Дарья Сергеевна'}, {'from': 'gpt', 'value': 'Филиоппва Дарья Сергеевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф17-ДЦ (КДО, КДО). Дата начала работы в организации: 2024-11-18. Дата рождения: 1996-11-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фврсоиа Дарья Васильевна'}, {'from': 'gpt', 'value': 'Фврсоиа Дарья Васильевна работает в организации Волшебный Доктор на должности\n fРуководитель направления урологии, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2024-12-16. Дата рождения: 1989-10-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фкоеева Алина Владимировна'}, {'from': 'gpt', 'value': 'Фкоеева Алина Владимировна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2023-07-17. Дата рождения: 1998-11-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фимон Роман Викторович'}, {'from': 'gpt', 'value': 'Фимон Роман Викторович работает в организации Волшебный Доктор на должности\n fДиректор департамента ИТ, в филиале Управление (Департамент ИТ, Департамент ИТ). Дата начала работы в организации: 2021-04-05. Дата рождения: 1984-03-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фкняоова Анна Ивановна'}, {'from': 'gpt', 'value': 'Фкняоова Анна Ивановна работает в организации Волшебный Доктор на должности\n fВрач-физиотерапевт, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-11-18. Дата рождения: 1966-02-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фуанцрзов Антон Вячеславович'}, {'from': 'gpt', 'value': 'Фуанцрзов Антон Вячеславович работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-04-15. Дата рождения: 1978-07-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фмисран Екатерина Юрьевна'}, {'from': 'gpt', 'value': 'Фмисран Екатерина Юрьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2010-11-19. Дата рождения: 1990-03-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фролов Константин Борисович'}, {'from': 'gpt', 'value': 'Фролов Константин Борисович работает в организации Волшебный Доктор на должности\n fВрач-сердечно-сосудистый хирург, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2014-06-04. Дата рождения: 1971-12-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Флорова Тамара Владимировна'}, {'from': 'gpt', 'value': 'Флорова Тамара Владимировна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф14 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-01. Дата рождения: 1959-08-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фрвлооа Татьяна Александровна'}, {'from': 'gpt', 'value': 'Фрвлооа Татьяна Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2023-06-26. Дата рождения: 1999-10-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фурудй Анастасия Анатольевна'}, {'from': 'gpt', 'value': 'Фурудй Анастасия Анатольевна работает в организации Волшебный Доктор на должности\n fКлинический психолог, в филиале Ф02 (Центр психического здоровья, Центр психического здоровья). Дата начала работы в организации: 2024-09-24. Дата рождения: 1999-02-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фунмарова Зоя Леонидовна'}, {'from': 'gpt', 'value': 'Фунмарова Зоя Леонидовна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2020-10-27. Дата рождения: 1971-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Фувсора Юлия Викторовна'}, {'from': 'gpt', 'value': 'Фувсора Юлия Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2025-01-20. Дата рождения: 1971-01-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хашыбев Андрей Александрович'}, {'from': 'gpt', 'value': 'Хашыбев Андрей Александрович работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале ГЦ-Ф15 (Отделение лучевой диагностики, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-09-09. Дата рождения: 1981-08-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хактеав Мингиян Сергеевич'}, {'from': 'gpt', 'value': 'Хактеав Мингиян Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф12 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2020-12-17. Дата рождения: 1996-07-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Халаксная Анастасия Александровна'}, {'from': 'gpt', 'value': 'Халаксная Анастасия Александровна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2019-10-29. Дата рождения: 1977-06-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хаеелва Юлия Владимировна'}, {'from': 'gpt', 'value': 'Хаеелва Юлия Владимировна работает в организации Волшебный Доктор на должности\n fНачальник отдела дистанционного обучения, в филиале Управление (Учебный центр/Отдел дистанционного обучения, Департамент по работе с персоналом). Дата начала работы в организации: 2022-03-14. Дата рождения: 1993-09-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хкндожао Юлия Владимировна'}, {'from': 'gpt', 'value': 'Хкндожао Юлия Владимировна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф16 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2022-04-25. Дата рождения: 1996-04-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хаотаннва Яна Александровна'}, {'from': 'gpt', 'value': 'Хаотаннва Яна Александровна работает в организации Волшебный Доктор на должности\n fВрач-координатор сменный, в филиале ГЦ-Ф15 (Медицинский пульт, Коммерческий департамент ГЦ). Дата начала работы в организации: 2023-10-09. Дата рождения: 1986-11-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Харитонов Андрей Иванович'}, {'from': 'gpt', 'value': 'Харитонов Андрей Иванович работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2019-11-18. Дата рождения: 1967-10-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хвритоноаа Полина Александровна'}, {'from': 'gpt', 'value': 'Хвритоноаа Полина Александровна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф06 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2021-09-27. Дата рождения: 1995-09-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Харчалива Наталья Александровна'}, {'from': 'gpt', 'value': 'Харчалива Наталья Александровна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф10 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2015-01-21. Дата рождения: 1978-01-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хаснаова Светлана Асафовна'}, {'from': 'gpt', 'value': 'Хаснаова Светлана Асафовна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-хирург, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2018-10-17. Дата рождения: 1987-05-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хсааншин Мансур Минасхатович'}, {'from': 'gpt', 'value': 'Хсааншин Мансур Минасхатович работает в организации Волшебный Доктор на должности\n fВрач-травматолог-ортопед, в филиале ГЦ-Ф15 (Отделение травматологии и ортопедии, Стационар). Дата начала работы в организации: 2017-09-06. Дата рождения: 1982-12-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ханясова Эльвира Ринатовна'}, {'from': 'gpt', 'value': 'Ханясова Эльвира Ринатовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2021-03-15. Дата рождения: 1998-04-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ханецко Евгений Игоревич'}, {'from': 'gpt', 'value': 'Ханецко Евгений Игоревич работает в организации Волшебный Доктор на должности\n fРуководитель офтальмологического центра, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2024-02-26. Дата рождения: 1990-12-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хаяатрчн Рима Норайровна'}, {'from': 'gpt', 'value': 'Хаяатрчн Рима Норайровна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале Ф11 (Администрация, Администрация). Дата начала работы в организации: 2020-11-03. Дата рождения: 1995-07-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хлюотсва Марина Анатольевна'}, {'from': 'gpt', 'value': 'Хлюотсва Марина Анатольевна работает в организации Волшебный Доктор на должности\n fРуководитель службы сервиса, в филиале ГЦ-Ф15 (Служба сервиса, Стационар). Дата начала работы в организации: 2019-12-02. Дата рождения: 1972-05-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хмеелк Александр Валерьевич'}, {'from': 'gpt', 'value': 'Хмеелк Александр Валерьевич работает в организации Волшебный Доктор на должности\n fВодитель легкового автомобиля, в филиале Управление (Автопарк дирекции, Аппарат дирекции). Дата начала работы в организации: 2012-08-22. Дата рождения: 1971-11-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хныкина Диана Антоновна'}, {'from': 'gpt', 'value': 'Хныкина Диана Антоновна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2021-01-14. Дата рождения: 1999-05-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хкдаоова Ирина Николаевна'}, {'from': 'gpt', 'value': 'Хкдаоова Ирина Николаевна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2020-07-28. Дата рождения: 1975-02-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хоеырдв Евгений Владимирович'}, {'from': 'gpt', 'value': 'Хоеырдв Евгений Владимирович работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф01 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2014-08-04. Дата рождения: 1986-09-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хрдыоев Сергей Анатольевич'}, {'from': 'gpt', 'value': 'Хрдыоев Сергей Анатольевич работает в организации Волшебный Доктор на должности\n fВрач-онколог, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-11-05. Дата рождения: 1974-10-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хосманлких Наталья Сергеевна'}, {'from': 'gpt', 'value': 'Хосманлких Наталья Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф11 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2024-07-08. Дата рождения: 1990-12-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хоолмова Ирина Владимировна'}, {'from': 'gpt', 'value': 'Хоолмова Ирина Владимировна работает в организации Волшебный Доктор на должности\n fВедущий специалист отдела кадров, в филиале Управление (Отдел кадров, Департамент по работе с персоналом). Дата начала работы в организации: 2024-11-25. Дата рождения: 1990-04-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хонемко Татьяна Алексеевна'}, {'from': 'gpt', 'value': 'Хонемко Татьяна Алексеевна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф09 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2013-02-27. Дата рождения: 1978-01-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хонодцкая Елена Михайловна'}, {'from': 'gpt', 'value': 'Хонодцкая Елена Михайловна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2023-12-27. Дата рождения: 1973-07-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хоидоцкнй Александр Александрович'}, {'from': 'gpt', 'value': 'Хоидоцкнй Александр Александрович работает в организации Волшебный Доктор на должности\n fСтарший техник, в филиале Управление (Отдел административно-хозяйственного и технического обеспечения, Административный департамент). Дата начала работы в организации: 2022-12-12. Дата рождения: 1975-03-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хсиртич Артем Максимович'}, {'from': 'gpt', 'value': 'Хсиртич Артем Максимович работает в организации Волшебный Доктор на должности\n fВрач-ортодонт, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2022-09-26. Дата рождения: 1996-07-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хрмоова Наталья Евгеньевна'}, {'from': 'gpt', 'value': 'Хрмоова Наталья Евгеньевна работает в организации Волшебный Доктор на должности\n fВрач-профпатолог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2023-03-06. Дата рождения: 1965-12-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Худраова Виктория Маркленовна'}, {'from': 'gpt', 'value': 'Худраова Виктория Маркленовна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф02 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2022-02-14. Дата рождения: 1992-04-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хувоназарода Марзия Бекназаровна'}, {'from': 'gpt', 'value': 'Хувоназарода Марзия Бекназаровна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2016-11-02. Дата рождения: 1978-12-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хусвноаа Комила Тойиржоновна'}, {'from': 'gpt', 'value': 'Хусвноаа Комила Тойиржоновна работает в организации Волшебный Доктор на должности\n fОперационная медицинская сестра (операционный медицинский брат), в филиале Ф02 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2012-09-05. Дата рождения: 1983-02-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хесунов Далер Олимович'}, {'from': 'gpt', 'value': 'Хесунов Далер Олимович работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2018-08-08. Дата рождения: 1986-05-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Хншеуова Эльмира Хушеновна'}, {'from': 'gpt', 'value': 'Хншеуова Эльмира Хушеновна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2025-01-30. Дата рождения: 2004-02-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цвааа Мария Кандитовна'}, {'from': 'gpt', 'value': 'Цвааа Мария Кандитовна работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф12 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2016-12-14. Дата рождения: 1992-03-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цнаева Елена Анатольевна'}, {'from': 'gpt', 'value': 'Цнаева Елена Анатольевна работает в организации Волшебный Доктор на должности\n fМенеджер стоматологического отделения, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2023-09-28. Дата рождения: 1974-09-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цёрав Алексей Николаевич'}, {'from': 'gpt', 'value': 'Цёрав Алексей Николаевич работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-09-02. Дата рождения: 1981-01-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цаёрва Мария Петровна'}, {'from': 'gpt', 'value': 'Цаёрва Мария Петровна работает в организации Волшебный Доктор на должности\n fВрач-травматолог-ортопед, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2025-03-03. Дата рождения: 1981-05-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цёрава Ольга Юрьевна'}, {'from': 'gpt', 'value': 'Цёрава Ольга Юрьевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Педиатрическое отделение, Стационар). Дата начала работы в организации: 2020-08-07. Дата рождения: 1972-09-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Царкиаев Алан Борисович'}, {'from': 'gpt', 'value': 'Царкиаев Алан Борисович работает в организации Волшебный Доктор на должности\n fВрач-травматолог-ортопед, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2022-04-11. Дата рождения: 1993-08-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цевтаев Иван Алексеевич'}, {'from': 'gpt', 'value': 'Цевтаев Иван Алексеевич работает в организации Волшебный Доктор на должности\n fЗаведующий стоматологическим центром, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2020-06-22. Дата рождения: 1985-01-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цветвока Людмила Сергеевна'}, {'from': 'gpt', 'value': 'Цветвока Людмила Сергеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2024-07-22. Дата рождения: 1988-12-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цевекоба Ирина Улюмджиевна'}, {'from': 'gpt', 'value': 'Цевекоба Ирина Улюмджиевна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2019-07-08. Дата рождения: 1966-04-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цитнира Жанна Эдуардовна'}, {'from': 'gpt', 'value': 'Цитнира Жанна Эдуардовна работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2019-01-09. Дата рождения: 1963-06-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Целоова Лариса Магомет-Гиреевна'}, {'from': 'gpt', 'value': 'Целоова Лариса Магомет-Гиреевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2014-08-06. Дата рождения: 1965-12-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цпурова Оксана Александровна'}, {'from': 'gpt', 'value': 'Цпурова Оксана Александровна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-11-14. Дата рождения: 1980-10-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цицуева Лейла Гивиевна'}, {'from': 'gpt', 'value': 'Цицуева Лейла Гивиевна работает в организации Волшебный Доктор на должности\n fМенеджер по закупкам, в филиале ГЦ-Ф15 (Отдел хозобеспечения, Стационар). Дата начала работы в организации: 2024-04-22. Дата рождения: 1981-06-08.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цбыа Николай Александрович'}, {'from': 'gpt', 'value': 'Цбыа Николай Александрович работает в организации Волшебный Доктор на должности\n fЗаведующий отделением анестезиологии и реанимации, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2024-04-08. Дата рождения: 1984-08-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цоганкыва Ирина Олеговна'}, {'from': 'gpt', 'value': 'Цоганкыва Ирина Олеговна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-11-28. Дата рождения: 2000-08-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Цикын Даниил Сергеевич'}, {'from': 'gpt', 'value': 'Цикын Даниил Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-уролог, в филиале ГЦ-Ф15 (Отделение урологии, Стационар). Дата начала работы в организации: 2022-11-21. Дата рождения: 1995-11-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чанвоа Алёна Георгиевна'}, {'from': 'gpt', 'value': 'Чанвоа Алёна Георгиевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2015-09-14. Дата рождения: 1971-01-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чаваера Кристина Артуровна'}, {'from': 'gpt', 'value': 'Чаваера Кристина Артуровна работает в организации Волшебный Доктор на должности\n fВрач-рентгенолог, в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2025-02-27. Дата рождения: 1998-05-27.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чакшина Людмила Дмитриевна'}, {'from': 'gpt', 'value': 'Чакшина Людмила Дмитриевна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2021-06-28. Дата рождения: 1970-06-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чекнниаа Нонна Александровна'}, {'from': 'gpt', 'value': 'Чекнниаа Нонна Александровна работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2004-03-10. Дата рождения: 1973-02-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чнрепеина Елена Николаевна'}, {'from': 'gpt', 'value': 'Чнрепеина Елена Николаевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Ф16 (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2013-12-19. Дата рождения: 1974-10-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Черквсоаа Виктория Андреевна'}, {'from': 'gpt', 'value': 'Черквсоаа Виктория Андреевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2024-09-23. Дата рождения: 1996-08-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чеакасскря Ася Арсеньевна'}, {'from': 'gpt', 'value': 'Чеакасскря Ася Арсеньевна работает в организации Волшебный Доктор на должности\n fМедицинский лабораторный техник, в филиале Лаборатория (Отделение БИП, Отделение БИП). Дата начала работы в организации: 2011-09-08. Дата рождения: 1985-07-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чкреез Владислав Станиславович'}, {'from': 'gpt', 'value': 'Чкреез Владислав Станиславович работает в организации Волшебный Доктор на должности\n fВрач-невролог, в филиале Ф07 (КДО, КДО). Дата начала работы в организации: 2024-05-27. Дата рождения: 1994-10-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чорнева Оксана Викторовна'}, {'from': 'gpt', 'value': 'Чорнева Оксана Викторовна работает в организации Волшебный Доктор на должности\n fСтаршая медицинская сестра, в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2015-04-08. Дата рождения: 1991-01-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чренова Олеся Георгиевна'}, {'from': 'gpt', 'value': 'Чренова Олеся Георгиевна работает в организации Волшебный Доктор на должности\n fМенеджер интернет-проектов, в филиале Управление (Отдел интернет-проектов, Департамент маркетинга и рекламы). Дата начала работы в организации: 2024-12-02. Дата рождения: 1980-04-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чорноусева Анастасия Игоревна'}, {'from': 'gpt', 'value': 'Чорноусева Анастасия Игоревна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале ГЦ-Ф15 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2022-06-06. Дата рождения: 1998-06-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Черонухова Анна Леонидовна'}, {'from': 'gpt', 'value': 'Черонухова Анна Леонидовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале ГЦ-Ф15 (Терапевтическое отделение, Стационар). Дата начала работы в организации: 2023-09-25. Дата рождения: 1995-01-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чешнырева Ирина Сергеевна'}, {'from': 'gpt', 'value': 'Чешнырева Ирина Сергеевна работает в организации Волшебный Доктор на должности\n fВрач-онколог, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2022-01-10. Дата рождения: 1990-05-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чернышвеа Оксана Павловна'}, {'from': 'gpt', 'value': 'Чернышвеа Оксана Павловна работает в организации Волшебный Доктор на должности\n fПроектировщик-дизайнер интерфейсов, в филиале Управление (Отдел разработки корпоративных систем, Центр разработки ПО). Дата начала работы в организации: 2016-09-22. Дата рождения: 1993-03-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чнреышова Ольга Анатольевна'}, {'from': 'gpt', 'value': 'Чнреышова Ольга Анатольевна работает в организации Волшебный Доктор на должности\n fСпециалист отдела внешнего обучения, оценки и развития, в филиале Управление (Учебный центр/Отдел внешнего обучения, оценки и развития, Департамент по работе с персоналом). Дата начала работы в организации: 2019-12-23. Дата рождения: 1970-03-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чернвяская Ольга Викторовна'}, {'from': 'gpt', 'value': 'Чернвяская Ольга Викторовна работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача по педиатрии и выездной помощи, в филиале Ф10 (Администрация, Администрация). Дата начала работы в организации: 2007-09-24. Дата рождения: 1969-03-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чеенярв Анатолий Анатольевич'}, {'from': 'gpt', 'value': 'Чеенярв Анатолий Анатольевич работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача, в филиале ГЦ-Ф15 (Администрация, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2020-09-14. Дата рождения: 1987-05-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чивлинцега Светлана Андреевна'}, {'from': 'gpt', 'value': 'Чивлинцега Светлана Андреевна работает в организации Волшебный Доктор на должности\n fСистемный аналитик, в филиале Управление (Отдел разработки корпоративных систем, Центр разработки ПО). Дата начала работы в организации: 2023-11-02. Дата рождения: 2001-02-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чжиова Юлия Вячеславовна'}, {'from': 'gpt', 'value': 'Чжиова Юлия Вячеславовна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2018-02-07. Дата рождения: 1982-03-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чикуонва Екатерина Александровна'}, {'from': 'gpt', 'value': 'Чикуонва Екатерина Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф03 (КДО, КДО). Дата начала работы в организации: 2020-08-20. Дата рождения: 1975-03-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чкриова Татьяна Николаевна'}, {'from': 'gpt', 'value': 'Чкриова Татьяна Николаевна работает в организации Волшебный Доктор на должности\n fВрач клинической лабораторной диагностики, в филиале Лаборатория (Отделение ОКИ, Отделение ОКИ). Дата начала работы в организации: 2013-11-13. Дата рождения: 1962-04-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чиствкояа Елена Владимировна'}, {'from': 'gpt', 'value': 'Чиствкояа Елена Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2014-04-07. Дата рождения: 1966-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чкстяиова Ольга Валентиновна'}, {'from': 'gpt', 'value': 'Чкстяиова Ольга Валентиновна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2011-06-02. Дата рождения: 1971-08-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чоилй Полина Григорьевна'}, {'from': 'gpt', 'value': 'Чоилй Полина Григорьевна работает в организации Волшебный Доктор на должности\n fВрач-терапевт участковый, в филиале Ф14 (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-10-02. Дата рождения: 1999-03-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ччбуенко Людмила Юрьевна'}, {'from': 'gpt', 'value': 'Ччбуенко Людмила Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2021-03-15. Дата рождения: 1981-10-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чувеава Любовь Николаевна'}, {'from': 'gpt', 'value': 'Чувеава Любовь Николаевна работает в организации Волшебный Доктор на должности\n fСтарший фельдшер-лаборант, в филиале Лаборатория (Администрация, Администрация). Дата начала работы в организации: 2005-04-22. Дата рождения: 1962-04-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чавушкина Елена Леонидовна'}, {'from': 'gpt', 'value': 'Чавушкина Елена Леонидовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра-анестезист, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2023-09-11. Дата рождения: 1992-10-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чвйкоуа Светлана Андреевна'}, {'from': 'gpt', 'value': 'Чвйкоуа Светлана Андреевна работает в организации Волшебный Доктор на должности\n fРуководитель направления офтальмологии, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2024-11-11. Дата рождения: 1986-07-26.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чвприкоуа Кристина Николаевна'}, {'from': 'gpt', 'value': 'Чвприкоуа Кристина Николаевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф06 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2019-07-04. Дата рождения: 1991-06-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чыпругина Галина Николаевна'}, {'from': 'gpt', 'value': 'Чыпругина Галина Николаевна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф16 (Центр женского здоровья и косметологии, Центр женского здоровья и косметологии). Дата начала работы в организации: 2021-03-01. Дата рождения: 1974-07-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чуарков Павел Валентинович'}, {'from': 'gpt', 'value': 'Чуарков Павел Валентинович работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале ГЦ-Ф15 (Отделение оторинолангологии, Стационар). Дата начала работы в организации: 2018-01-15. Дата рождения: 1980-07-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чуерков Никита Максимович'}, {'from': 'gpt', 'value': 'Чуерков Никита Максимович работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2024-02-19. Дата рождения: 1996-04-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чурюомва Светлана Александровна'}, {'from': 'gpt', 'value': 'Чурюомва Светлана Александровна работает в организации Волшебный Доктор на должности\n fБухгалтер по учету операций обязательного медицинского страхования, в филиале Управление (Отдел учета расходов, Финансовый департамент). Дата начала работы в организации: 2021-10-11. Дата рождения: 1982-03-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чуучев Евгений Станиславович'}, {'from': 'gpt', 'value': 'Чуучев Евгений Станиславович работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача по хирургии, в филиале ГЦ-Ф15 (Администрация, Стационар). Дата начала работы в организации: 2024-09-24. Дата рождения: 1977-11-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Чхалтишвири Илона Отариевна'}, {'from': 'gpt', 'value': 'Чхалтишвири Илона Отариевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Ф05 (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-01-18. Дата рождения: 1977-07-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шлбаинская Ольга Анатольевна'}, {'from': 'gpt', 'value': 'Шлбаинская Ольга Анатольевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2023-08-07. Дата рождения: 1967-10-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шааловскбя Антонина Илларионовна'}, {'from': 'gpt', 'value': 'Шааловскбя Антонина Илларионовна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф01 (АХО, Вспомогательная служба). Дата начала работы в организации: 2025-02-17. Дата рождения: 1950-07-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шадирна Ксения Викторовна'}, {'from': 'gpt', 'value': 'Шадирна Ксения Викторовна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2009-09-29. Дата рождения: 1981-12-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шаинздие Марина Автандиловна'}, {'from': 'gpt', 'value': 'Шаинздие Марина Автандиловна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2022-11-17. Дата рождения: 1988-01-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шоипав Халид Магомедович'}, {'from': 'gpt', 'value': 'Шоипав Халид Магомедович работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф01 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-09-05. Дата рождения: 2001-04-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шаикрова Алия Амангельдиевна'}, {'from': 'gpt', 'value': 'Шаикрова Алия Амангельдиевна работает в организации Волшебный Доктор на должности\n fРуководитель отдела мониторинга и экстренной госпитализации, в филиале ГЦ-Ф15 (Отдел мониторинга и экстренной госпитализации, Коммерческий департамент ГЦ). Дата начала работы в организации: 2007-09-01. Дата рождения: 1971-12-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шглааинова Людмила Дмитриевна'}, {'from': 'gpt', 'value': 'Шглааинова Людмила Дмитриевна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2019-09-19. Дата рождения: 1968-10-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шатома Владислав Александрович'}, {'from': 'gpt', 'value': 'Шатома Владислав Александрович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог, в филиале Ф10 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2018-10-22. Дата рождения: 1996-01-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шамшвтдиноеа Гульнара Максутовна'}, {'from': 'gpt', 'value': 'Шамшвтдиноеа Гульнара Максутовна работает в организации Волшебный Доктор на должности\n fГлавный администратор, в филиале Ф09 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2017-08-15. Дата рождения: 1975-02-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шанаавзова Марьям Шанавазовна'}, {'from': 'gpt', 'value': 'Шанаавзова Марьям Шанавазовна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2019-03-20. Дата рождения: 1980-08-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шанкеив Чингис Альбертович'}, {'from': 'gpt', 'value': 'Шанкеив Чингис Альбертович работает в организации Волшебный Доктор на должности\n fВрач-стоматолог-хирург, в филиале Ф02 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-11-18. Дата рождения: 1996-11-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шаплваоов Александр Ренович'}, {'from': 'gpt', 'value': 'Шаплваоов Александр Ренович работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2015-04-06. Дата рождения: 1962-07-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шапаволова Надежда Александровна'}, {'from': 'gpt', 'value': 'Шапаволова Надежда Александровна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2018-11-19. Дата рождения: 1984-10-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шипошнакова Ольга Александровна'}, {'from': 'gpt', 'value': 'Шипошнакова Ольга Александровна работает в организации Волшебный Доктор на должности\n fВрач функциональной диагностики, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2024-11-05. Дата рождения: 1972-04-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шпраао Антон Сергеевич'}, {'from': 'gpt', 'value': 'Шпраао Антон Сергеевич работает в организации Волшебный Доктор на должности\n fВрач-челюстно-лицевой хирург, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2021-02-01. Дата рождения: 1995-03-20.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шраапова Алена Андреевна'}, {'from': 'gpt', 'value': 'Шраапова Алена Андреевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф15А (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-02-08. Дата рождения: 2000-06-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шаргордоская Анастасия Ивановна'}, {'from': 'gpt', 'value': 'Шаргордоская Анастасия Ивановна работает в организации Волшебный Доктор на должности\n fСанитарка, в филиале ГЦ-Ф15 (ОПЕРБЛОК, Стационар). Дата начала работы в организации: 2016-11-01. Дата рождения: 1966-01-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шорава Марина Олеговна'}, {'from': 'gpt', 'value': 'Шорава Марина Олеговна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2014-12-01. Дата рождения: 1962-05-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шатаолва Юлия Федоровна'}, {'from': 'gpt', 'value': 'Шатаолва Юлия Федоровна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2016-11-09. Дата рождения: 1989-04-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шативола Екатерина Николаевна'}, {'from': 'gpt', 'value': 'Шативола Екатерина Николаевна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2023-12-04. Дата рождения: 1986-11-04.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шаторва Валентина Викторовна'}, {'from': 'gpt', 'value': 'Шаторва Валентина Викторовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по физиотерапии, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2015-09-09. Дата рождения: 1969-08-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шашника Наталья Александровна'}, {'from': 'gpt', 'value': 'Шашника Наталья Александровна работает в организации Волшебный Доктор на должности\n fОператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2024-06-06. Дата рождения: 1984-09-02.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шашоква Татьяна Александровна'}, {'from': 'gpt', 'value': 'Шашоква Татьяна Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2023-06-01. Дата рождения: 1979-04-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шевц Олеся Васильевна'}, {'from': 'gpt', 'value': 'Шевц Олеся Васильевна работает в организации Волшебный Доктор на должности\n fЗаместитель главного врача, в филиале Ф03 (Администрация, Администрация). Дата начала работы в организации: 2015-01-19. Дата рождения: 1978-06-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Швецова Ольга Юрьевна'}, {'from': 'gpt', 'value': 'Швецова Ольга Юрьевна работает в организации Волшебный Доктор на должности\n fВрач-оториноларинголог, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2024-10-14. Дата рождения: 1988-06-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шевкунекно Елена Евгеньевна'}, {'from': 'gpt', 'value': 'Шевкунекно Елена Евгеньевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2018-01-24. Дата рождения: 1967-10-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шевнечко Софья Тимофеевна'}, {'from': 'gpt', 'value': 'Шевнечко Софья Тимофеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф16 (КДО, КДО). Дата начала работы в организации: 2024-02-19. Дата рождения: 2001-11-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шйеко Ирина Владимировна'}, {'from': 'gpt', 'value': 'Шйеко Ирина Владимировна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2019-02-21. Дата рождения: 1968-10-05.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шдлуеченко Татьяна Петровна'}, {'from': 'gpt', 'value': 'Шдлуеченко Татьяна Петровна работает в организации Волшебный Доктор на должности\n fВрач-эксперт-оториноларинголог, в филиале ГЦ-Ф15 (Отделение оторинолангологии, Стационар). Дата начала работы в организации: 2015-12-09. Дата рождения: 1956-10-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шекнер Мария Олеговна'}, {'from': 'gpt', 'value': 'Шекнер Мария Олеговна работает в организации Волшебный Доктор на должности\n fЗаместитель директора медицинского департамента, в филиале Управление (Медицинский департамент, Медицинский департамент). Дата начала работы в организации: 2013-03-11. Дата рождения: 1970-10-24.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шереветома Аминат Рашидовна'}, {'from': 'gpt', 'value': 'Шереветома Аминат Рашидовна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2024-07-22. Дата рождения: 1991-03-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шествкоаа Алина Викторовна'}, {'from': 'gpt', 'value': 'Шествкоаа Алина Викторовна работает в организации Волшебный Доктор на должности\n fМенеджер отдела телемаркетинга, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2016-12-15. Дата рождения: 1986-02-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шихонена Елена Федоровна'}, {'from': 'gpt', 'value': 'Шихонена Елена Федоровна работает в организации Волшебный Доктор на должности\n fГлавный оператор сменный, в филиале Управление (Call центр, Департамент продаж и поддержки клиентов). Дата начала работы в организации: 2009-01-28. Дата рождения: 1972-03-18.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шолив Эдуард Анатольевич'}, {'from': 'gpt', 'value': 'Шолив Эдуард Анатольевич работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф06 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2020-08-17. Дата рождения: 1966-06-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шивола Ирина Борисовна'}, {'from': 'gpt', 'value': 'Шивола Ирина Борисовна работает в организации Волшебный Доктор на должности\n fВрач-аллерголог-иммунолог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2021-03-09. Дата рождения: 1966-07-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шмичук Дарья Сергеевна'}, {'from': 'gpt', 'value': 'Шмичук Дарья Сергеевна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф05 (Отделение лучевой диагностики, Отделение лучевой диагностики). Дата начала работы в организации: 2019-03-13. Дата рождения: 1987-12-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Широков Юрий Михайлович'}, {'from': 'gpt', 'value': 'Широков Юрий Михайлович работает в организации Волшебный Доктор на должности\n fВрач-колопроктолог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2025-03-03. Дата рождения: 1973-10-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шлмёина Ольга Викторовна'}, {'from': 'gpt', 'value': 'Шлмёина Ольга Викторовна работает в организации Волшебный Доктор на должности\n fКоординатор сервисных работ, в филиале ГЦ-Ф15 (Служба сервиса, Стационар). Дата начала работы в организации: 2020-01-21. Дата рождения: 1964-03-30.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шмарниа Нина Леонидовна'}, {'from': 'gpt', 'value': 'Шмарниа Нина Леонидовна работает в организации Волшебный Доктор на должности\n fГардеробщик, в филиале Ф10 (АХО, Вспомогательная служба). Дата начала работы в организации: 2024-10-02. Дата рождения: 1950-01-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шлемёва Ирина Евгеньевна'}, {'from': 'gpt', 'value': 'Шлемёва Ирина Евгеньевна работает в организации Волшебный Доктор на должности\n fВрач-акушер-гинеколог, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2021-10-25. Дата рождения: 1971-02-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шьелмкова Елена Александровна'}, {'from': 'gpt', 'value': 'Шьелмкова Елена Александровна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф11 (Отделение ВОП/педиатрии, Отделение ВОП/педиатрии). Дата начала работы в организации: 2019-10-14. Дата рождения: 1994-04-29.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шкапова Олеся Александровна'}, {'from': 'gpt', 'value': 'Шкапова Олеся Александровна работает в организации Волшебный Доктор на должности\n fМенеджер по плановой госпитализации сменный, в филиале ГЦ-Ф15 (Отдел госпитализаций и сервиса, Коммерческий департамент ГЦ). Дата начала работы в организации: 2024-05-13. Дата рождения: 1979-05-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шпвкоаа Полина Константиновна'}, {'from': 'gpt', 'value': 'Шпвкоаа Полина Константиновна работает в организации Волшебный Доктор на должности\n fВрач-психотерапевт, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2019-01-11. Дата рождения: 1989-05-11.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шипс Валерия Владимировна'}, {'from': 'gpt', 'value': 'Шипс Валерия Владимировна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф17-ДЦ (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2024-03-04. Дата рождения: 1994-04-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шубвеаа Дина Александровна'}, {'from': 'gpt', 'value': 'Шубвеаа Дина Александровна работает в организации Волшебный Доктор на должности\n fЭстетист, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2023-03-23. Дата рождения: 1977-05-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шуибн Дмитрий Михайлович'}, {'from': 'gpt', 'value': 'Шуибн Дмитрий Михайлович работает в организации Волшебный Доктор на должности\n fВрач-сердечно-сосудистый хирург, в филиале ГЦ-Ф15 (Отделение сердечно-сосудистой хирургии, Стационар). Дата начала работы в организации: 2023-05-15. Дата рождения: 1985-08-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шубинков Александр Леонидович'}, {'from': 'gpt', 'value': 'Шубинков Александр Леонидович работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале ГЦ-Ф15 (Отделение анестезиологии и реанимации, Стационар). Дата начала работы в организации: 2020-02-03. Дата рождения: 1960-08-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шлвауова Тамара Николаевна'}, {'from': 'gpt', 'value': 'Шлвауова Тамара Николаевна работает в организации Волшебный Доктор на должности\n fВрач-кардиолог, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2025-02-17. Дата рождения: 1986-06-21.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шувраина Ольга Владимировна'}, {'from': 'gpt', 'value': 'Шувраина Ольга Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф11 (КДО, КДО). Дата начала работы в организации: 2013-10-28. Дата рождения: 1971-03-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шркюуова Нигар Хейирбековна'}, {'from': 'gpt', 'value': 'Шркюуова Нигар Хейирбековна работает в организации Волшебный Доктор на должности\n fВрач-гастроэнтеролог, в филиале Ф09 (КДО, КДО). Дата начала работы в организации: 2025-03-05. Дата рождения: 1997-07-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шлубаева Ольга Петровна'}, {'from': 'gpt', 'value': 'Шлубаева Ольга Петровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра по массажу, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2024-12-12. Дата рождения: 1974-09-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шилука Денис Александрович'}, {'from': 'gpt', 'value': 'Шилука Денис Александрович работает в организации Волшебный Доктор на должности\n fВрач-эндоскопист, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2025-03-03. Дата рождения: 1993-06-17.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шугьла Елизавета Борисовна'}, {'from': 'gpt', 'value': 'Шугьла Елизавета Борисовна работает в организации Волшебный Доктор на должности\n fВрач-колопроктолог, в филиале Ф01 (КДО, КДО). Дата начала работы в организации: 2024-09-02. Дата рождения: 1998-12-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Шувома Елена Николаевна'}, {'from': 'gpt', 'value': 'Шувома Елена Николаевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф14 (КДО, КДО). Дата начала работы в организации: 2023-08-24. Дата рождения: 1977-05-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Щкдьао Андрей Александрович'}, {'from': 'gpt', 'value': 'Щкдьао Андрей Александрович работает в организации Волшебный Доктор на должности\n fВедущий врач-травматолог-ортопед, в филиале ГЦ-Ф15 (Отделение травматологии и ортопедии, Стационар). Дата начала работы в организации: 2022-06-14. Дата рождения: 1981-02-09.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Щеолгва Дарья Андреевна'}, {'from': 'gpt', 'value': 'Щеолгва Дарья Андреевна работает в организации Волшебный Доктор на должности\n fАдминистратор, в филиале Ф05 (Служба поддержки клиентов, Служба Регистратуры и поддержки клиентов). Дата начала работы в организации: 2024-12-02. Дата рождения: 2003-07-15.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Щвглоеа Зоя Павловна'}, {'from': 'gpt', 'value': 'Щвглоеа Зоя Павловна работает в организации Волшебный Доктор на должности\n fВрач-офтальмолог, в филиале ГЦ-Ф15 (Офтальмологический центр, Офтальмологический центр). Дата начала работы в организации: 2017-09-11. Дата рождения: 1990-08-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Щевлога Юлия Андреевна'}, {'from': 'gpt', 'value': 'Щевлога Юлия Андреевна работает в организации Волшебный Доктор на должности\n fЗаведующий клинико-диагностическим отделением, в филиале Ф07 (Администрация, Администрация). Дата начала работы в организации: 2023-07-10. Дата рождения: 1996-09-01.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Щеркабова Наталия Геннадиевна'}, {'from': 'gpt', 'value': 'Щеркабова Наталия Геннадиевна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2024-10-01. Дата рождения: 1989-09-13.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Щуника Наталья Николаевна'}, {'from': 'gpt', 'value': 'Щуника Наталья Николаевна работает в организации Волшебный Доктор на должности\n fРентгенолаборант, в филиале Ф10 (КДО, КДО). Дата начала работы в организации: 2014-04-16. Дата рождения: 1963-02-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Щуверич Татьяна Владимировна'}, {'from': 'gpt', 'value': 'Щуверич Татьяна Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2025-02-24. Дата рождения: 1972-03-28.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Эвматора Гулшайир Толоновна'}, {'from': 'gpt', 'value': 'Эвматора Гулшайир Толоновна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2023-09-21. Дата рождения: 1989-01-03.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Юднекова Вера Борисовна'}, {'from': 'gpt', 'value': 'Юднекова Вера Борисовна работает в организации Волшебный Доктор на должности\n fВрач-терапевт, в филиале Ф01 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2011-12-06. Дата рождения: 1953-05-16.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Юеенва Динара Рашидовна'}, {'from': 'gpt', 'value': 'Юеенва Динара Рашидовна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф12 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-10-30. Дата рождения: 2002-11-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Юинсова Юлия Абакаровна'}, {'from': 'gpt', 'value': 'Юинсова Юлия Абакаровна работает в организации Волшебный Доктор на должности\n fВрач-дерматовенеролог, в филиале ГЦ-Ф15 (Центр косметологии и пластической хирургии, Стационар). Дата начала работы в организации: 2021-03-22. Дата рождения: 1992-04-22.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Юирн Андрей Алексеевич'}, {'from': 'gpt', 'value': 'Юирн Андрей Алексеевич работает в организации Волшебный Доктор на должности\n fВедущий врач-хирург, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-11-05. Дата рождения: 1986-09-10.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Юокрва Юлия Сергеевна'}, {'from': 'gpt', 'value': 'Юокрва Юлия Сергеевна работает в организации Волшебный Доктор на должности\n fПомощник менеджера стоматологического центра, в филиале Управление (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2024-10-17. Дата рождения: 1982-10-07.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Юрчнеко Екатерина Александровна'}, {'from': 'gpt', 'value': 'Юрчнеко Екатерина Александровна работает в организации Волшебный Доктор на должности\n fВрач-эндокринолог, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2025-01-27. Дата рождения: 1997-07-23.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Ювоса Юлия Витальевна'}, {'from': 'gpt', 'value': 'Ювоса Юлия Витальевна работает в организации Волшебный Доктор на должности\n fВрач-педиатр участковый, в филиале Ф17-ДЦ (Выездная помощь, Выездная помощь). Дата начала работы в организации: 2025-02-17. Дата рождения: 1995-06-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Юшквоа Ирина Владимировна'}, {'from': 'gpt', 'value': 'Юшквоа Ирина Владимировна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф05 (Стоматологический центр, Стоматологический центр). Дата начала работы в организации: 2024-05-27. Дата рождения: 1995-08-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Яогвкина Ирина Александровна'}, {'from': 'gpt', 'value': 'Яогвкина Ирина Александровна работает в организации Волшебный Доктор на должности\n fВрач ультразвуковой диагностики, в филиале Ф02 (КДО, КДО). Дата начала работы в организации: 2023-12-18. Дата рождения: 1964-01-12.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Яоквлева Екатерина Николаевна'}, {'from': 'gpt', 'value': 'Яоквлева Екатерина Николаевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале ГЦ-Ф15 (КДО, ПОЛИКЛИНИКА №15). Дата начала работы в организации: 2018-07-09. Дата рождения: 1974-03-31.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Яковлвеа Надежда Геннадьевна'}, {'from': 'gpt', 'value': 'Яковлвеа Надежда Геннадьевна работает в организации Волшебный Доктор на должности\n fВрач общей практики (семейный врач), в филиале Ф01 (Отделение ВОП, Отделение ВОП). Дата начала работы в организации: 2020-01-20. Дата рождения: 1979-09-25.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Янзурагова Елена Викторовна'}, {'from': 'gpt', 'value': 'Янзурагова Елена Викторовна работает в организации Волшебный Доктор на должности\n fВрач-хирург, в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2024-10-07. Дата рождения: 1984-02-14.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Янчекно Сергей Владимирович'}, {'from': 'gpt', 'value': 'Янчекно Сергей Владимирович работает в организации Волшебный Доктор на должности\n fМенеджер отдела по работе с физическими лицами, в филиале Ф11 (Центр продаж физическим лицам, Коммерческий департамент). Дата начала работы в организации: 2021-10-04. Дата рождения: 1983-09-19.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Яхьвеяа Гюльпери Рустамхановна'}, {'from': 'gpt', 'value': 'Яхьвеяа Гюльпери Рустамхановна работает в организации Волшебный Доктор на должности\n fМедицинская сестра стоматологического кабинета, в филиале Ф10 (Стоматологическое отделение, Стоматологическое отделение). Дата начала работы в организации: 2024-07-15. Дата рождения: 2003-07-06.'}], [{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Яшниа Екатерина Александровна'}, {'from': 'gpt', 'value': 'Яшниа Екатерина Александровна работает в организации Волшебный Доктор на должности\n fМедицинская сестра палатная (постовая), в филиале ГЦ-Ф15 (Хирургическое отделение, Стационар). Дата начала работы в организации: 2022-03-01. Дата рождения: 1979-04-23.'}]]

data = {
    'conversations': dataset_conversations,
    'source':['infini-instruct-top-500k']*len(dataset_conversations),
    'score':[0.5]*len(dataset_conversations),
}
dataset = Dataset.from_dict(data)
print('ff')
print(dataset[1]['conversations'])

ff
[{'from': 'system', 'value': 'You are an assistant'}, {'from': 'human', 'value': 'Расскажи про сотрудника Абоясбва Валентина Алексеевна'}, {'from': 'gpt', 'value': 'Абоясбва Валентина Алексеевна работает в организации Волшебный Доктор на должности\n fМедицинская сестра, в филиале Ф12 (КДО, КДО). Дата начала работы в организации: 2015-12-09. Дата рождения: 1970-05-14.'}]


We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [12]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/1492 [00:00<?, ? examples/s]

Map:   0%|          | 0/1492 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [13]:
dataset[5]["conversations"]

[{'content': 'You are an assistant', 'role': 'system'},
 {'content': 'Расскажи про сотрудника Абдвлоуа Ляйсан Рустамовна',
  'role': 'user'},
 {'content': 'Абдвлоуа Ляйсан Рустамовна работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2022-09-19. Дата рождения: 1995-04-23.',
  'role': 'assistant'}]

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [14]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nРасскажи про сотрудника Абдвлоуа Ляйсан Рустамовна<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nАбдвлоуа Ляйсан Рустамовна работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2022-09-19. Дата рождения: 1995-04-23.<|eot_id|>'

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Tokenizing to ["text"] (num_proc=2):   0%|          | 0/1492 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [16]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/1492 [00:00<?, ? examples/s]

We verify masking is actually done:

In [17]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nРасскажи про сотрудника Абдвлоуа Ляйсан Рустамовна<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nАбдвлоуа Ляйсан Рустамовна работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2022-09-19. Дата рождения: 1995-04-23.<|eot_id|>'

In [18]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                             \n\nАбдвлоуа Ляйсан Рустамовна работает в организации Волшебный Доктор на должности\n fВрач-психиатр, в филиале Ф05 (КДО, КДО). Дата начала работы в организации: 2022-09-19. Дата рождения: 1995-04-23.<|eot_id|>'

We can see the System and Instruction prompts are successfully masked!

In [19]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.959 GB of memory reserved.


In [20]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,492 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/1,865,526,272 (1.30% trained)


Step,Training Loss
1,3.383300
2,3.243500
3,3.139300
4,3.098000
5,2.741700
6,2.534100
7,2.046300
8,1.872600
9,1.702200
10,1.429000


In [21]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

146.346 seconds used for training.
2.44 minutes used for training.
Peak reserved memory = 4.959 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 33.641 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [30]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "В каком филиале работает Влдаимиров Александр Андреевич?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                         temperature = 1.0, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nВ каком филиале работает Влдаимиров Александр Андреевич?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nВлдаимиров Александр Андреевич работает в организации Волшебный Доктор на должности\n fВрач-анестезиолог-реаниматолог, в филиале Ф06 (КДО, КДО). Дата начала работы в организации: 2019-10-14. Дата рождения: 1980-06-08.<|eot_id|>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding numbers. 

The sequence you provided was: 1, 1, 2, 3, 5, 8, 13

The next number in the sequence would be 21, which is 8 + 13. The sequence continues as: 21, 34, 55, 89, 144, 233.<|eot_id|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower, located in the heart of Paris, stands tall among the city's historic and cultural landmarks. This iron structure, standing at an impressive 324 meters high, offers breathtaking views of the City of Light's iconic landscape. The Eiffel Tower was built for the 1889 World's Fair and has since become a symbol of French engineering and culture.<|eot_id|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
